# <div align=center> Analyzing the Supply Chain Network </div>
# <div align=center> Created from the Conventional Relation Extraction Model and Chat-GPT: </div>
# <div align=center> Focusing On S&P 500 Companies </div>
###  <div align=center> Jeong Yeo(faiiry9@kaist.ac.kr); Jaejun Kim(jaejun.kim305@gmail.com); Ilju Park(dlfwnqkr12@gmail.com); Il-chul Moon(icmoon@kaist.ac.kr)<br/> </div>
본 코드는 ChatGPT와 SSAN 모델을 활용하여 S&P 500 기업들의 10K 리포트로부터 relation extraction한 데이터를 활용하여 supply chain network를 구축하는 과정 및 결과를 보여줍니다.
### SSAN Model Relation Extraction 과정
1) 문서내의 개체간의 종속성을 추출합니다.
2) 종속성을 모델링하기 위해 각 층에 self-attention을 활용합니다.
3) transformation module을 이용하여 attentive bias를 생성합니다.
* Binary relation prediction:
$$ P(r|e_s, e_o) = \text{sigmoid}(e_s^T W_r e_o), \text{ where } e_s \in \mathbb{R}^{d_e}, W_r \in \mathbb{R}^{d_e d_e} $$

* Cross Entropy Loss:
$$ L = \sum_{<s,o>}\sum_{r} \text{CrossEntropy}(P(r|e_s, e_o), y_r(e_s, e_o))\text{, where y = target label}$$

#### SSAN Architecture
<img src = 'https://raw.githubusercontent.com/BenfengXu/SSAN/master/SSAN.png' alt = 'SSAN' width = '800'/>

Xu, B., Wang, Q., Lyu, Y., Zhu, Y., & Mao, Z. (2021). Entity structure within and throughout: Modeling mention dependencies for document-level relation extraction. Proceedin gs of the AAAI Conference on Artificial Intelligence, 35(16), 14149–14157. doi:10.1609/aaai.v35i16.17665

### GPT Relation Extraction 과정
1) 문서 전체를 처리하여 문맥, 객체 및 관계를 이해합니다.
2) 객체와 관계의 언어적 패턴을 식별합니다.
3) 사전 학습된 지식을 바탕으로 관계를 추론합니다.

# 0. Initial Setting

In [ ]:
import pickle
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
from tabulate import tabulate
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib_venn import venn3, venn2
from tqdm import tqdm
from collections import defaultdict, Counter
import seaborn as sns
from community import community_louvain
import dataframe_image as dfi
import random
from datasets import load_dataset
pd.set_option('display.max_colwidth', 200)
%matplotlib inline

# 1. Dataload
본 섹션에서는 SSAN과 GPT로 추출한 entity와 relation의 데이터프레임을 불러오고 관계 그래프를 시각화합니다.

In [ ]:
def making_DF(csv_name, company):
    df = pd.read_csv('{}.csv'.format(csv_name))#.iloc[:,1:]
    #print(df.isna().sum())

    df = df.dropna(subset=['num_para'], how='any', axis=0)
    #print(df.isna().sum())

    gpt_rel = df[df['relations'].notnull()]
    gpt_rel = gpt_rel.reset_index(drop=True)

    #print('=======DATA LOAD DONE========')


    # 한 문단 당 relations들 한 리스트로 만들기 i.e.['1. Company @ strategic alliance @ None', '2. Company @ Caterpillar Inc. @ strategic alliance', ..]
    rel_li = []
    rel_index_li = []
    for idx, rel in enumerate(gpt_rel['relations']):
        if '\n' in rel:
            rel_li.append(rel.split('\n'))
            #print(rel.split('\n'))
            for i in range(len(rel.split('\n'))):
                rel_index_li.append(idx)
                #print(idx)
            #print('='*20)
        else:
            if '1. (' in rel:
                rel_li.append(rel)
                #print(rel)
                #print('='*20)
                rel_index_li.append(idx)

    #print('=======RELs to a LIST DONE========')


    # 한 리스트로 만들기
    import itertools

    def from_iterable(iterables):
        for it in iterables:
            for element in it:
                yield element

    rel_li = list(itertools.chain(*rel_li)) 

    #print('=======MAKING ONE REL LIST DONE========')

    # 넘버링 떼기
    rels_li = []
    idx_li = []
    for idx, rel in enumerate(rel_li):
        if rel[1:3]== '. ':
            if rel[3]=='(':
                rels_li.append(rel[4:-1])
                idx_li.append(rel_index_li[idx])
            else:
                rels_li.append(rel[3:])
                idx_li.append(rel_index_li[idx])
        elif rel[2:4]== '. ':
            if rel[4]=='(':
                rels_li.append(rel[5:-1])
                idx_li.append(rel_index_li[idx])
            else:
                rels_li.append(rel[4:])
                idx_li.append(rel_index_li[idx])
    print('len of idx and rel:', len(idx_li), len(rels_li))
    #print('=======REMOVING NUMBERING DONE========')

    # @ 기준으로 h,t,r로 나누기
    r_li = []
    para_li = []
    for rel, idx in zip(rels_li,idx_li):
        if len(rel.split(' @ '))==3:
            if 'entity' not in rel.split(' @ ')[0] and 'entity' not in rel.split(' @ ')[1]:
                r_li.append(rel.split(' @ '))
                para_li.append(gpt_rel['para'].iloc[idx][:-1])
        # else:
            # print(rel.split(' @ '))
    #print('=======DEVIDE h,t,r DONE========')



    #================================================

    # df로 만들기
    kg_df = pd.DataFrame(r_li, columns = ['source','target','edge'])
    kg_df['para'] = para_li
    kg_df = kg_df.drop_duplicates()#keep='last')

    #print('=======MAKING DF DONE========')

    # edge = none인 행 제거
    nem_df = kg_df.dropna(subset=['edge'], how='any', axis=0)
    # print(nem_df.isna().sum())

    none_row = nem_df[nem_df['source']=='None'].index
    NEM_df = nem_df.drop(none_row, inplace=False)
    nem_df  = NEM_df.reset_index(drop=True)

    none_row = nem_df[nem_df['target']=='None'].index
    NEM_df = nem_df.drop(none_row, inplace=False)
    nem_df  = NEM_df.reset_index(drop=True)

    none_row = nem_df[nem_df['edge']=='None'].index
    NEM_df = nem_df.drop(none_row, inplace=False)
    NEM_df  = NEM_df.reset_index(drop=True)

    #print('=======REMOVING ROW: None in h,t,r DONE========')

    # # entity라고 나온거 제거
    # given_rel = ["head of government", "country", "place of birth", "place of death", "father", "mother", "spouse", "country of citizenship", "continent", "instance of", "head of state", "capital", "official language", "position held", "child", "author", "member of sports team", "director", "screenwriter", "educated at", "composer", "member of political party", "employer", "founded by", "league", "publisher", "owned by", "located in the administrative territorial entity", "genre", "operator", "religion", "contains administrative territorial entity", "follows", "followed by", "headquarters location", "cast member", "producer", "award received", "creator", "parent taxon", "ethnic group", "performer", "manufacturer", "developer", "series", "sister city", "legislative body", "basin country", "located in or next to body of water", "military branch", "record label", "production company", "location", "subclass of", "subsidiary", "part of", "original language of work", "platform", "mouth of the watercourse", "original network", "member of", "chairperson", "country of origin", "has part", "residence", "date of birth", "date of death", "inception", "dissolved, abolished or demolished", "publication date", "start time", "end time", "point in time", "conflict", "characters", "lyrics by", "located on terrain feature", "participant", "influenced by", "location of formation", "parent organization", "notable work", "separated from", "narrative location", "work location", "applies to jurisdiction", "product or material produced", "unemployment rate", "territory claimed by", "participant of", "replaces", "replaced by", "capital of", "languages spoken, written or signed", "present in work", "sibling"]
    # given_rel_row = []

    # for i in range(len(NEM_df)):
    #     if any(g_rel == NEM_df['edge'][i] for g_rel in given_rel):
    #         given_rel_row.append([NEM_df['source'][i],NEM_df['target'][i],NEM_df['edge'][i]])
    # #print('=======REMOVING ROW: None in h,t,r DONE========')

    # given rel에 존재하는 데이터만 저장
    given_rel = ["head of government", "country", "place of birth", "place of death", "father", "mother", "spouse", "country of citizenship", "continent", "instance of", "head of state", "capital", "official language", "position held", "child", "author", "member of sports team", "director", "screenwriter", "educated at", "composer", "member of political party", "employer", "founded by", "league", "publisher", "owned by", "located in the administrative territorial entity", "genre", "operator", "religion", "contains administrative territorial entity", "follows", "followed by", "headquarters location", "cast member", "producer", "award received", "creator", "parent taxon", "ethnic group", "performer", "manufacturer", "developer", "series", "sister city", "legislative body", "basin country", "located in or next to body of water", "military branch", "record label", "production company", "location", "subclass of", "subsidiary", "part of", "original language of work", "platform", "mouth of the watercourse", "original network", "member of", "chairperson", "country of origin", "has part", "residence", "date of birth", "date of death", "inception", "dissolved, abolished or demolished", "publication date", "start time", "end time", "point in time", "conflict", "characters", "lyrics by", "located on terrain feature", "participant", "influenced by", "location of formation", "parent organization", "notable work", "separated from", "narrative location", "work location", "applies to jurisdiction", "product or material produced", "unemployment rate", "territory claimed by", "participant of", "replaces", "replaced by", "capital of", "languages spoken, written or signed", "present in work", "sibling"]
    given_rel_row = []

    for i in range(len(NEM_df)):
        if any(g_rel == NEM_df['edge'][i] for g_rel in given_rel):
            given_rel_row.append([NEM_df['para'][i],NEM_df['source'][i],NEM_df['target'][i],NEM_df['edge'][i]])

    final_DF2 = pd.DataFrame(given_rel_row, columns = ['para','source','target','relation'])

    final_DF = final_DF2[['source','target','relation']]


    #print('=======EXTRACTING JUST GIVEN REL DONE========')

    # 저장 
    final_DF.to_csv('./{}_DF1.csv'.format(company))
    #print('=======SAVE AS DF DONE========')
    final_DF2.to_excel(excel_writer='./{}.xlsx'.format(company)) 
    #dfi.export(final_DF, './RESULT/{}_RE_By_{}.png'.format(company, 'GPT'), max_cols=-1, max_rows=-1)

    return final_DF2

In [ ]:
# S&P500 기업들의 이름과 티커를 불러옵니다.
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
table = pd.read_html(url)
sp500_df = table[0]

# 티커와 회사명을 딕셔너리로 매핑
ticker_to_name = pd.Series(sp500_df.Security.values, index=sp500_df.Symbol).to_dict()


In [ ]:
# example: first 10 companies
for ticker, name in list(ticker_to_name.items())[:10]:
    print(f'{ticker}: {name}')

In [ ]:
# 추출된 entity와 관계의 데이터프레임을 불러옵니다.
gpt_tag = pd.read_csv('GPT_SP_DF_TAG.csv')
gpt_tag.head(5)

* para = pragraph
* source = entity
* target = entity
* relation = relation
* source_ner, target_ner = ner
* sorted_pair
* sorted_pair = entity pair

In [ ]:
# number of relations
len(gpt_tag['relation'].unique())

총 96개의 관계가 있습니다.
### 각 관계 수 별 bar chart

In [ ]:
# number of each relations
edge_counts = gpt_tag['relation'].value_counts()

# bar chart of relation type counts
plt.figure(figsize=(20, 18))  # Adjust the size as needed
plt.barh(edge_counts.index, edge_counts.values, color='slategray') 
plt.xlabel('Count')
plt.ylabel('Relation Type')
plt.title('Relation Type Counts in Descending Order')
plt.gca().invert_yaxis()  # To display the highest count at the top
plt.show()

In [ ]:
# 데이터를 불러옵니다.
gpt_just = pd.read_csv('GPT_Just_S_T.csv')
gpt_just.head(5)

* para = paragraph
* source = entity
* target = entity
* relation = relation
* source_ner, target_ner = ner
* company = company

In [ ]:
# Creating a directed graph
G = nx.DiGraph()

# Adding edges from the DataFrame
for _, row in gpt_just.iterrows():
    G.add_edge(row['source'], row['target'], edge=row['relation'])
    
    # Calculate centrality measures (e.g., degree centrality)
centrality = nx.degree_centrality(G)

# Preparing to categorize nodes by their NER tags
node_categories = defaultdict(list)

# source와 target을 딕셔너리로 변환
source_dict = gpt_just.set_index('source')['source_ner'].to_dict()
target_dict = gpt_just.set_index('target')['target_ner'].to_dict()

for node in G.nodes:
    # 딕셔너리를 사용하여 빠르게 노드 카테고리 찾기
    if node in source_dict:
        category = source_dict[node]
    elif node in target_dict:
        category = target_dict[node]
    else:
        category = 'Unknown'
    
    node_categories[category].append(node)
    
# # Sorting nodes in each category by centrality
top_centrality_by_category = {category: sorted(nodes, key=lambda x: centrality[x], reverse=True)[:10]
                              for category, nodes in node_categories.items()}

# top_centrality_by_category
# Converting the top_centrality_by_category dictionary to a DataFrame
df_top_centrality = pd.DataFrame([{"Category": category, "Node": node, "Centrality": centrality[node]}
                                  for category, nodes in top_centrality_by_category.items()
                                  for node in nodes])

df_top_centrality.head(5)

### source_node별 subgraph
* source_node = 'Tesla, Inc.'

In [ ]:
# computes the shortest path to all nodes reachable from the source_node
# (node: length)
source_node = 'Tesla, Inc.'
reachable_nodes = nx.single_source_shortest_path_length(G, source_node)
list(reachable_nodes.items())[:10]

source_node를 'Tesla, Inc.'로 설정했을때 첫 10개의 reachable_nodes입니다.

In [ ]:
print(len(list(reachable_nodes.keys())))
# first 100 reachable nodes
first_100_reachable_nodes = dict(list(reachable_nodes.items())[:100])

'Tesla, Inc.'에는 190269개의 reachable nodes가 있고, 그래프를 그리기 위해 100개의 노드만 선택했습니다.

In [ ]:
# Extract the subgraph of reachable nodes
subgraph = G.subgraph(first_100_reachable_nodes.keys())

# Use the Kamada-Kawai layout for a hierarchical structure
pos = nx.kamada_kawai_layout(subgraph)

# Draw the subgraph with Kamada-Kawai layout
plt.figure(figsize=(15, 15))
nx.draw(subgraph, pos, with_labels=True, node_size=20, font_size=8, alpha=0.7, arrows=True)
plt.title(f"Hierarchical View of Reachable Nodes from {source_node}")
plt.show()

아래의 코드는 전체 reachable_nodes에 대한 그래프 입니다. 연산량이 많아 주석처리 되어있습니다.

In [ ]:
"""
# heavy computation
# source_node can be replaced with other nodes
source_node = 'Tesla, Inc.'
reachable_nodes = nx.single_source_shortest_path_length(G, source_node)

# Extract the subgraph of reachable nodes
subgraph = G.subgraph(reachable_nodes.keys())

# Use the Kamada-Kawai layout for a hierarchical structure
pos = nx.kamada_kawai_layout(subgraph)

# Draw the subgraph with Kamada-Kawai layout
plt.figure(figsize=(15, 15))
nx.draw(subgraph, pos, with_labels=True, node_size=20, font_size=8, alpha=0.7, arrows=True)
plt.title(f"Hierarchical View of Reachable Nodes from {source_node}")
plt.show()
"""

연산량이 많아 100개의 노드만 선택하여 그래프를 구축하였습니다.
* source_node = 'lithium'

In [ ]:
# example with lithium
source_node = 'lithium'
reachable_nodes = nx.single_source_shortest_path_length(G, source_node)
# first 100 reachable nodes
first_100_reachable_nodes = dict(list(reachable_nodes.items())[:100])

# Extract the subgraph of reachable nodes
subgraph = G.subgraph(first_100_reachable_nodes.keys())

# Use the Kamada-Kawai layout for a hierarchical structure
pos = nx.kamada_kawai_layout(subgraph)

# Draw the subgraph with Kamada-Kawai layout
plt.figure(figsize=(15, 15))
nx.draw(subgraph, pos, with_labels=True, node_size=20, font_size=8, alpha=0.7, arrows=True)
plt.title(f"Hierarchical View of Reachable Nodes from {source_node}")
plt.show()

#### source_node와 target_node별 최단 path를 보여줍니다.

In [ ]:
# 소스노드와 타겟노드 사이의 최단 path를 보여줍니다.
# find and display the shortest path from one specific node to another in a network
source_node = 'lithium'
# computes the shortest path to all reachable nodes
shortest_paths = nx.single_source_shortest_path(G, source_node)

# can be replaced with other target nodes
target_node = 'Tesla, Inc.'
# target_node is in the keys of the shortest_paths, 
if target_node in shortest_paths:
    path = shortest_paths[target_node]
    print(f"The shortest path from {source_node} to {target_node} is:")
    print(path)
else:
    print(f"There is no path from {source_node} to {target_node}.")

In [ ]:
# when the target_node is replaced from Tesla, Inc. to Tesla
target_node = 'Tesla'
if target_node in shortest_paths:
    path = shortest_paths[target_node]
    print(f"The shortest path from {source_node} to {target_node} is:")
    print(path)
else:
    print(f"There is no path from {source_node} to {target_node}.")

In [ ]:
# undirected graph
bi_G = nx.Graph()

for _, row in gpt_just.iterrows():
    bi_G.add_edge(row['source'], row['target'], edge=row['relation'])

In [ ]:
source_node = 'lithium'
# compute the shortest path length from the source node to other nodes
reachable_nodes = nx.single_source_shortest_path_length(bi_G, source_node)
# shortest path key = target nodes, valeus = shortest path from the source node to target nodes
shortest_paths = nx.single_source_shortest_path(bi_G, source_node)

# target_node can be replaced with other nodes
target_node = 'Tesla'
# if target_node is present in shortest_paths
if target_node in shortest_paths:
    # retrieves the shortest path from source_node to target_node
    path = shortest_paths[target_node]
    print(f"The shortest path from {source_node} to {target_node} is:")
    print(path)
else:
    print(f"There is no path from {source_node} to {target_node}.")

In [ ]:
target_node = 'Tesla, Inc.'  # 'target'을 원하는 목표 노드의 실제 이름으로 바꾸세요.
if target_node in shortest_paths:
    path = shortest_paths[target_node]
    print(f"The shortest path from {source_node} to {target_node} is:")
    print(path)
else:
    print(f"There is no path from {source_node} to {target_node}.")

Undirected Graph를 통해 시각화합니다.
* source_node = 'lithium'

In [ ]:
source_node = 'lithium'
# compute the shortest path length from the source node to other nodes
reachable_nodes = nx.single_source_shortest_path_length(bi_G, source_node)
print(len(list(reachable_nodes.keys())))
# first 100 reachable nodes
first_100_reachable_nodes = dict(list(reachable_nodes.items())[:100])

총 268602개의 reachable_nodes가 있고 연산량이 많아 100개의 노드만을 선택하여 그래프를 구축하였습니다.

In [ ]:
#  도달 가능한 노드만을 포함하는 서브그래프를 추출합니다.
# replaced G with bi_G
subgraph = bi_G.subgraph(first_100_reachable_nodes.keys())

# Kamada-Kawai 레이아웃을 사용하여 서브그래프의 노드 위치를 계산합니다.
pos = nx.kamada_kawai_layout(subgraph)

# 서브그래프를 시각화합니다.
plt.figure(figsize=(15, 15))
nx.draw(subgraph, pos, with_labels=True, node_size=50, font_size=8, alpha=0.7, arrows=True)
plt.title(f"Graph Visualization from '{source_node}'")
plt.show()

아래의 코드는 전체 reachable_nodes에 대한 그래프 입니다. 연산량이 많아 주석처리 되어있습니다.

In [ ]:
"""
# heavy computation
#  도달 가능한 노드만을 포함하는 서브그래프를 추출합니다.
subgraph = G.subgraph(reachable_nodes.keys())

# Kamada-Kawai 레이아웃을 사용하여 서브그래프의 노드 위치를 계산합니다.
pos = nx.kamada_kawai_layout(subgraph)

# 서브그래프를 시각화합니다.
plt.figure(figsize=(15, 15))
nx.draw(subgraph, pos, with_labels=True, node_size=50, font_size=8, alpha=0.7, arrows=True)
plt.title(f"Graph Visualization from '{source_node}'")
plt.show()
"""

# 2. Scoring
본 섹션에서는 정의된 함수로 데이터프레임을 전처리하고 성능지표를 계산합니다.
## preprocessing Dataframes with spacy

In [ ]:
def CC(csv_name, loading_file_name):
    file_name = loading_file_name

    df = pd.read_csv('{}.csv'.format(csv_name))#.iloc[:,1:]
    #print(df.isna().sum())

    df = df.dropna(subset=['num_para'], how='any', axis=0)
    #print(df.isna().sum())

    gpt_rel = df[df['relations'].notnull()]
    gpt_rel = gpt_rel.reset_index(drop=True)
        
    # 각 문단당 relations, para, num_para를 딕셔너리로 저장
    data_list = []
    given_rel = ["head of government", "country", "place of birth", "place of death", "father", "mother", "spouse", "country of citizenship", "continent", "instance of", "head of state", "capital", "official language", "position held", "child", "author", "member of sports team", "director", "screenwriter", "educated at", "composer", "member of political party", "employer", "founded by", "league", "publisher", "owned by", "located in the administrative territorial entity", "genre", "operator", "religion", "contains administrative territorial entity", "follows", "followed by", "headquarters location", "cast member", "producer", "award received", "creator", "parent taxon", "ethnic group", "performer", "manufacturer", "developer", "series", "sister city", "legislative body", "basin country", "located in or next to body of water", "military branch", "record label", "production company", "location", "subclass of", "subsidiary", "part of", "original language of work", "platform", "mouth of the watercourse", "original network", "member of", "chairperson", "country of origin", "has part", "residence", "date of birth", "date of death", "inception", "dissolved, abolished or demolished", "publication date", "start time", "end time", "point in time", "conflict", "characters", "lyrics by", "located on terrain feature", "participant", "influenced by", "location of formation", "parent organization", "notable work", "separated from", "narrative location", "work location", "applies to jurisdiction", "product or material produced", "unemployment rate", "territory claimed by", "participant of", "replaces", "replaced by", "capital of", "languages spoken, written or signed", "present in work", "sibling"]

    for index, row in gpt_rel.iterrows():
        relations = row['relations'].split('\n')
        relations = [relation.strip() for relation in relations if relation.strip()]
        para = row['para'][:-1]
        num_para = row['num_para']
        
        for relation in relations:
            if ("None" not in relation) and ("none" not in relation):
                # 넘버링을 제거하고 source, target, relation 분리
                parts = relation.split('@')
                if len(parts) == 3:
                    source = parts[0].split(' ')[1].strip()
                    target = parts[1].strip()
                    relation = parts[2].strip()

                    if relation in given_rel:
                        # source = source.replace("we", real_com_name).replace("We", real_com_name)\
                        #             .replace("Company", real_com_name).replace("company", real_com_name)
                        # target = target.replace("we", real_com_name).replace("We", real_com_name)\
                        #             .replace("Company", real_com_name).replace("company", real_com_name)

                        relation_dict = {
                            'num_para': num_para,
                            'para': para,
                            'source': source,
                            'target': target,
                            'relation': relation
                        }
                        data_list.append(relation_dict)


    # 위에서 생성한 data_list 사용
    data_df = pd.DataFrame(data_list)
    #data_df['company'] = csv_name[23:]
    data_df = data_df.drop_duplicates(subset=['source', 'target', 'relation'])

    DF = data_df

    # 저장 
    data_df.to_csv('.{}.csv'.format(file_name))

    # 'source' 또는 'target' 열의 값이 'para' 열에 속하는 경우 'o' 할당

    for i, row in data_df.iterrows():
        if pd.notna(row['para']):
            source_in_para = row['source'] in row['para']
            target_in_para = row['target'] in row['para']
            
            if source_in_para and target_in_para:
                data_df.loc[i, 'entity exist?'] = 'o'
            else:
                data_df.loc[i, 'entity exist?'] = 'x'

    data_df.to_excel(excel_writer='{}.xlsx'.format(file_name), index=False)  
 
    return DF

In [ ]:
# SpaCy를 이용해 relation을 추출한 데이터를 불러옵니다.
after_spacy = pd.read_csv('f1_after_spacy_gpt.csv')
after_spacy.head(1)

In [ ]:
# 데이터를 불러옵니다.
preprocessed = pd.read_excel('F1_a_s_preprocessed.xlsx')
preprocessed.head(1)

In [ ]:
GPT_SP_DF1 = CC('f1_after_spacy_gpt', 'F1_a_s_preprocessed')

CC 함수를 이용해 'f1_after_spacy_gpt' dataframe과 'F1_a_s_preprocessed' dataframe을 처리하여 GPT_SP_DF1에 저장합니다.

In [ ]:
def making_DF(csv_name, company):
    df = pd.read_csv('{}.csv'.format(csv_name))#.iloc[:,1:]

    df = df.dropna(subset=['num_para'], how='any', axis=0)

    gpt_rel = df[df['relations'].notnull()]
    gpt_rel = gpt_rel.reset_index(drop=True)

    # 한 문단 당 relations들 한 리스트로 만들기 i.e.['1. Company @ strategic alliance @ None', '2. Company @ Caterpillar Inc. @ strategic alliance', ..]
    rel_li = []
    rel_index_li = []
    for idx, rel in enumerate(gpt_rel['relations']):
        if '\n' in rel:
            rel_li.append(rel.split('\n'))
            #print(rel.split('\n'))
            for i in range(len(rel.split('\n'))):
                rel_index_li.append(idx)
                #print(idx)
            #print('='*20)
        else:
            if '1. (' in rel:
                rel_li.append(rel)
                #print(rel)
                #print('='*20)
                rel_index_li.append(idx)

    # 한 리스트로 만들기
    import itertools

    def from_iterable(iterables):
        for it in iterables:
            for element in it:
                yield element

    rel_li = list(itertools.chain(*rel_li)) 

    # 넘버링 떼기
    rels_li = []
    idx_li = []
    for idx, rel in enumerate(rel_li):
        if rel[1:3]== '. ':
            if rel[3]=='(':
                rels_li.append(rel[4:-1])
                idx_li.append(rel_index_li[idx])
            else:
                rels_li.append(rel[3:])
                idx_li.append(rel_index_li[idx])
        elif rel[2:4]== '. ':
            if rel[4]=='(':
                rels_li.append(rel[5:-1])
                idx_li.append(rel_index_li[idx])
            else:
                rels_li.append(rel[4:])
                idx_li.append(rel_index_li[idx])
    print('len of idx and rel:', len(idx_li), len(rels_li))

    # @ 기준으로 h,t,r로 나누기
    r_li = []
    para_li = []
    for rel, idx in zip(rels_li,idx_li):
        if len(rel.split(' @ '))==3:
            if 'entity' not in rel.split(' @ ')[0] and 'entity' not in rel.split(' @ ')[1]:
                r_li.append(rel.split(' @ '))
                para_li.append(gpt_rel['para'].iloc[idx][:-1])
        # else:
            # print(rel.split(' @ '))

    # df로 만들기
    kg_df = pd.DataFrame(r_li, columns = ['source','target','edge'])
    kg_df['para'] = para_li
    kg_df = kg_df.drop_duplicates()#keep='last')


    # edge = none인 행 제거
    nem_df = kg_df.dropna(subset=['edge'], how='any', axis=0)

    none_row = nem_df[nem_df['source']=='None'].index
    NEM_df = nem_df.drop(none_row, inplace=False)
    nem_df  = NEM_df.reset_index(drop=True)

    none_row = nem_df[nem_df['target']=='None'].index
    NEM_df = nem_df.drop(none_row, inplace=False)
    nem_df  = NEM_df.reset_index(drop=True)

    none_row = nem_df[nem_df['edge']=='None'].index
    NEM_df = nem_df.drop(none_row, inplace=False)
    NEM_df  = NEM_df.reset_index(drop=True)

    # given rel에 존재하는 데이터만 저장
    given_rel = ["head of government", "country", "place of birth", "place of death", "father", "mother", "spouse", "country of citizenship", "continent", "instance of", "head of state", "capital", "official language", "position held", "child", "author", "member of sports team", "director", "screenwriter", "educated at", "composer", "member of political party", "employer", "founded by", "league", "publisher", "owned by", "located in the administrative territorial entity", "genre", "operator", "religion", "contains administrative territorial entity", "follows", "followed by", "headquarters location", "cast member", "producer", "award received", "creator", "parent taxon", "ethnic group", "performer", "manufacturer", "developer", "series", "sister city", "legislative body", "basin country", "located in or next to body of water", "military branch", "record label", "production company", "location", "subclass of", "subsidiary", "part of", "original language of work", "platform", "mouth of the watercourse", "original network", "member of", "chairperson", "country of origin", "has part", "residence", "date of birth", "date of death", "inception", "dissolved, abolished or demolished", "publication date", "start time", "end time", "point in time", "conflict", "characters", "lyrics by", "located on terrain feature", "participant", "influenced by", "location of formation", "parent organization", "notable work", "separated from", "narrative location", "work location", "applies to jurisdiction", "product or material produced", "unemployment rate", "territory claimed by", "participant of", "replaces", "replaced by", "capital of", "languages spoken, written or signed", "present in work", "sibling"]
    given_rel_row = []

    for i in range(len(NEM_df)):
        if any(g_rel == NEM_df['edge'][i] for g_rel in given_rel):
            given_rel_row.append([NEM_df['para'][i],NEM_df['source'][i],NEM_df['target'][i],NEM_df['edge'][i]])

    final_DF2 = pd.DataFrame(given_rel_row, columns = ['para','source','target','relation'])

    final_DF = final_DF2[['source','target','relation']]

    # 저장 
    final_DF.to_csv('./{}_DF1.csv'.format(company))
    #print('=======SAVE AS DF DONE========')
    final_DF2.to_excel(excel_writer='./{}.xlsx'.format(company)) 
    #dfi.export(final_DF, './RESULT/{}_RE_By_{}.png'.format(company, 'GPT'), max_cols=-1, max_rows=-1)

    return final_DF2

In [ ]:
def load_data(file_path):
    """CSV 파일을 안전하게 읽어옵니다."""
    if not os.path.exists(file_path):
        print(f"파일이 존재하지 않습니다: {file_path}")
        return pd.DataFrame()
    try:
        return pd.read_csv(file_path)
    except Exception as e:
        print(f"파일 읽기 오류: {e}")
        return pd.DataFrame()

def extract_relations(gpt_rel):
    """'relations' 열에서 관계를 추출합니다."""
    rel_li, rel_index_li = [], []
    for idx, rel in enumerate(gpt_rel['relations']):
        # 'relations' 열의 형식 검증 및 정제
        if not isinstance(rel, str):
            continue
        rel = rel.strip()
        if '\n' in rel:
            parts = rel.split('\n')
        else:
            parts = [rel]
        for part in parts:
            if '.' in part and '@' in part:
                rel_li.append(part)
                rel_index_li.append(idx)

    # 숫자 제거 및 트리플렛 추출
    triples = []
    for idx, rel in enumerate(rel_li):
        parts = rel.split('@')
        if len(parts) == 3:
            source, target, relation = [x.strip() for x in parts]
            source = source[source.find('.') + 1:].strip()  # 숫자 제거
            triples.append((source, target, relation, gpt_rel['para'].iloc[rel_index_li[idx]]))

    return triples

def create_dataframe(triples, given_rel):
    """트리플렛을 포함하는 새로운 DataFrame을 생성합니다."""
    df = pd.DataFrame(triples, columns=['source', 'target', 'relation', 'para'])
    # 주어진 관계에 해당하는 데이터만 필터링
    df = df[df['relation'].isin(given_rel)]
    return df.drop_duplicates()

def process(csv_name, company):
    file_path = f'{csv_name}.csv'
    df = load_data(file_path)
    if df.empty:
        return pd.DataFrame()

    # 필요한 열 선택
    df = df.dropna(subset=['num_para'])
    gpt_rel = df[df['relations'].notnull()].reset_index(drop=True)

    # 관계 추출
    triples = extract_relations(gpt_rel)

    # 주어진 관계 목록
    given_rel = ["head of government", "country", "place of birth", "place of death", "father", "mother", "spouse", "country of citizenship", "continent", "instance of", "head of state", "capital", "official language", "position held", "child", "author", "member of sports team", "director", "screenwriter", "educated at", "composer", "member of political party", "employer", "founded by", "league", "publisher", "owned by", "located in the administrative territorial entity", "genre", "operator", "religion", "contains administrative territorial entity", "follows", "followed by", "headquarters location", "cast member", "producer", "award received", "creator", "parent taxon", "ethnic group", "performer", "manufacturer", "developer", "series", "sister city", "legislative body", "basin country", "located in or next to body of water", "military branch", "record label", "production company", "location", "subclass of", "subsidiary", "part of", "original language of work", "platform", "mouth of the watercourse", "original network", "member of", "chairperson", "country of origin", "has part", "residence", "date of birth", "date of death", "inception", "dissolved, abolished or demolished", "publication date", "start time", "end time", "point in time", "conflict", "characters", "lyrics by", "located on terrain feature", "participant", "influenced by", "location of formation", "parent organization", "notable work", "separated from", "narrative location", "work location", "applies to jurisdiction", "product or material produced", "unemployment rate", "territory claimed by", "participant of", "replaces", "replaced by", "capital of", "languages spoken, written or signed", "present in work", "sibling"]

    # 새로운 DataFrame 생성
    final_df = create_dataframe(triples, given_rel)

    # 결과 저장
    # final_df.to_csv(f'./{company}_DF1.csv', index=False)
    # final_df.to_excel(f'./{company}.xlsx', index=False)

    return final_df

# 사용 예시
result_df2 = process('f1_after_spacy_gpt','F1_a_s_preprocessed')

In [ ]:
result_df2 = process('f1_after_spacy_gpt','F1_a_s_preprocessed')
result_df2

process 함수를 이용하여 'f1_after_spacy_gpt' dataframe과 'F1_a_s_preprocessed' dataframe을 처리하여 result_df2에 저장합니다.

In [ ]:
import json
name_type_dict = {}

with open('output_text_file.json', 'r', encoding='utf-8') as file:
    inputs_json = json.load(file)

for item in inputs_json:
    if item['vertexSet']:  # vertexSet이 비어있지 않은 경우에만 작업을 수행
        for vertex in item['vertexSet']:
            for v in vertex:
                name = v['name']
                type_ = v['type']
                name_type_dict[name] = type_

In [ ]:
list(name_type_dict.items())[:10]

In [ ]:
# 데이터프레임의 각 행에 대해 단어를 가져와서 NER 태그를 찾습니다.
## source 의 단어가 name_type_dict에 있으면 ner 태크 가져오기

# iterate through the 'source' column and retrieve the NER tag if the word is in name_type_dict and generate a lsit
ner_tags = [name_type_dict.get(word, 'unknown') for word in result_df2['source']]

# 이제 ner_tags 리스트는 데이터프레임의 행 수와 일치합니다.
## source_ner_2에 태그 저장
# add a new column source_ner_2 and store ner_tags
result_df2['source_ner_2'] = ner_tags

# 데이터프레임의 각 행에 대해 단어를 가져와서 NER 태그를 찾습니다.
## target과 name_type_dict가 겹치면 ner 태그 가져오기

# iterate through the 'target' column and retrieve the NER tag if the word is in name_type_dict and generate a lsit
ner_tags = [name_type_dict.get(word, 'unknown') for word in result_df2['target']]

# 이제 ner_tags 리스트는 데이터프레임의 행 수와 일치합니다.
## target_ner_2에 태그 저장
# add a new column target_ner_2 and store ner_tags
result_df2['target_ner_2'] = ner_tags

# drop 'unknown' values
result_df2 = result_df2[(result_df2['source_ner_2']!='unknown')&(result_df2['target_ner_2']!='unknown')]

result_df2[:5]

In [ ]:
GPT_SP_DF = making_DF('f1_after_spacy_gpt','F1_a_s_preprocessed')

## check if word is in para
def is_word_in_para(word, para):
    return word in para

# Apply the function to each row
## checks if the source and target values are in the para column
GPT_SP_DF['source_in_para'] = GPT_SP_DF.apply(lambda row: is_word_in_para(row['source'], row['para']), axis=1)
GPT_SP_DF['target_in_para'] = GPT_SP_DF.apply(lambda row: is_word_in_para(row['target'], row['para']), axis=1)

## GPT_SP_DF2 is same as GPT_SP_DF
GPT_SP_DF2= GPT_SP_DF.copy()
## GPT_DF2 only includes rows that have True values in the source_in_para and target_in_para columns
GPT_DF2 = GPT_SP_DF[(GPT_SP_DF['source_in_para'] == True) & (GPT_SP_DF['target_in_para'] == True)]

making_DF 함수를 이용하여 'f1_after_spacy_gpt' dataframe과 'F1_a_s_preprocessed' dataframe을 처리하여 GPT_SP_DF에 저장합니다.

## Preprocessing Dataframes without spacy

In [ ]:
GPT_SP_DF3 = CC('f1_gpt', 'F1_preprocessed')
# only include rows wheere the 'entity exist' column == 'o'
GPT_SP_DF3 = GPT_SP_DF3[GPT_SP_DF3['entity exist?']=='o']

CC 함수를 이용해 'f1_gpt' dataframe과 'F1_preprocessed' dataframe을 처리하여 GPT_SP_DF3에 저장합니다.

In [ ]:
GPT_SP_DF3.head(3)

GPT_SP_DF3에는 기존 문장, 추출된 entity, 관계, 그리고 추출된 entity가 기존 문장에 있는지 여부 데이터가 포함되어 있습니다.

In [ ]:
# different from the previous GPT_SP_DF
# previous input: ('f1_after_spacy_gpt','F1_a_s_preprocessed')
GPT_SP_DF = making_DF('f1_gpt', 'F1_preprocessed')
# function that checks if word is in para
def is_word_in_para(word, para):
    return word in para

# apply the function to each row
GPT_SP_DF['source_in_para'] = GPT_SP_DF.apply(lambda row: is_word_in_para(row['source'], row['para']), axis=1)
GPT_SP_DF['target_in_para'] = GPT_SP_DF.apply(lambda row: is_word_in_para(row['target'], row['para']), axis=1)

GPT_SP_DF4 = GPT_SP_DF.copy()

GPT_DF4 = GPT_SP_DF[(GPT_SP_DF['source_in_para'] == True) & (GPT_SP_DF['target_in_para'] == True)]

In [ ]:
# 사용 예시
result_df4 = process('f1_gpt','F1_preprocessed')
result_df4.head(5)

procss 함수로 'f1_gpt' dataframe과 F1_preprocessed dataframe을 처리한 후 result_df4에 저장했습니다.

## Metric

In [ ]:
# docred dataset의 validation 세트를 로드합니다.
validation_set = load_dataset("docred", split='validation')

In [ ]:
# 초기 통계 변수 초기화
total_documents = len(validation_set)
total_entities = 0
total_sentences = 0
# count occurrences of different relation types
relation_types = Counter()

# 각 문서에 대한 정보 파악
for document in validation_set:
    # 엔티티 수 계산
    total_entities += sum(len(vertex) for vertex in document['vertexSet'])

    # 문장 수 계산
    total_sentences += len(document['sents'])

    # 관계 유형 계산
    for relation_type in document['labels']:
        relation_types[relation_type] += 1

# 평균 엔티티 수와 문서 길이 계산
average_entities_per_doc = total_entities / total_documents
average_sentences_per_doc = total_sentences / total_documents

# 결과 출력
print(f"Total Documents: {total_documents}")
print(f"Average Entities per Document: {average_entities_per_doc}")
print(f"Average Sentences per Document: {average_sentences_per_doc}")
print(f"Relation Types: {relation_types}")

In [ ]:
# 각 문서의 엔티티 수와 문장 수를 저장할 리스트를 초기화합니다.
entity_counts = []
sentence_counts = []

# 각 문서에 대한 엔티티 수와 문장 수를 계산합니다.
for document in validation_set:
    entity_counts.append(len(document['vertexSet']))
    sentence_counts.append(len(document['sents']))

# 엔티티 수와 문장 수의 최대와 최소를 계산합니다.
max_entities = max(entity_counts)
min_entities = min(entity_counts)
max_sentences = max(sentence_counts)
min_sentences = min(sentence_counts)

print(f'maxiumum number of entities: {max_entities}, minimum number of entities: {min_entities}, maximum number of sentences: {max_sentences}, minimum number of sentences: {min_sentences}')

In [ ]:
# 초기 통계 변수 초기화
# count the occurrence of different relation texts
relation_texts = Counter()
# count the occurrence of entity pairs
entity_pair_relations = Counter()

# 각 문서에 대한 정보 파악
for document in validation_set:
    # 각 관계에 대한 정보 추출
    relations = document['labels']
    relation_texts_list = relations['relation_text']
    heads = relations['head']
    tails = relations['tail']

    # 관계 유형과 라벨 분포 계산
    for relation_text in relation_texts_list:
        relation_texts[relation_text] += 1

    # 엔티티 쌍 관계 분포 계산
    for head, tail in zip(heads, tails):
        entity_pair_relations[(head, tail)] += 1

# 결과 출력
print(f"Relation Texts: {relation_texts}")
print(f"Entity Pair Relations: {entity_pair_relations}")

각 관계별 count수와 entity별 unique pair의 갯수를 출력합니다.

In [ ]:
# 각 문서에서 관계의 개수를 저장할 리스트 초기화
relation_counts_per_doc = []

# 각 문서를 순회하며 관계의 개수를 계산
for document in validation_set:
    # 현재 문서에서 관계의 개수를 세고 리스트에 추가
    relation_counts_per_doc.append(len(document['labels']['relation_id']))
    # print(document['labels']['relation_id'])

# 평균, 최대, 최소 관계 개수 계산
average_relations = sum(relation_counts_per_doc) / len(relation_counts_per_doc)
max_relations = max(relation_counts_per_doc)
min_relations = min(relation_counts_per_doc)

print(f'average number of relations per document: {average_relations}, maximum number relations in a document: {max_relations}, mininum number of relations in a document: {min_relations}') #, relation_counts_per_doc

### Relation Type별 Count Bar Chart

In [ ]:
# 가정된 데이터에서 relation_texts Counter 객체를 생성합니다.
# 실제 데이터로 대체해야합니다.

# 데이터를 준비합니다: 관계 유형과 해당 카운트
labels, values = zip(*relation_texts.items())

# 관계 유형을 기준으로 내림차순 정렬합니다.
labels, values = zip(*sorted(zip(labels, values), key=lambda x: x[1], reverse=True))

# 히스토그램을 생성합니다.
plt.figure(figsize=(15, 20))  # 그래프의 크기를 설정합니다.
plt.barh(labels, values, color='slategray')  # 수평 막대 그래프 생성
plt.xlabel('Count')  # x축 라벨
plt.ylabel('Relation Type')  # y축 라벨
plt.title('Relation Type Counts in Descending Order', fontsize=20)  # 그래프 제목
plt.gca().invert_yaxis()  # y축의 순서를 뒤집어 내림차순으로 만듭니다.
plt.show()  # 그래프를 표시합니다.

In [ ]:
with open('SSAN_result_exist_relation1.p', 'rb') as f:
    ssan_result = pickle.load(f)

### 성능 지표

In [ ]:
# score calculation
def calculate_f1_score(predicted_relations, validation_relations):
    # True Positives, False Positives, False Negatives 초기화
    TP = sum(pred in validation_relations for pred in predicted_relations)
    FP = sum(pred not in validation_relations for pred in predicted_relations)
    FN = sum(true_rel not in predicted_relations for true_rel in validation_relations)

    # Precision, Recall 계산
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0

    # F1 Score 계산
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1_score


# extracts and returns tuples of relations from GPT_DF and validation_set
def pre_true_DataSet(GPT_DF,validation_set):
    # initialize list for predicted relations
    predicted_relations = []
    # extract relations from GPT_DF
    # create a tuple containing the 'source', 'target', and 'relation'.
    for i, row in GPT_DF.iterrows():
        predicted_relations.append((row['source'], row['target'], row['relation'])) #, row['relation'], set([row['source'], row['target']]
    
    # initialize list for validation relations
    validation_relations = []
    # extract relations from validation set
    for item in validation_set:
        # 각 문장에 대한 정점 집합(vertexSet)과 레이블을 추출
        vertex_set = item['vertexSet']
        labels = item['labels']

        # 각 관계에 대하여
        # iterate labels
        for head, tail, relation in zip(labels['head'], labels['tail'], labels['relation_text']):
            # 'head'와 'tail' 인덱스를 사용하여 각 관계의 개체 이름을 얻음
            # extract name of the head entity from 'vertex_set'
            head_entity = vertex_set[head][0]['name']  # 첫 번째 인스턴스를 사용
            # extract name of the tail entity
            tail_entity = vertex_set[tail][0]['name']  # 첫 번째 인스턴스를 사용

            # 관계를 튜플 형태로 리스트에 추가
            validation_relations.append((head_entity, tail_entity, relation)) # , relation, set([head_entity, tail_entity])

    # predicted_relations contains tuples of relations extracted from 'GPT_DF'
    # validation_relations contains tuples of relations extracted from 'validation_set'
    return predicted_relations, validation_relations


In [ ]:
def pre_true_DataSet_set(GPT_DF,validation_set):
    predicted_relations = []
    for i, row in GPT_DF.iterrows():
        predicted_relations.append(set([row['source'], row['target']])) #, row['relation'], set([row['source'], row['target']]
        predicted_relations = [tuple(s) for s in predicted_relations]
        predicted_relations = list(set(predicted_relations))

    validation_relations = []
    for item in validation_set:
        # 각 문장에 대한 정점 집합(vertexSet)과 레이블을 추출
        vertex_set = item['vertexSet']
        labels = item['labels']

        # 각 관계에 대하여
        for head, tail, relation in zip(labels['head'], labels['tail'], labels['relation_text']):
            # 'head'와 'tail' 인덱스를 사용하여 각 관계의 개체 이름을 얻음
            head_entity = vertex_set[head][0]['name']  # 첫 번째 인스턴스를 사용
            tail_entity = vertex_set[tail][0]['name']  # 첫 번째 인스턴스를 사용

            # 관계를 튜플 형태로 리스트에 추가
            validation_relations.append(set([head_entity, tail_entity])) # , relation, set([head_entity, tail_entity])
            validation_relations = [tuple(s) for s in validation_relations]
            validation_relations = list(set(validation_relations))
    return predicted_relations, validation_relations

* GPT_DF2

In [ ]:
# GPT_DF2
predicted_relations_gpt_small, validation_relations = pre_true_DataSet(GPT_DF2,validation_set)
# F1 점수 계산
precision, recall ,f1_score = calculate_f1_score(predicted_relations_gpt_small, validation_relations)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")

* GPT_DF4

In [ ]:
# GPT_DF4
predicted_relations_gpt_larger, validation_relations = pre_true_DataSet(GPT_DF4,validation_set)
# F1 점수 계산
precision, recall ,f1_score = calculate_f1_score(predicted_relations_gpt_larger, validation_relations)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")

* extracting predicted relations from result_df2 dataframe and result_df4 dataframe

In [ ]:
"""
# result_df2
predicted_relations, validation_relations = pre_true_DataSet(result_df2,validation_set)
# F1 점수 계산
precision, recall ,f1_score = calculate_f1_score(predicted_relations, validation_relations)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")
"""

In [ ]:
"""
predicted_relations, validation_relations = pre_true_DataSet(result_df4,validation_set)
# F1 점수 계산
precision, recall ,f1_score = calculate_f1_score(predicted_relations, validation_relations)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")
"""

In [ ]:
# adding NER columns, source_ner_2 and target_ner_2 to result_df4

# 데이터프레임의 각 행에 대해 단어를 가져와서 NER 태그를 찾습니다.
ner_tags = [name_type_dict.get(word, 'unknown') for word in result_df4['source']]

# 이제 ner_tags 리스트는 데이터프레임의 행 수와 일치합니다.
result_df4['source_ner_2'] = ner_tags

# 데이터프레임의 각 행에 대해 단어를 가져와서 NER 태그를 찾습니다.
ner_tags = [name_type_dict.get(word, 'unknown') for word in result_df4['target']]

# 이제 ner_tags 리스트는 데이터프레임의 행 수와 일치합니다.
result_df4['target_ner_2'] = ner_tags

result_df4 = result_df4[(result_df4['source_ner_2']!='unknown')&(result_df4['target_ner_2']!='unknown')]

ner 값이 'unknown'인 row를 삭제합니다.

In [ ]:
predicted_relations, validation_relations = pre_true_DataSet(result_df4,validation_set)
# F1 점수 계산
precision, recall ,f1_score = calculate_f1_score(predicted_relations, validation_relations)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")

ner값이 'unknown'인 row를 삭제 후 precision과 F1 Score는 증가, recall은 감소했습니다.

### SSAN with relation 성능지표

In [ ]:
predicted_relations= []
for i in range(len(ssan_result)):
    predicted_relations.append(ssan_result[i]['relation'])
predicted_relations =  [item for sublist in predicted_relations  for item in sublist]
_, validation_relations = pre_true_DataSet(GPT_DF4,validation_set)
# F1 점수 계산
precision, recall ,f1_score = calculate_f1_score(predicted_relations, validation_relations)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")

### SSAN without relation 성능지표

In [ ]:
# 기존 값보다 precision, recall, f1 score 값이 낮습니다.
# list of sets
predicted_relations= []
for i in range(len(ssan_result)):
    predicted_relations.append(ssan_result[i]['relation'])
predicted_relations =  [item for sublist in predicted_relations  for item in sublist]

# transformation: only contains source entity, target entity, and relation entity.
predicted_without_relations = [set([s,t]) for s,t,r in predicted_relations]
predicted_without_relations = [tuple(sorted(relation)) for relation in predicted_without_relations]

_, validation_relations = pre_true_DataSet(GPT_DF4,validation_set)
validation_without_relations = [(source, target) for source, target, _ in validation_relations]
# F1 점수 계산
precision, recall ,f1_score = calculate_f1_score(predicted_without_relations, validation_without_relations)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")

### ChatGPT with relation 성능지표
'GPT_DF2 dataframe'과 'GPT_DF4' dataframe에서 추출한 predicted_relations를 합친 후 precision, recall, f1_score를 계산합니다.

In [ ]:
# 'GPT_DF2' dataframe 과 'GPT_DF4' dataframe에서 추출한 predicted_relations를 합칩니다.
pr = list(predicted_relations_gpt_larger+predicted_relations)

precision, recall ,f1_score = calculate_f1_score(pr, validation_relations)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")

### ChatGPT without relation 성능지표
'GPT_DF2 dataframe'과 'GPT_DF4' dataframe에서 추출한 predicted_relations를 합친 후 relation을 제외한 precision, recall, f1_score를 계산합니다.

In [ ]:
# ChatGPT with relation 성능지표
pr = list(predicted_relations_gpt_larger+predicted_relations)
pr_without_relations = [(entity_1, entity_2) for entity_1, entity_2, _ in pr]
valdiation_without_relations = [(entity_1, entity_2) for entity_1, entity_2, _ in validation_relations]

precision, recall ,f1_score = calculate_f1_score(pr_without_relations, valdiation_without_relations)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")

relation을 제외하고 entity 쌍들만 비교를 합니다. precision, recall, F1 score는 predicted entity pair가 validation set에 얼만큼 일치하는지 여부에 따라 계산되었습니다.

### SSAN Relations and Entities

In [ ]:
# entities
# ssan_result[i]['relation']
# 각 문서에서 관계의 개수를 저장할 리스트 초기화
entity_counts_per_doc = []

# 각 문서를 순회하며 관계의 개수를 계산
for document in ssan_result:
    # 현재 문서에서 관계의 개수를 세고 리스트에 추가
    entity_counts_per_doc.append(len(document['entities']))
    # print(document['labels']['relation_id'])

# 평균, 최대, 최소 관계 개수 계산
average_entities = sum(entity_counts_per_doc) / len(entity_counts_per_doc)
max_entities = max(entity_counts_per_doc)
min_entities = min(entity_counts_per_doc)

print(f'average number of entities per doc: {average_entities}, maximum number of entites: {max_entities}, minimum entities: {min_entities}') #, relation_counts_per_doc

각 문서별 entity와 relation의 갯수를 count 합니다.

In [ ]:
# relations
# ssan_result[i]['relation']
# 각 문서에서 관계의 개수를 저장할 리스트 초기화
relation_counts_per_doc = []

# 각 문서를 순회하며 관계의 개수를 계산
for document in ssan_result:
    # 현재 문서에서 관계의 개수를 세고 리스트에 추가
    relation_counts_per_doc.append(len(document['relation']))
    # print(document['labels']['relation_id'])

# 평균, 최대, 최소 관계 개수 계산
average_relations = sum(relation_counts_per_doc) / len(relation_counts_per_doc)
max_relations = max(relation_counts_per_doc)
min_relations = min(relation_counts_per_doc)

print(f'average number of relations per doc: {average_relations}, maximum number of relations: {max_relations}, minimum number of relations: {min_relations}') #, relation_counts_per_doc

In [ ]:
# 초기 통계 변수 초기화
relation_texts = Counter()
#entity_pair_relations = Counter()

# 각 문서에 대한 정보 파악
# loop through each 'document' in the 'ssan_result'
for document in ssan_result:
    # 각 관계에 대한 정보 추출
    relations = document['relation']

    for i in relations:
        # counts the occurence of each type of relation
        relation_texts[i[2]] += 1
 

# 카운트 순으로 정렬합니다.
sorted_relations = relation_texts.most_common()

# 정렬된 결과를 출력합니다.
sorted_relations[:10]

SSAN 모델에서 가장 많은 relations입니다.

### GPT Relations and Entities

In [ ]:
# 예시 데이터 프레임을 생성합니다. 실제로는 파일에서 읽어오거나 다른 소스에서 가져올 수 있습니다.
df = GPT_DF4.copy()

# 문서별로 엔티티 수와 관계 수를 계산합니다.
doc_entities = defaultdict(set)
doc_relations = defaultdict(int)

for index, row in df.iterrows():
    para_id = hash(tuple(row['para']))  # 문단을 해싱하여 고유한 문서 ID 생성
    doc_entities[para_id].update([row['source'], row['target']])
    doc_relations[para_id] += 1

# 엔티티와 관계 수에 대한 통계를 계산합니다.
entity_counts = [len(entities) for entities in doc_entities.values()]
relation_counts = list(doc_relations.values())

# 엔티티에 대한 통계
average_entities = sum(entity_counts) / len(entity_counts)
max_entities = max(entity_counts)
min_entities = min(entity_counts)

# 관계에 대한 통계
average_relations = sum(relation_counts) / len(relation_counts)
max_relations = max(relation_counts)
min_relations = min(relation_counts)

# 결과 출력
print(f'average number of unique entities: {average_entities}, maximum number of unique entities: {max_entities}, minimum number of unique entities: {min_entities}, average number of relations: {average_relations}, maximum number of relations: {max_relations}, minimum relations: {min_relations}')

In [ ]:
# 관계 유형의 분포를 계산합니다.
relation_distribution = Counter(df['relation'])

# 가장 많이 나타나는 상위 3개의 관계 유형을 찾습니다.
most_common_relations = relation_distribution.most_common(3)

# 결과를 출력하는 문자열을 포맷합니다.
most_common_relations_report = "데이터셋은 다음과 같은 상위 3개의 관계 유형을 포함합니다: "
most_common_relations_report += ", ".join([f"'{relation[0]}'({relation[1]}회)" for relation in most_common_relations]) + "."

most_common_relations_report

### 벤다이어그램

In [ ]:
# convert predicted_relations into a set
setA = set(predicted_relations)
# convert predicted_relations into a set
setB = set(predicted_relations_gpt_small)
# convert predicted_relations into a set
setC = set(predicted_relations_gpt_larger)

# 벤 다이어그램 그리기
venn3([setA, setB, setC], ('SSAN', 'ChatGPT-small spacy', 'Set C-large spacy'))

# 그래프 표시
plt.show()

* predicted_relations_gpt_small 벤 다이어그램

In [ ]:
# convert predicted_relations into a set
setA = set(predicted_relations)
# convert predicted_relations into a set
setB = set(predicted_relations_gpt_small)
# convert predicted_relations into a set
setC = set(predicted_relations_gpt_larger)
# convert predicted_relations into a set
setD = set(validation_relations)

# 각 집합의 라벨에 전체 개수 포함
labels = ['True label\n(Total: {})'.format(len(setD)),
          'SSAN\n(Total: {})'.format(len(setA)),
          'RE ChatGPT\n(Total: {})'.format(len(setB))]
# 벤 다이어그램 그리기
venn3([setD, setA, setB], set_labels=labels)

# 그래프 표시
plt.show()

* predicted_relations_gpt_larger 벤 다이어그램

In [ ]:
# convert predicted_relations into a set
setA = set(predicted_relations)
# convert predicted_relations into a set
setB = set(predicted_relations_gpt_small)
# convert predicted_relations into a set
setC = set(predicted_relations_gpt_larger)
# convert predicted_relations into a set
setD = set(validation_relations)

# 각 집합의 라벨에 전체 개수 포함
labels = ['True label\n(Total: {})'.format(len(setD)),
          'SSAN\n(Total: {})'.format(len(setA)),
          'RE ChatGPT\n(Total: {})'.format(len(setC))]
# 벤 다이어그램 그리기
venn3([setD, setA, setC], set_labels=labels)

# 그래프 표시
plt.show()

* Venn Diagram With Relations

In [ ]:
# convert predicted_relations into a set
setA = set(predicted_relations)
# convert predicted_relations into a set
pr = list(predicted_relations_gpt_larger+predicted_relations)
setC = set(pr)
# convert predicted_relations into a set
setD = set(validation_relations)

# 각 집합의 라벨에 전체 개수 포함
labels = ['True label\n(Total: {})'.format(len(setD)),
          'SSAN\n(Total: {})'.format(len(setA)),
          'SSAN + ChatGPT\n(Total: {})'.format(len(setC))]
# 벤 다이어그램 그리기
venn3([setD, setA, setC], set_labels=labels)

# 그래프 표시
plt.show()

* 두 모델을 합쳤을때 recall은 증가하지만 precision은 감소합니다.
* ChatGPT 모델은 SSAN 모델보다 224개의 triplet을 더 추출했습니다.
* SSAN과 ChatGPT를 혼합하면 보완하는 relation extraction 모델이 될수 있습니다.

* Venn Diagram Without Relations

In [ ]:
# 벤 다이어그램의 모양이 슬라이드와 다릅니다
# convert predicted_relations into a set
setA_r = set(predicted_without_relations)
# convert predicted_relations into a set
pr = list(predicted_relations_gpt_larger+predicted_relations)
pr_without_relations = [(entity_1, entity_2) for entity_1, entity_2, _ in pr]
setC_r = set(pr_without_relations)
# convert predicted_relations into a set
setD_r = set(validation_without_relations)

# 각 집합의 라벨에 전체 개수 포함
labels = ['True label\n(Total: {})'.format(len(setD_r)),
          'SSAN\n(Total: {})'.format(len(setA_r)),
          'SSAN + ChatGPT\n(Total: {})'.format(len(setC_r))]
# 벤 다이어그램 그리기
venn3([setD_r, setA_r, setC_r], set_labels=labels)

# 그래프 표시
plt.show()

* setB에 있는 triplet을 제외한 후 setA와 setC의 triplet에 대한 그래프를 시각화 합니다.

In [ ]:
# create a new counter object
relation_texts = Counter()
# setA & setC - setB = elements that are common to both setA and setC
for triplet in list(setA & setC - setB):
    # count the third item in each 'triplet' tuple
    relation_texts[triplet[2]] += 1
list(relation_texts.items())[:10]

In [ ]:
# 가정된 데이터에서 relation_texts Counter 객체를 생성합니다.
# 실제 데이터로 대체해야합니다.


# 데이터를 준비합니다: 관계 유형과 해당 카운트
# extract extract labels and counter values
labels, values = zip(*relation_texts.items())

# 관계 유형을 기준으로 내림차순 정렬합니다.
# sorting the relations in descending order
labels, values = zip(*sorted(zip(labels, values), key=lambda x: x[1], reverse=True))

# 히스토그램을 생성합니다.
plt.figure(figsize=(15, 20))  # 그래프의 크기를 설정합니다.
plt.barh(labels, values, color='slategray')  # 수평 막대 그래프 생성
plt.xlabel('Count')  # x축 라벨
plt.ylabel('Relation Type')  # y축 라벨
plt.title('Relation Type Counts in Descending Order', fontsize=20)  # 그래프 제목
plt.gca().invert_yaxis()  # y축의 순서를 뒤집어 내림차순으로 만듭니다.
plt.show()  # 그래프를 표시합니다.

* setA, setB, setC에 있는 모든 triplet 포함한 그래프를 시각화 합니다.

In [ ]:
# create a new counter object
relation_texts = Counter()
# setA & setC & setB = elements that are common to setA, setC, and setB
for triplet in list(setA & setC & setB):
    # count the third item in each 'triplet' tuple
    relation_texts[triplet[2]] += 1

# 가정된 데이터에서 relation_texts Counter 객체를 생성합니다.
# 실제 데이터로 대체해야합니다.


# 데이터를 준비합니다: 관계 유형과 해당 카운트
# extract extract labels and counter values
labels, values = zip(*relation_texts.items())

# 관계 유형을 기준으로 내림차순 정렬합니다.
# sorting the relations in descending order
labels, values = zip(*sorted(zip(labels, values), key=lambda x: x[1], reverse=True))

# 히스토그램을 생성합니다.
plt.figure(figsize=(15, 20))  # 그래프의 크기를 설정합니다.
plt.barh(labels, values, color='slategray')  # 수평 막대 그래프 생성
plt.xlabel('Count')  # x축 라벨
plt.ylabel('Relation Type')  # y축 라벨
plt.title('Relation Type Counts in Descending Order', fontsize=20)  # 그래프 제목
plt.gca().invert_yaxis()  # y축의 순서를 뒤집어 내림차순으로 만듭니다.
plt.show()  # 그래프를 표시합니다.

# 3. Grouping
본 섹션에서는 데이터프레임내의 entity pair들의 similarity를 계산한후, 각 임계값 별 단어 grouping의 변화를 시각화 합니다.

In [ ]:
# similarity dataframe
sim_DF = pd.read_csv('SIM_df_all.csv')
sim_DF.head(5)

* entity 1 = entity 1
* entity 2 = entity 2
* similarity_percent = similarity percent

In [ ]:
# entity1과 entity2 쌍에 대해 similarity_percent가 가장 높은 행만 남깁니다.
# for each unique pair of 'entity1' and 'entity2', it forms a group
# finding indexes of rows with maximum simlarity percent
# finds the index of the row with the maximum value in the column 'similarity_percent'
idx = sim_DF.groupby(['entity1', 'entity2'])['similarity_percent'].idxmax()

# 위에서 얻은 인덱스를 사용하여 중복 없는 데이터 프레임을 얻습니다.
# create a new dataframe with unique highest similarity rows
sim_df = sim_DF.loc[idx].reset_index(drop=True)

In [ ]:
SSAN_SP_DF = pd.read_csv('SSAN_SP_DF_TAG.csv')
SSAN_SP_DF.head(5)

In [ ]:
GPT_SP_DF = pd.read_csv('GPT_SP_DF_TAG.csv')
GPT_SP_DF.head(5)

In [ ]:
# drop rows with 'Unkown' values
GPT_SP_DF = GPT_SP_DF[(GPT_SP_DF['source_ner']!='Unknown')&(GPT_SP_DF['target_ner']!='Unknown')]
GPT_SP_DF.head(5)

In [ ]:
# filtering for high similiarity pairs
def create_representative_df(SP_DF, sim_df, similarity_threshold):
    # 유사도가 주어진 임계값 이상인 쌍만 필터링
    high_similarity_pairs = sim_df[sim_df['similarity_percent'] >= similarity_threshold]

    # 그래프 생성
    G = nx.Graph()

    # 엔티티 쌍을 그래프의 엣지로 추가
    for index, row in high_similarity_pairs.iterrows():
        G.add_edge(row['entity1'], row['entity2'], weight=row['similarity_percent'])

    # 연결된 컴포넌트 리스트
    connected_components = list(nx.connected_components(G))

    # 각 연결된 컴포넌트에 대한 PageRank 계산
    subgraph_representatives = {}
    for component in connected_components:
        subgraph = G.subgraph(component)
        pagerank = nx.pagerank(subgraph, weight='weight')
        # 가장 중요한 노드를 대표 엔티티로 선택
        representative = max(pagerank, key=pagerank.get)
        for node in component:
            subgraph_representatives[node] = representative
    
    source_normalized = 'source_normalized_{}'.format(similarity_threshold)
    target_normalized = 'target_normalized_{}'.format(similarity_threshold)

    # 원본 데이터에서 source와 target을 대표 엔티티로 매핑
    SP_DF[source_normalized] = SP_DF['source'].apply(lambda x: subgraph_representatives.get(x, x))
    SP_DF[target_normalized] = SP_DF['target'].apply(lambda x: subgraph_representatives.get(x, x))

    # 딕셔너리를 데이터 프레임으로 변환
    representative_df = pd.DataFrame(list(subgraph_representatives.items()), columns=['Word', 'Representative'])

    return representative_df

In [ ]:
# example with SSAN
representative_df_95 = create_representative_df(SSAN_SP_DF, sim_df, 95)
representative_df_90 = create_representative_df(SSAN_SP_DF, sim_df, 90)
representative_df_85 = create_representative_df(SSAN_SP_DF, sim_df, 85)
representative_df_80 = create_representative_df(SSAN_SP_DF, sim_df, 80)
representative_df_75 = create_representative_df(SSAN_SP_DF, sim_df, 75)
representative_df_70 = create_representative_df(SSAN_SP_DF, sim_df, 70)
representative_df_65 = create_representative_df(SSAN_SP_DF, sim_df, 65)
representative_df_60 = create_representative_df(SSAN_SP_DF, sim_df, 60)

In [ ]:
# example with GPT
g_representative_df_95 = create_representative_df(GPT_SP_DF, sim_df, 95)
g_representative_df_90 = create_representative_df(GPT_SP_DF, sim_df, 90)
g_representative_df_85 = create_representative_df(GPT_SP_DF, sim_df, 85)
g_representative_df_80 = create_representative_df(GPT_SP_DF, sim_df, 80)
g_representative_df_75 = create_representative_df(GPT_SP_DF, sim_df, 75)
g_representative_df_70 = create_representative_df(GPT_SP_DF, sim_df, 70)
g_representative_df_65 = create_representative_df(GPT_SP_DF, sim_df, 65)
g_representative_df_60 = create_representative_df(GPT_SP_DF, sim_df, 60)

In [ ]:
# creating a 'sorted_pair' column
# creating tuples of 'source' and 'target' and sort alphabetically
GPT_SP_DF['sorted_pair'] = GPT_SP_DF.apply(lambda row: tuple(sorted([row['source'], row['target']], key=lambda x: (isinstance(x, str), x))), axis=1)
# remove duplicates and drop the 'sorted_pair' column
GPT_SP_DF_sorted = GPT_SP_DF.drop_duplicates(subset='sorted_pair').drop('sorted_pair', axis=1)

데이터프레임내의 entity pair를 알파벳순으로 정렬 후 중복되는 entity pair를 drop합니다.

In [ ]:
# dropping rows with 'Unknown' values in the 'source_ner' and 'target_ner' columns
GPT_SP_DF_sorted = GPT_SP_DF_sorted[(GPT_SP_DF_sorted['source_ner']!='Unknown') & (GPT_SP_DF_sorted['target_ner']!='Unknown')]

'source_ner' 과 'target_ner' 값이 'Unknown'인 row를 드랍합니다.

In [ ]:
# 가상의 데이터를 생성합니다. 실제 데이터로 대체해야 합니다.
# 이 데이터는 임시 데이터로, 실제 그래프를 생성할 때는 실제 데이터셋을 사용해야 합니다.

def generate_representative_df(sim_df, similarity_threshold):
    # 유사도가 주어진 임계값 이상인 쌍만 필터링
    high_similarity_pairs = sim_df[sim_df['similarity_percent'] >= similarity_threshold]

    # 그래프 생성
    G = nx.Graph()

    # 엔티티 쌍을 그래프의 엣지로 추가
    for index, row in high_similarity_pairs.iterrows():
        G.add_edge(row['entity1'], row['entity2'], weight=row['similarity_percent'])

    # 연결된 컴포넌트 리스트
    connected_components = list(nx.connected_components(G))

    # 각 연결된 컴포넌트에 대한 PageRank 계산
    subgraph_representatives = {}
    for component in connected_components:
        subgraph = G.subgraph(component)
        pagerank = nx.pagerank(subgraph, weight='weight')
        # 가장 중요한 노드를 대표 엔티티로 선택
        representative = max(pagerank, key=pagerank.get)
        for node in component:
            subgraph_representatives[node] = representative

    # 대표 엔티티를 데이터 프레임으로 변환
    representative_df = pd.DataFrame.from_dict(subgraph_representatives, orient='index', columns=['Representative'])
    representative_df.reset_index(inplace=True)
    representative_df.rename(columns={'index': 'Word'}, inplace=True)

    return representative_df

In [ ]:
# 여러 유사도 임계값에 대한 대표 단어 데이터프레임 생성
thresholds = [95, 90, 85, 80, 75, 70, 65, 60]
representative_dfs = {t: generate_representative_df(sim_df, t) for t in thresholds}

# 겹치지 않게 모든 데이터프레임 시각화
fig, axs = plt.subplots(len(thresholds), 1, figsize=(15, 40))  # 그래프 크기 조정

for i, t in enumerate(thresholds):
    df = representative_dfs[t]
    
    # 데이터 포인트 사이의 겹침을 줄이기 위해, 각 대표 단어에 대한 y 위치를 조정합니다.
    unique_representatives = df['Representative'].unique()
    y_positions = pd.Series(range(len(unique_representatives)), index=unique_representatives)
    df['y_position'] = df['Representative'].map(y_positions)
    
    # 각 임계값에 대한 대표 단어의 분포를 시각화
    axs[i].scatter(df.index, df['y_position'], label=f'{t}% Similarity Threshold',s=3)
    axs[i].set_yticks(y_positions)
    axs[i].set_yticklabels(unique_representatives, fontsize=3)
    axs[i].legend()
    axs[i].set_title(f'Representative Words at {t}% Similarity Threshold')
    axs[i].set_xlabel('Words')
    axs[i].set_ylabel('Representative Words (Groups)')

plt.tight_layout()
plt.show()

In [ ]:
# sim_df는 유사도 데이터를 포함하는 데이터프레임입니다.
# thresholds는 다양한 유사도 임계값을 나타냅니다.
thresholds = [95, 90, 85, 80, 75, 70, 65, 60]

# 각 임계값에 대한 데이터프레임을 저장하는 딕셔너리
representative_dfs = {t: generate_representative_df(sim_df, t) for t in thresholds}

# 모든 엔티티와 대표 엔티티의 유니크한 리스트를 생성합니다.
all_entities = pd.unique(sim_df[['entity1', 'entity2']].values.ravel('K'))
all_representatives = pd.unique(pd.concat([df['Representative'] for df in representative_dfs.values()]))

# 엔티티와 대표 엔티티에 대해 인덱스를 생성합니다.
entity_to_index = {entity: i for i, entity in enumerate(all_entities)}
representative_to_index = {rep: i for i, rep in enumerate(all_representatives)}

# 색상 맵 생성
colors = plt.cm.get_cmap('viridis', len(thresholds))

# 그래프 생성
plt.figure(figsize=(300, 300))

# 각 임계값에 대해 데이터 포인트를 그래프에 추가
for i, (threshold, df) in enumerate(representative_dfs.items()):
    # 대표 엔티티의 인덱스를 얻습니다.
    df['Representative_Index'] = df['Representative'].map(representative_to_index)
    
    # 원래 엔티티의 인덱스를 얻습니다.
    df['Word_Index'] = df['Word'].map(entity_to_index)
    
    # 데이터 포인트를 그래프에 추가
    plt.scatter(df['Word_Index'], df['Representative_Index'], color=colors(i), label=f'Threshold {threshold}%')

# 레이블과 타이틀 추가
plt.xlabel('Entities')
plt.ylabel('Representative Entities')
plt.title('Entity vs Representative Entity at Different Similarity Thresholds')
plt.xticks(np.arange(len(all_entities)), all_entities, rotation=90)
plt.yticks(np.arange(len(all_representatives)), all_representatives)
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
sim_DF = pd.read_csv('SIM_df_all-mpnet-base-v2_2.csv')
# convert the 'similarity_percent' column to percent
sim_DF['similarity_percent'] = sim_DF['similarity_percent'] * 100

# entity1과 entity2 쌍에 대해 similarity_percent가 가장 높은 행만 남깁니다.
# groupby와 idxmax를 사용하여 가장 높은 similarity_percent를 가진 인덱스를 얻습니다.
idx = sim_DF.groupby(['entity1', 'entity2'])['similarity_percent'].idxmax()

# 위에서 얻은 인덱스를 사용하여 중복 없는 데이터 프레임을 얻습니다.
sim_df = sim_DF.loc[idx].reset_index(drop=True)

gpt_sim_DF = pd.read_csv('GPT_SIM_df_all-mpnet-base-v2.csv')

# entity1과 entity2 쌍에 대해 similarity_percent가 가장 높은 행만 남깁니다.
# groupby와 idxmax를 사용하여 가장 높은 similarity_percent를 가진 인덱스를 얻습니다.
gpt_idx = gpt_sim_DF.groupby(['entity1', 'entity2'])['similarity_percent'].idxmax()

# 위에서 얻은 인덱스를 사용하여 중복 없는 데이터 프레임을 얻습니다.
gpt_sim_df = gpt_sim_DF.loc[gpt_idx].reset_index(drop=True)

'sim_DF' 데이터프레임과 'gpt_sim_DF' 데이터프레임에서 groupby와 idxmax를 사용하여 가장 높은 similarity_percent를 가진 인덱스를 얻습니다. 이후 해당 인덱스로 중복 없는 데이터 프레임을 얻고 sim_df와 gpt_sim_df에 각각 저장합니다.

In [ ]:
# 예제로 사용할 SSAN_SP_DF 데이터프레임 생성 (이것은 실제 데이터로 대체되어야 합니다)
thresholds = [95, 90, 85, 80, 75, 70, 65, 60]

# 유사도 임계값별로 SSAN_SP_DF의 source와 target을 대표 단어로 매핑
for threshold in thresholds:
    representative_df = create_representative_df(SSAN_SP_DF, sim_df, threshold)
    # create a dictionary from the representative_df for mapping
    rep_dict = dict(zip(representative_df['Word'], representative_df['Representative']))
    # map the 'source' column to its representative words, keeping orignial words where no mapping exists
    SSAN_SP_DF[f'source_rep_{threshold}'] = SSAN_SP_DF['source'].map(rep_dict).fillna(SSAN_SP_DF['source'])
    # map the 'target' column to its representative words, keeping orignial words where no mapping exists
    SSAN_SP_DF[f'target_rep_{threshold}'] = SSAN_SP_DF['target'].map(rep_dict).fillna(SSAN_SP_DF['target'])

    representative_df = create_representative_df(GPT_SP_DF, gpt_sim_df, threshold)
    # create a dictionary from the representative_df for mapping
    rep_dict = dict(zip(representative_df['Word'], representative_df['Representative']))
    # map the 'source' column to its representative words, keeping orignial words where no mapping exists
    GPT_SP_DF[f'source_rep_{threshold}'] = GPT_SP_DF['source'].map(rep_dict).fillna(GPT_SP_DF['source'])
    # map the 'target' column to its representative words, keeping orignial words where no mapping exists
    GPT_SP_DF[f'target_rep_{threshold}'] = GPT_SP_DF['target'].map(rep_dict).fillna(GPT_SP_DF['target'])

    GPT_SP_DF2 = GPT_SP_DF.copy()
    representative_df = create_representative_df(GPT_SP_DF2, gpt_sim_df, threshold)
    # create a dictionary from the representative_df for mapping
    rep_dict = dict(zip(representative_df['Word'], representative_df['Representative']))
    # map the 'source' column to its representative words, keeping orignial words where no mapping exists
    GPT_SP_DF2[f'source_rep_{threshold}'] = GPT_SP_DF2['source'].map(rep_dict).fillna(GPT_SP_DF2['source'])
    # map the 'target' column to its representative words, keeping orignial words where no mapping exists
    GPT_SP_DF2[f'target_rep_{threshold}'] = GPT_SP_DF2['target'].map(rep_dict).fillna(GPT_SP_DF2['target'])
# 이제 SSAN_SP_DF 데이터프레임에는 각 임계값에 따른 source와 target의 대표 단어가 매핑된 컬럼들이 추가되어 있습니다.
# 예를 들어, 'source_rep_95', 'target_rep_95', ..., 'source_rep_60', 'target_rep_60' 등의 컬럼들이 있을 것입니다.

similarity threshold가 entity 그룹화에 어떤 영향을 미치는지 확인합니다.

In [ ]:
# dropping columns
SSAN_SP_DF_without_column = SSAN_SP_DF.drop(['edge','company','source_ner_2','target_ner_2'], axis=1)
# filtering rows with 'Unknown' values
SSAN_SP_DF_without_column = SSAN_SP_DF_without_column[(SSAN_SP_DF_without_column['source_ner']!='Unknown')&(SSAN_SP_DF_without_column['target_ner']!='Unknown')]
# dropping columns
SSAN_SP_DF_without_column = SSAN_SP_DF_without_column.drop(['source_ner_3','target_ner_3','source_ner','target_ner'], axis=1)

# dropping columns
GPT_SP_DF_without_column = GPT_SP_DF.drop(['para','relation','company','source_ner_2','target_ner_2'], axis=1)
# filtering rows with 'Unknown' values
GPT_SP_DF_without_column = GPT_SP_DF_without_column[(GPT_SP_DF_without_column['source_ner_4']!='Unknown')&(GPT_SP_DF_without_column['target_ner_4']!='Unknown')]
# dropping columns
GPT_SP_DF_without_column = GPT_SP_DF_without_column.drop(['source_ner_3','target_ner_3','source_ner_4','target_ner_4','target_ner','source_ner'], axis=1)

In [ ]:
# NER 태그를 카테고리에 매핑하는 함수
def map_ner_to_category(ner_tag):
    if ner_tag in ['GPE', 'LOC','FAC']:
        return 'Country'
    elif ner_tag in ['ORG']:
        return 'Firm'
    elif ner_tag in ['MATERIAL']: # 'MONEY',
        return 'Resource'
    elif ner_tag in [ 'WORK_OF_ART', 'PRODUCT']:#'DATE', 'TIME', 'EVENT'
        return 'Technology'
    else:
        return 'Unknown'  # 혹은 다른 기본값

In [ ]:
# apply the function to SSAN_SP_DF and GPT_SP_DF2
SSAN_SP_DF['source_ner'] = SSAN_SP_DF['source_ner_3'].apply(map_ner_to_category)
SSAN_SP_DF['target_ner'] = SSAN_SP_DF['target_ner_3'].apply(map_ner_to_category)

GPT_SP_DF2['source_ner'] = GPT_SP_DF2['source_ner_3'].apply(map_ner_to_category)
GPT_SP_DF2['target_ner'] = GPT_SP_DF2['target_ner_3'].apply(map_ner_to_category)

In [ ]:
# 파일 저장
SSAN_SP_DF[(SSAN_SP_DF['source_ner_3']!='Unknown')&(SSAN_SP_DF['target_ner_3']!='Unknown')].to_csv('SSAN_SP_DF_after_norm.csv', index=False)
GPT_SP_DF[(GPT_SP_DF['source_ner_4']!='Unknown')&(GPT_SP_DF['target_ner_4']!='Unknown')].to_csv('GPT_SP_DF_after_norm.csv', index=False)
GPT_SP_DF2[(GPT_SP_DF2['source_ner_4']!='Unknown')&(GPT_SP_DF2['target_ner_4']!='Unknown')].to_csv('GPT_SP_DF_after_norm2.csv', index=False)

In [ ]:
# filter 'Unknown' values in the 'source_ner_4' and the 'target_ner_4' columns
GPT_SP_DF2 = GPT_SP_DF2[(GPT_SP_DF2['source_ner_4']!='Unknown')&(GPT_SP_DF2['target_ner_4']!='Unknown')]

## 전처리된 DataFrames

In [ ]:
SSAN_SP_DF = pd.read_csv('SSAN_SP_DF_after_norm.csv')
GPT_SP_DF = pd.read_csv('GPT_SP_DF_after_norm.csv')
GPT_SP_DF2 = pd.read_csv('GPT_SP_DF_after_norm2.csv')

전처리된 데이터프레임을 불러옵니다.

In [ ]:
# 고유한 엔티티 쌍의 개수를 구합니다.
unique_entity_pairs = SSAN_SP_DF.drop_duplicates(subset=['source', 'target'])
number_of_unique_pairs = unique_entity_pairs.shape[0]

print(f"Number of unique entity pairs: {number_of_unique_pairs}")

# 고유한 엔티티 쌍의 개수를 구합니다.
unique_entity_pairs = GPT_SP_DF.drop_duplicates(subset=['source', 'target'])
number_of_unique_pairs = unique_entity_pairs.shape[0]

print(f"Number of unique entity pairs: {number_of_unique_pairs}")

'SSAN_SP_DF' 데이터프레임에는 10252개의 고유한 entity pair가 있고 'GPT_SP_DF' 데이터프레임에는 25326개의 고유한 entity pair가 있습니다.

In [ ]:
print(f'number of unique "edge" values in SSAN_SP_DF: {SSAN_SP_DF["edge"].nunique()}')
print(f'number of unique "relation" values in GPT_SP_DF: {GPT_SP_DF["relation"].nunique()}')

* SSAN_SP_DF의 relation별 갯수를 막대 그래프로 시각화 합니다.

In [ ]:
# bar plot relation type counts in descending order for SSAN_SP_DF
# 'relation' 열에서 각 관계의 종류별로 개수를 세고, 내림차순으로 정렬합니다.
relation_counts = SSAN_SP_DF['edge'].value_counts().sort_values()

# 가로 막대 그래프를 그립니다.
plt.figure(figsize=(15, 20))  # 그래프의 크기를 조정합니다.
relation_counts.plot(kind='barh',color='slategray')  # 수평 막대 그래프
plt.title('Relation Type Counts in Descending Order')
plt.xlabel('Count')
plt.ylabel('Relation Type')

# 긴 레이블을 다루기 위해 x축 레이블의 각도를 조정할 수 있습니다.
plt.xticks(rotation=45) 
plt.tight_layout()  # 레이아웃을 조정하여 레이블이 잘리지 않도록 합니다.
plt.show()

* GPT_SP_DF의 relation별 갯수를 막대 그래프로 시각화 합니다. 

In [ ]:
# bar plot relation type counts in descending order for GPT_SP_DF
# 'relation' 열에서 각 관계의 종류별로 개수를 세고, 내림차순으로 정렬합니다.
relation_counts = GPT_SP_DF['relation'].value_counts().sort_values()

# 가로 막대 그래프를 그립니다.
plt.figure(figsize=(15, 20))  # 그래프의 크기를 조정합니다.
relation_counts.plot(kind='barh',color='slategray')  # 수평 막대 그래프
plt.title('Relation Type Counts in Descending Order')
plt.xlabel('Count')
plt.ylabel('Relation Type')

# 긴 레이블을 다루기 위해 x축 레이블의 각도를 조정할 수 있습니다.
plt.xticks(rotation=45) 
plt.tight_layout()  # 레이아웃을 조정하여 레이블이 잘리지 않도록 합니다.
plt.show()

In [ ]:
# 임계값에 따른 네트워크 정보를 계산하는 함수
def network_info(df, source_col, target_col):
    # 그래프 생성
    # convert tabular data into a graph format rows = edges
    G = nx.from_pandas_edgelist(df, source=source_col, target=target_col)
    # 네트워크 정보 계산
    # calculate number of nodes and edges
    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()
    # computes the density of the graph
    density = nx.density(G)
    # calculate average degree of the graph
    avg_degree = sum(dict(G.degree()).values()) / num_nodes
    # determine the number of connected components in the graph
    num_components = nx.number_connected_components(G)
    
    return {
        'num_nodes': num_nodes,
        'num_edges': num_edges,
        'density': density,
        'average_degree': avg_degree,
        'connected_components': num_components
    }

* SSAN_SP_DF 데이터프레임의 네트워크 정보를 추출합니다.

In [ ]:
# for SSAN_SP_DF
# 각 임계값에 따른 네트워크 정보 추출
# initialize an empty dictionary
network_stats = {}
thresholds = ['95', '90', '85']
for t in thresholds:
    network_stats[f'{t}%'] = network_info(SSAN_SP_DF, f'source_rep_{t}', f'target_rep_{t}')

# 결과를 데이터프레임으로 변환
network_stats_df = pd.DataFrame(network_stats).T
network_stats_df.reset_index(inplace=True)
network_stats_df.rename(columns={'index': 'Raw'}, inplace=True)

network_stats_df

* GPT_SP_DF 데이터프레임의 네트워크 정보를 추출합니다.

In [ ]:
# for GPT_SP_DF
# 각 임계값에 따른 네트워크 정보 추출
network_stats = {}
thresholds = ['95', '90', '85']
for t in thresholds:
    network_stats[f'{t}%'] = network_info(GPT_SP_DF, f'source_rep_{t}', f'target_rep_{t}')

# 결과를 데이터프레임으로 변환
network_stats_df = pd.DataFrame(network_stats).T
network_stats_df.reset_index(inplace=True)
network_stats_df.rename(columns={'index': 'Raw'}, inplace=True)

network_stats_df

* GPT_SP_DF2 데이터프레임의 네트워크 정보를 추출합니다.

In [ ]:
# for GPT_SP_DF2
# 각 임계값에 따른 네트워크 정보 추출
network_stats = {}
thresholds = ['95', '90', '85']
for t in thresholds:
    network_stats[f'{t}%'] = network_info(GPT_SP_DF2, f'source_rep_{t}', f'target_rep_{t}')

# 결과를 데이터프레임으로 변환
network_stats_df = pd.DataFrame(network_stats).T
network_stats_df.reset_index(inplace=True)
network_stats_df.rename(columns={'index': 'Raw'}, inplace=True)

network_stats_df

* Similarity threshold별 representative words의 분포를 시각화합니다.

In [ ]:
# 예시 임계값 리스트 (이 부분은 실제 임계값 리스트로 대체해야 함)
thresholds = [95, 90, 85, 80, 75, 70, 65, 60]

# 예시 데이터프레임 생성 (이 부분은 실제 데이터프레임으로 대체해야 함)
# SSAN_SP_DF = ...

# 그래프 생성
fig, ax = plt.subplots(figsize=(15, 10))  # 전체 그래프 크기 조정
# y축에 해당하는 고유 대표 단어들을 저장하는 리스트
y_axis_words = []

# 각 임계값에 대해 반복
for threshold in thresholds:
    # unique_representative 값 설정
    unique_representatives = pd.unique(SSAN_SP_DF[f'source_rep_{threshold}'].tolist() + 
                                       SSAN_SP_DF[f'target_rep_{threshold}'].tolist())
    # y축에 매핑
    y_positions = pd.Series(range(len(unique_representatives)), index=unique_representatives)

    # y축 단어 리스트에 추가
    y_axis_words.extend(unique_representatives)
    
    # 데이터 프레임 생성
    df_mapped = SSAN_SP_DF[[f'source_rep_{threshold}', f'target_rep_{threshold}']].melt(var_name='type', value_name='Representative')
    df_mapped['y_position'] = df_mapped['Representative'].map(y_positions)
    
    # 데이터 프레임 sorting
    sorted_df = df_mapped.sort_values(by=['y_position', 'Representative'])
    sorted_df = sorted_df.reset_index(drop=True)
    
    # 그래프에 데이터 추가 (각 임계값에 대해 다른 색상 사용)
    ax.scatter(sorted_df.index, sorted_df['y_position'], label=f'{threshold}% Similarity Threshold', s=1)

# 축 레이블 및 제목 설정
ax.set_xlabel('Words before Normalization')
ax.set_ylabel('Representative Words (Groups)')
ax.set_title('Representative Words at Various Similarity Thresholds [SSAN_SP_DF]')


# 레전드 설정 (점 크기 증가)
ax.legend(markerscale=5, scatterpoints=1, fontsize='medium')

plt.tight_layout()
plt.savefig("combined_similarity_plot.png")
plt.show()

* Similarity threshold별 representative words의 분포를 시각화합니다.

In [ ]:
# 예시 임계값 리스트 (이 부분은 실제 임계값 리스트로 대체해야 함)
thresholds = [100, 95, 90, 85, 80, 75, 70, 65]

GPT_SP_DF['source_rep_100']= GPT_SP_DF['source']
GPT_SP_DF['target_rep_100']= GPT_SP_DF['target']

# 예시 데이터프레임 생성 (이 부분은 실제 데이터프레임으로 대체해야 함)
# SSAN_SP_DF = ...

# 그래프 생성
fig, ax = plt.subplots(figsize=(70, 20))  # 전체 그래프 크기 조정
# y축에 해당하는 고유 대표 단어들을 저장하는 리스트
y_axis_words = []

# 각 임계값에 대해 반복
for threshold in thresholds:
    unique_representatives = pd.unique(GPT_SP_DF['source'].tolist() + 
                                       GPT_SP_DF['target'].tolist())
    y_positions = pd.Series(range(len(unique_representatives)), index=unique_representatives)

    # y축 단어 리스트에 추가
    y_axis_words.extend(unique_representatives)
    
    df_mapped = GPT_SP_DF[[f'source_rep_{threshold}', f'target_rep_{threshold}']].melt(var_name='type', value_name='Representative')
    df_mapped['y_position'] = df_mapped['Representative'].map(y_positions)
    
    # sorted_df = df_mapped.sort_values(by=['y_position', 'Representative'])

    sorted_df = df_mapped.reset_index(drop=True)
    
    # 그래프에 데이터 추가 (각 임계값에 대해 다른 색상 사용)
    ax.scatter(sorted_df.index, sorted_df['y_position'], label=f'{threshold}% Similarity Threshold', s=1)

# 축 레이블 및 제목 설정
ax.set_xlabel('Words before Normalization')
ax.set_ylabel('Representative Words (Groups)')
ax.set_title('Representative Words at Various Similarity Thresholds')


# 레전드 설정 (점 크기 증가)
ax.legend(markerscale=5, scatterpoints=1, fontsize='medium')

plt.tight_layout()
# plt.savefig("GPT_combined_similarity_plot.png")
plt.show()

* Similarity threshold별 representative words의 변화를 시각화합니다.

In [ ]:
# 예시 임계값 리스트 (실제 임계값 리스트로 대체해야 함)
thresholds = [100, 95, 90, 85, 80, 75, 70, 65]

# 예시 데이터프레임 생성 (실제 데이터프레임으로 대체해야 함)
# GPT_SP_DF = ...

# 그래프 생성
fig, ax = plt.subplots(figsize=(20, 15))  # 전체 그래프 크기 조정

# 각 임계값에 대해 반복
for threshold in thresholds:
    # 대표 단어와 원본 단어를 매핑하는 데이터프레임 생성
    df_mapped = GPT_SP_DF[['source', f'source_rep_{threshold}', 'target', f'target_rep_{threshold}']].melt(var_name='type', value_name='Representative')

    # 단어와 대표 단어의 관계를 x, y 값으로 사용
    if threshold == 100:
        # 임계값이 100일 경우, 단어는 자신을 대표하므로 y=x 선을 따라 표시
        ax.plot(list(set(GPT_SP_DF['source'].to_list()+GPT_SP_DF['target'].to_list())), list(set(GPT_SP_DF['source'].to_list()+GPT_SP_DF['target'].to_list())), 'o', label=f'{threshold}% Similarity Threshold', markersize=1)
    else:
        # 그 외의 경우, 원본 단어의 인덱스와 대표 단어의 인덱스를 사용
        df_mapped['Word_Index'] = df_mapped.groupby('Representative').ngroup()
        ax.scatter(df_mapped['Word_Index'], df_mapped['Representative'], label=f'{threshold}% Similarity Threshold', s=1)

# 레전드 및 제목 설정
ax.legend(markerscale=5, scatterpoints=1, fontsize='medium')
ax.set_title('Word Representation Change with Different Similarity Thresholds')
ax.set_xlabel('Word Index')
ax.set_ylabel('Representative Words')

plt.tight_layout()
plt.show()

* threshold별 word transformation을 시각화합니다.

In [ ]:
# threshold별로 column 이름 리스트 저장
source_cols = [f"source_rep_{t}" for t in thresholds]
target_cols = [f"target_rep_{t}" for t in thresholds]

transformation_data = []

# threshold loop
for t in thresholds:
    for index, row in GPT_SP_DF.iterrows():
        # Source transformations
        transformation_data.append({
            "Word": row["source"],
            "Transformed_Word": row[f"source_rep_{t}"],
            "Threshold": t,
            "Type": "Source"
        })
        
        # Target transformations
        transformation_data.append({
            "Word": row["target"],
            "Transformed_Word": row[f"target_rep_{t}"],
            "Threshold": t,
            "Type": "Target"
        })

# Creating a DataFrame from the list of dictionaries
transformations = pd.DataFrame(transformation_data)

# Plotting the transformations
# ... (plotting code remains the same)


# Plotting the transformations
plt.figure(figsize=(100, 100))
sns.scatterplot(data=transformations, x="Word", y="Transformed_Word", hue="Threshold", style="Type")
plt.xticks(rotation=45)
plt.title("Word Transformations Across Different Thresholds")
plt.xlabel("Original Word")
plt.ylabel("Transformed Word")
plt.show()

* Similarity threshold별 word pair mapping의 변화를 시각화합니다.

In [ ]:
# Example data loading (replace this with your actual data loading)
# SSAN_SP_DF = pd.read_csv('your_data.csv')

# Combine 'source' and 'target' columns for x-axis
SSAN_SP_DF['words_before'] = SSAN_SP_DF['source'] + ' + ' + SSAN_SP_DF['target']

# Create a figure for the plot
fig, ax = plt.subplots(figsize=(15, 10))

# Iterate over each threshold and plot the data
thresholds = [80, 75, 70, 65, 60]
for threshold in thresholds:
    # Combine the representative words for the current threshold
    SSAN_SP_DF[f'words_after_{threshold}'] = SSAN_SP_DF[f'source_rep_{threshold}'] + ' + ' + SSAN_SP_DF[f'target_rep_{threshold}']
    
    # Create a mapping of words_before to words_after
    mapping = SSAN_SP_DF.groupby('words_before')[f'words_after_{threshold}'].unique().apply(lambda x: ', '.join(x))
    
    # Plot the data
    ax.scatter(mapping.index, mapping, label=f'{threshold}% Similarity Threshold', s=1)

ax.set_xlabel('Original Words (source + target)')
ax.set_ylabel('Mapped Words (source_rep + target_rep)')
ax.set_xticklabels(SSAN_SP_DF['words_before'], rotation=90, fontsize=5)
ax.legend()
ax.set_title('Change in Representative Words Across Different Similarity Thresholds')

plt.tight_layout()
# plt.savefig("threshold_comparison_plot.png")
plt.show()

* Similarity threshold별로 normalizing 후 단어들의 변화를 시각화 합니다.

In [ ]:
"""
# Extracting the thresholds and preparing the dataframe for plotting
thresholds = [95, 90, 85, 80, 75, 70, 65, 60]

# Merging source and target words before and after normalization
df['words_before'] = df['source']  + df['target']
for threshold in thresholds:
    df[f'words_after_{threshold}'] = df[f'source_rep_{threshold}'] + ' ' + df[f'target_rep_{threshold}']

# Preparing for plotting
fig, ax = plt.subplots(figsize=(15, 10))

# Plotting the data for each threshold
for threshold in thresholds:
    # Extracting words before and after for the current threshold
    words_before = df['words_before'].str.split().explode().unique()
    words_after = df[f'words_after_{threshold}'].str.split().explode().unique()
    
    # Mapping words_before to their index to align them on the x-axis
    x_positions = pd.Series(range(len(words_before)), index=words_before)
    
    # Mapping words_after to the index of words_before to get their y-axis position
    y_positions = pd.Series(x_positions.index, index=x_positions.values).loc[words_after].values
    
    # Plotting
    ax.plot(x_positions, y_positions, marker='o', linestyle='', label=f'{threshold}% Similarity Threshold')

# Setting labels and title
ax.set_xlabel('Words Before Normalization')
ax.set_ylabel('Words After Normalization (by similarity threshold)')
ax.set_title('Word Changes Across Different Similarity Thresholds')

# Adding a legend
ax.legend()

# Show the plot
plt.show()
"""

* Similarity threshold별 word normalization을 시각화합니다.

In [ ]:
# 그래프 생성

fig, ax = plt.subplots(figsize=(15, 10))

# 각 임계값에 대해 반복
for threshold in thresholds:
    source_col = f'source_rep_{threshold}'
    target_col = f'target_rep_{threshold}'

    # source 및 target 정규화된 단어 그리기
    ax.scatter(SSAN_SP_DF['source'], SSAN_SP_DF[source_col], label=f'Source {threshold}% Similarity', alpha=0.5)
    ax.scatter(SSAN_SP_DF['target'], SSAN_SP_DF[target_col], label=f'Target {threshold}% Similarity', alpha=0.5)

# 축 레이블 및 제목 설정
ax.set_xlabel('Words before Normalization')
ax.set_ylabel('Words after Normalization')
ax.set_title('Word Normalization at Various Similarity Thresholds')
ax.legend()

plt.tight_layout()
# plt.savefig("word_normalization_plot.png")
plt.show()

* Similarity threshold별 'SSAN_SP_DF' 데이터프레임의 representative grouping의 분포를 시각화합니다.

In [ ]:
# 겹치지 않게 모든 데이터프레임 시각화
fig, axs = plt.subplots(len(thresholds), 1, figsize=(15, 40))  # 그래프 크기 조정

for i, threshold in enumerate(thresholds):
    print(threshold)
    # 임계값별로 대표 단어에 대한 y 위치를 조정합니다.
    unique_representatives = pd.unique(SSAN_SP_DF[f'source_rep_{threshold}'].tolist() + 
                                       SSAN_SP_DF[f'target_rep_{threshold}'].tolist())
    y_positions = pd.Series(range(len(unique_representatives)), index=unique_representatives)
    
    # 모든 source와 target의 대표 단어를 매핑합니다.
    df_mapped = SSAN_SP_DF[[f'source_rep_{threshold}', f'target_rep_{threshold}']].melt(var_name='type', value_name='Representative')
    df_mapped['y_position'] = df_mapped['Representative'].map(y_positions)


     # 데이터를 그룹화하고, 각 그룹별로 정렬합니다.
    sorted_df = df_mapped.sort_values(by=['y_position', 'Representative'])
    sorted_df = sorted_df.reset_index(drop=True)
    # 각 임계값에 대한 대표 단어의 분포를 시각화
    axs[i].scatter(sorted_df.index, sorted_df['y_position'], label=f'{threshold}% Similarity Threshold', s=1)
    axs[i].set_yticks(y_positions)
    axs[i].set_yticklabels(y_positions.index, fontsize=5)
    axs[i].legend()
    axs[i].set_title(f'Representative Words at {threshold}% Similarity Threshold')
    axs[i].set_ylabel('Representative Words (Groups)')

plt.tight_layout()
plt.savefig("sample_plot.png")
plt.show()

* 'SSAN_SP_DF' 데이터프레임의 edge 수를 그래프로 시각화합니다.

In [ ]:
# 'edge' 열의 각 종류별 개수를 계산
edge_counts = SSAN_SP_DF['edge'].value_counts()

# 개수가 많은 순서대로 정렬
edge_counts = edge_counts.sort_values(ascending=True)
# 그래프 크기 설정
plt.figure(figsize=(14,10))
# 막대 그래프 시각화
plt.barh(edge_counts.index, edge_counts.values)
plt.xlabel('Count')
plt.ylabel('Edge Type')
plt.title('Relation Type Counts in Descending Order')
plt.show()

* Similarity threshold별로 unique entity 개수를 세고 시각화합니다.

In [ ]:
# 임계값별 entity 개수
unique_entities_counts =[]
thresholds = [95, 90, 85, 80, 75, 70, 65, 60]

for i, threshold in enumerate(thresholds):
    #print(threshold)
    # 임계값별로 대표 단어에 대한 y 위치를 조정합니다.
    unique_representatives = pd.unique(SSAN_SP_DF[f'source_rep_{threshold}'].tolist() + 
                                       SSAN_SP_DF[f'target_rep_{threshold}'].tolist())
    unique_entities_counts.append(len(unique_representatives))
    print(f"Threshold {threshold}%: {len(unique_representatives)} unique entities")
    # 선 그래프 생성
thresholds_with_raw = ['Raw'] + [str(t) + '%' for t in thresholds]
unique_entities_counts_with_raw = [9079] + unique_entities_counts
plt.figure(figsize = (10, 6))
plt.plot(thresholds_with_raw, unique_entities_counts_with_raw, marker = 'o')
plt.title('Number of Unique Entities by Similarity Threshold')
plt.xlabel('Similarity Threshold (%)')
plt.ylabel('Number of Unique Entities')
plt.grid(True)
plt.xticks(thresholds_with_raw)

#plt.savefig('unique_entities_by_similarity.png')
plt.show()

* Similarity threshold별 Graph complexity(노드의 수) 변화를 시각화 합니다.

In [ ]:
# 예시 데이터 (실제 데이터로 교체 필요)
similarity_thresholds = ['Raw', '95%', '90%', '85%', '80%', '75%', '70%', '65%', '60%']
thresholds = ['Raw', 95, 90, 85, 80, 75, 70, 65, 60]  # 'Raw' 추가

# 각 임계값에 대해 노드와 엣지의 수를 저장할 리스트 초기화
number_of_nodes = []
number_of_edges = []

# 데이터 프레임 (실제 사용 시 SSAN_SP_DF 데이터프레임으로 대체)
# SSAN_SP_DF = ...

for threshold in thresholds:
    # 'Raw' 데이터에 대한 처리
    if threshold == 'Raw':
        G = nx.DiGraph()
        for s, t in zip(SSAN_SP_DF['source'], SSAN_SP_DF['target']):
            G.add_edge(s, t)
    else:
        # 각 임계값에 대한 대표 노드 칼럼을 가져옵니다.
        source = SSAN_SP_DF[f'source_rep_{threshold}']
        target = SSAN_SP_DF[f'target_rep_{threshold}']
        G = nx.DiGraph()
        for s, t in zip(source, target):
            G.add_edge(s, t)

    # 노드와 엣지의 수를 리스트에 추가
    number_of_nodes.append(G.number_of_nodes())
    number_of_edges.append(G.number_of_edges())

fig, ax1 = plt.subplots(figsize=(12, 7))

# 막대 그래프 생성 (노드의 수)
bars = ax1.bar(similarity_thresholds, number_of_nodes, color='lightblue')
ax1.set_xlabel('Similarity Threshold (%)')
ax1.set_ylabel('Number of Nodes', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# 바 속에 각 막대의 값 표시
for bar in bars:
    yval = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2, yval - (yval*0.05), yval, ha='center', va='top', color='blue')

# 두 번째 y축 추가 (엣지의 수)
ax2 = ax1.twinx()  
lines = ax2.plot(similarity_thresholds, number_of_edges, color='grey', marker='o', linestyle='-', linewidth=2)
ax2.set_ylabel('Number of Edges', color='grey')  
ax2.tick_params(axis='y', labelcolor='grey')

# 점 위에 각 점의 값 표시
for i, (xy, txt) in enumerate(zip(lines[0].get_xydata(), number_of_edges)):
    ax2.annotate(txt, xy=xy, textcoords="offset points", xytext=(0,10), ha='center')

# 제목 추가
plt.title('Graph Size Change by Similarity Threshold')

# 그래프 저장
plt.savefig('graph_size_change.png')

# 그래프 표시
plt.show()

* Similarity threshold별 silhouette coefficient를 시각화합니다.

In [ ]:
# Silhouette coefficient vs threshold 그래프
# silhouette score, thresholds 지정
silhouette_scores= [0.7684047, 0.76462686, 0.71767795, 0.56971604, 0.5110331, 0.4427302, 0.36146897, 0.29856825]
thresholds = ['95%', '90%', '85%', '80%', '75%', '70%', '65%', '60%']
plt.figure(figsize=(10, 6))
plt.plot(thresholds, silhouette_scores, marker='o')
plt.xlabel('Threshold')
plt.ylabel('Silhouette Coefficient')
plt.title('Silhouette Coefficient vs Threshold')
plt.grid(True)

# 결과 저장
plt.savefig('silhouette_coefficient_vs_threshold.png')  # 저장 경로
plt.show()

# 4. SSAN & GPT Graph Visualization

SSAN 모델을 통해 분석한 데이터와 GPT를 통해 획득한 데이터를 EDA를 통해 특성을 파악합니다. 각각의 데이터프레임에서 threshold별, node별로 출발지와 목적지 정보를 기반으로 그래프를 생성하고 시각화하는 기능을 수행합니다. 이를 통해 supply chain network를 분석하고 시각화하는 기능을 수행할 수 있습니다.

### Data EDA

데이터 파일을 import합니다.

In [ ]:
SSAN_SP_DF = pd.read_csv('SSAN_SP_DF_after_norm.csv')
GPT_SP_DF2 = pd.read_csv('GPT_SP_DF_after_norm2.csv')

필요 없는 열을 제거합니다.

In [ ]:
SSAN_SP_DF = SSAN_SP_DF.drop(['source_ner_2','target_ner_2','source_ner_3','target_ner_3'],axis=1)
GPT_SP_DF2 = GPT_SP_DF2.drop(['para','source_ner_2','target_ner_2','source_ner_3','target_ner_3','source_ner_4','target_ner_4','sorted_pair'],axis=1) #GPT_SP_DF2에서 'para','source_ner_2','target_ner_2','source_ner_3','target_ner_3','source_ner_4','target_ner_4','sorted_pair'열 제거

dc_check: 방향성이 있는 그래프를 생성하고 degree_centrality를 측정하여 중심성을 기준으로 내림차순으로 정리합니다.<br> 상위 30개 노드 선별 하고 각 카테고리에 대한 상위 30개 노드 정보 포함하는 dataframe을 생성하여 digraph와 생성한 dataframe를 return합니다.

In [ ]:
#방향성이 있는 그래프를 생성
def dc_check(fin_df, edge_name):
    # Creating a directed graph(방향성이 있는 그래프 생성)
    G = nx.DiGraph()

    # Adding edges from the DataFrame
    for _, row in fin_df.iterrows():
        G.add_edge(row['source'], row['target'], edge=row[edge_name]) #edge 추가

    # Calculate centrality measures (e.g., degree centrality) 노드에 각 degree centrality(각 노드가 다른 노드랑 얼마나 연결되어 있는가)측정 
    centrality = nx.degree_centrality(G)

    # Preparing to categorize nodes by their NER tags, 
    # #디폴트값이 list인 딕션너리 생성
    node_categories = defaultdict(list)

    # Categorizing nodes
    for node in G.nodes:
        # Finding the node in the DataFrame and categorizing it, 
        # #해당하는 변수에 배당
        if node in fin_df['source'].values:
            category = fin_df[fin_df['source'] == node]['source_ner'].values[0]
        elif node in fin_df['target'].values:
            category = fin_df[fin_df['target'] == node]['target_ner'].values[0]
        else:
            category = 'Unknown'
        
        node_categories[category].append(node)

    # # Sorting nodes in each category by centrality, 
    # 중심성을 기준으로 내림차순으로 정리 -> 상위 30개 노드 선별
    top_centrality_by_category = {category: sorted(nodes, key=lambda x: centrality[x], reverse=True)[:30]
                                for category, nodes in node_categories.items()}

    # top_centrality_by_category
    # Converting the top_centrality_by_category dictionary to a DataFrame, 
    # 카테고리에 대한 상위 30개 노드 정보 포함하는 DF
    df_top_centrality = pd.DataFrame([{"Category": category, "Node": node, "Centrality": centrality[node]}
                                    for category, nodes in top_centrality_by_category.items()
                                    for node in nodes])

    return G, df_top_centrality


import한 데이터프레임인 SSAN_SP_DF와 GPT_SP_DF2를 df_check함수에 대입합니다.

In [ ]:
ssan_G, ssan_dc_df = dc_check(SSAN_SP_DF, 'edge')
gpt_G, gpt_dc_df = dc_check(GPT_SP_DF2, 'relation')

결과값인 ssan_dc_df와 gpt_dc_df를 엑셀로 변환합니다.

In [ ]:
ssan_dc_df.to_excel('ssan_raw_dc.xlsx',index = False)
gpt_dc_df.to_excel('gpt_raw_dc.xlsx',index = False)

dc_check_threshold: dc_check과 동일한 매커니즘이지만 입력값으로 thre(:threshold 값)를 하나 더 입력 받아 thre별로 dc_check 수행합니다.

In [ ]:
def dc_check_threshold(fin_df, thre, edge_name):
    # Creating a directed graph
    G = nx.DiGraph()

    # Adding edges from the DataFrame
    for _, row in fin_df.iterrows():
        G.add_edge(row['source_rep_{}'.format(thre)], row['target_rep_{}'.format(thre)], edge=row[edge_name])

    # Calculate centrality measures (e.g., degree centrality)
    centrality = nx.degree_centrality(G)

    # Preparing to categorize nodes by their NER tags
    node_categories = defaultdict(list)

    # Categorizing nodes
    for node in G.nodes:
        # Finding the node in the DataFrame and categorizing it
        if node in fin_df['source_rep_{}'.format(thre)].values:
            category = fin_df[fin_df['source_rep_{}'.format(thre)] == node]['source_ner'].values[0]
        elif node in fin_df['target_rep_{}'.format(thre)].values:
            category = fin_df[fin_df['target_rep_{}'.format(thre)] == node]['target_ner'].values[0]
        else:
            category = 'Unknown'
        
        node_categories[category].append(node)

    # Sorting nodes in each category by centrality
    top_centrality_by_category = {category: sorted(nodes, key=lambda x: centrality[x], reverse=True)[:30]
                                for category, nodes in node_categories.items()}

    # top_centrality_by_category
    # Converting the top_centrality_by_category dictionary to a DataFrame
    df_top_centrality = pd.DataFrame([{"Category": category, "Node": node, "Centrality": centrality[node]}
                                    for category, nodes in top_centrality_by_category.items()
                                    for node in nodes])

    return G, df_top_centrality

SSAN_SP_DF와 GPT_SP_DF2를 dc_check_threshold를 thre(threshold)별로 수행합니다. 

In [ ]:
ssan_G_95, ssan_dc_df_95= dc_check_threshold(SSAN_SP_DF,95, 'edge')
gpt_G_95, gpt_dc_df_95 = dc_check_threshold(GPT_SP_DF2,95, 'relation')

ssan_G_90, ssan_dc_df_90= dc_check_threshold(SSAN_SP_DF,90, 'edge')
gpt_G_90, gpt_dc_df_90 = dc_check_threshold(GPT_SP_DF2,90, 'relation')

ssan_G_85, ssan_dc_df_85= dc_check_threshold(SSAN_SP_DF,85, 'edge')
gpt_G_85, gpt_dc_df_85 = dc_check_threshold(GPT_SP_DF2,85, 'relation')

ssan_G_80, ssan_dc_df_80= dc_check_threshold(SSAN_SP_DF,80, 'edge')
gpt_G_80, gpt_dc_df_80 = dc_check_threshold(GPT_SP_DF2,80, 'relation')

statistic1: node_count: node수 측정, edge_count: edge 수 측정, density: 밀도 측정, connected_components: 그래프에서 약한 구성요소의 수를 계산하여 node_count, edge_count, density, average_degree를 각각 출력하고 반환합니다.

In [ ]:
def statistic1(filtered_G):
# 노드 수와 엣지 수
    node_count = filtered_G.number_of_nodes()
    edge_count = filtered_G.number_of_edges()

    # 그래프 밀도
    density = nx.density(filtered_G)

    # 평균 차수
    average_degree = sum(dict(filtered_G.degree()).values()) / node_count

    # 연결된 구성 요소 수
    connected_components = nx.number_weakly_connected_components(filtered_G)

    print('node_count: ',node_count,'\nedge_count: ', edge_count,'\ndensity: ', density, 
          '\naverage degree: ',average_degree,'\nconnected components: ', connected_components, 
          )
    return node_count, edge_count, density, average_degree, connected_components

In [ ]:
statistic1(ssan_G)
statistic1(ssan_G_95)
statistic1(ssan_G_90)
statistic1(ssan_G_85)

statistic1(gpt_G)
statistic1(gpt_G_95)
statistic1(gpt_G_90)
statistic1(gpt_G_85)

dc_check_threshold의 각 출력값을 엑셀로 변환합니다.

In [ ]:
ssan_dc_df_95.to_excel('ssan_95_dc.xlsx',index = False)
gpt_dc_df_95.to_excel('gpt_95_dc.xlsx',index = False)

ssan_dc_df_90.to_excel('ssan_90_dc.xlsx',index = False)
gpt_dc_df_90.to_excel('gpt_90_dc.xlsx',index = False)

ssan_dc_df_85.to_excel('ssan_85_dc.xlsx',index = False)
gpt_dc_df_85.to_excel('gpt_85_dc.xlsx',index = False)

ssan_dc_df_80.to_excel('ssan_80_dc.xlsx',index = False)
gpt_dc_df_80.to_excel('gpt_80_dc.xlsx',index = False)

SP_DF 데이터프레임 생성합니다.

In [ ]:
#열 이름 변경: relation -> edge
GPT_SP_DF2.rename(columns={'relation': 'edge'}, inplace=True)
#SSAN_SP_DF와 GPT_SP_DF2를 행 방향으로 연결해서 새로운 DF 생성
SP_DF = pd.concat([SSAN_SP_DF, GPT_SP_DF2], ignore_index=True)
#SP_DF 중복행 제거 후 인덱스 재설정
SP_DF = SP_DF.drop_duplicates().reset_index(drop=True)

In [ ]:
SP_DF = SP_DF.drop_duplicates(subset=['source', 'target'], keep='first').reset_index(drop=True) #SP_DF에서 'source' 및 'target' 열을 기준으로 중복된 행을 제거 -> 동일한 행이 있는 경우 첫 번째 발생만 유지되고 나머지 중복 행이 제거 -> 새로운 인덱스 생성
SP_DF

SP_DF 데이터프레임을 thre(threshold) 별로 dc_check 함수를 통해 그래프 생성합니다.

In [ ]:
com_G, com_dc_df = dc_check(SP_DF, 'edge')


In [ ]:
com_G_95, com_dc_df_95= dc_check_threshold(SP_DF,95, 'edge')

com_G_90, com_dc_df_90= dc_check_threshold(SP_DF,90, 'edge')

com_G_85, com_dc_df_85= dc_check_threshold(SP_DF,85, 'edge')


dc_check 함수 출력값인 thre(threshold) 별 각 그래프를 statistic1 함수를 통해 데이터 추출합니다.

In [ ]:
statistic1(com_G)
statistic1(com_G_95)
statistic1(com_G_90)
statistic1(com_G_85)

SSAN_SP_DF의 thre(threshold) 별 교집합을 벤다이어그램을 통해 시각화합니다.

In [ ]:
#SSAN_SP_DF에 'source'와 'target' 열에서 얻은 두 리스트의 합집합을 사용하여 집합을 생성
setA = set(SSAN_SP_DF['source'].to_list()+SSAN_SP_DF['target'].to_list())
#SSAN_SP_DF에 'source_rep_95'와 'target_rep_95' 열에서 얻은 두 리스트의 합집합을 사용하여 집합을 생성
setB = set(SSAN_SP_DF['source_rep_95'].to_list()+SSAN_SP_DF['target_rep_95'].to_list())
#SSAN_SP_DF에 'source_rep_05'와 'target_rep_90' 열에서 얻은 두 리스트의 합집합을 사용하여 집합을 생성
setC = set(SSAN_SP_DF['source_rep_90'].to_list()+SSAN_SP_DF['target_rep_90'].to_list())

# 벤 다이어그램 그리기
#venn3 함수를 이용하여 벤다이어그램 생성 -> setA, setB, setC라는 세 개의 집합을 가정하고 이들의 교집합을 Venn 다이어그램으로 시각화
venn3([setA, setB, setC], ('SSAN', 'ChatGPT-small spacy', 'Set C-large spacy'))

# 그래프 표시
plt.show()

In [ ]:
setA = set(SSAN_SP_DF['source_rep_90'].to_list()+SSAN_SP_DF['target_rep_90'].to_list())
setB = set(SSAN_SP_DF['source_rep_85'].to_list()+SSAN_SP_DF['target_rep_85'].to_list())
setC = set(SSAN_SP_DF['source_rep_80'].to_list()+SSAN_SP_DF['target_rep_80'].to_list())

# 벤 다이어그램 그리기
venn3([setA, setB, setC], ('SSAN', 'ChatGPT-small spacy', 'Set C-large spacy'))

# 그래프 표시
plt.show()

Graph_and_Visual: 각 노드간의 관계 그래프를 시각화하는 함수를 정의합니다.

In [ ]:
def Graph_and_Visual(fin_df, source, target, isin_ner_li, Title, pos):
    #Country' 또는 'Firm'으로 태그된 노드만 포함하는 행을 필터링(예시)
    #fin_df에서 'source_ner' 및 'target_ner' 열의 값이 리스트 isin_ner_li에 있는 경우에 해당하는 행만 선택
    filtered_df = fin_df[(fin_df['source_ner'].isin(isin_ner_li)) & (fin_df['target_ner'].isin(isin_ner_li))]

    # 필터링된 데이터프레임을 기반으로 그래프 다시 생성
    #from_pandas_edgelist 함수를 사용하여 필터링된 DataFrame(filtered_df)을 기반으로 유향 그래프(DiGraph)를 생성
    #그래프는 DataFrame의 'source' 및 'target' 열을 사용하여 구성되며, 'edge' 열은 간선 속성으로 할당
    filtered_G = nx.from_pandas_edgelist(filtered_df, source=source, target=target, edge_attr='edge', create_using=nx.DiGraph())
    
    # 색상 매핑 (이전에 정의된 'Country'와 'Firm'에 대한 색상만 사용)
    #제공된 color_map_filtered 사전은 그래프에서 다른 노드 유형에 대한 노드 색상을 지정하는 데 사용
    color_map_filtered = {'Country': 'royalblue', 'Firm': 'lightgreen','Resource': 'orange','Technology': 'darkred'}

    # 필터링된 그래프의 노드에 NER 태그 속성 추가
    # #filtered_df DataFrame의 행을 반복하며, 각 행의 정보를 기반으로 filtered_G 그래프의 노드의 'ner' 속성을 업데이트

    #filtered_df DataFrame의 각 행을 반복
    for _, row in filtered_df.iterrows(): 
        #현재 행의 source 노드가 filtered_G 그래프의 노드 중에 있는지 확인
        if row[source] in filtered_G.nodes:
            #source 노드가 존재하는 경우 해당 노드의 'ner' 속성을 DataFrame에서 가져온 'source_ner' 값으로 업데이트
            filtered_G.nodes[row[source]]['ner'] = row['source_ner']
        if row[target] in filtered_G.nodes: 
            filtered_G.nodes[row[target]]['ner'] = row['target_ner']

    # 노드 색상 할당 (필터링된 그래프에 대해)
    #filtered_G 그래프의 각 노드에 대한 노드 색상 목록(filtered_node_colors)을 생성
    #각 노드의 'ner' 속성을 기반으로 하며, 'ner' 속성을 특정 색상에 매핑하는 데 color_map_filtered 사전을 사용
    filtered_node_colors = [color_map_filtered[filtered_G.nodes[node]['ner']] for node in filtered_G.nodes]

    # 그래프 시각화
    plt.figure(figsize=(60,60))
    nx.draw(filtered_G, with_labels=True, node_color=filtered_node_colors, edge_color='gray', pos=pos, font_size=3, node_size=100)
    plt.title("Supply Chain Graph ({})".format(Title), fontdict={'fontsize': 100})
    plt.savefig("SSAN_graph_{}.png".format(source[-2:]))
    plt.show()
    return filtered_G

statistic: 앞서 선언한 statistic1 함수를 재구성(각 연결된 구성요소의 평균 최단 경로 길이 계산 추가)합니다.

In [ ]:
def statistic(filtered_G):
    # 필터링된 그래프의 통계 정보 재계산 (지름 제외)

    # 노드 수와 엣지 수
    node_count = filtered_G.number_of_nodes()
    edge_count = filtered_G.number_of_edges()

    # 그래프 밀도
    density = nx.density(filtered_G)

    # 평균 차수
    average_degree = sum(dict(filtered_G.degree()).values()) / node_count

    # 연결된 구성 요소 수
    connected_components = nx.number_weakly_connected_components(filtered_G)


    # 각 연결된 구성 요소의 평균 최단 경로 길이 계산

    average_path_lengths = []
    #유향 그래프 filtered_G의 약하게 연결된 컴포넌트를 순회
    for component in nx.weakly_connected_components(filtered_G):
        #현재의 약하게 연결된 컴포넌트에 해당하는 부분 그래프를 추출
        subgraph = filtered_G.subgraph(component)
        # 서브그래프가 강하게 연결되어 있지 않더라도 계산 가능한지 확인
        # 부분 그래프가 약하게 연결되어 있는지 확인
        if nx.is_weakly_connected(subgraph):
            # 모든 노드 쌍의 평균 최단 경로 길이 계산
            try:
                # 약하게 연결된 컴포넌트 내 모든 노드 쌍에 대한 최단 경로 길이를 계산
                lengths = dict(nx.all_pairs_shortest_path_length(subgraph))
                # 컴포넌트 내 모든 노드에 대한 최단 경로 길이의 총합을 계산
                total_length = sum(sum(lengths[n].values()) for n in lengths)
                # 약하게 연결된 컴포넌트에 대한 평균 경로 길이를 계산
                avg_length = total_length / (len(subgraph) * (len(subgraph) - 1))
                #평균 경로 길이를 average_path_lengths 리스트에 추가
                average_path_lengths.append(avg_length) 
            except Exception as e:
                # 오류 발생 시 0으로 설정
                #계산 중 오류가 발생한 경우에 대비하여 0 값을 리스트에 추가하여 예외를 처리
                average_path_lengths.append(0)

    # 통계 정보 출력 (지름 제외)
    print('node_count: ',node_count,'\nedge_count: ', edge_count,'\ndensity: ', density, 
          '\naverage degree: ',average_degree,'\nconnected components: ', connected_components, 
          '\naverage of average_path_lengths: ',sum(average_path_lengths)/len(average_path_lengths))
    return node_count, edge_count, density, average_degree, connected_components, sum(average_path_lengths)/len(average_path_lengths)





원본를 데이터 복사합니다.

In [ ]:
fin_df = SSAN_SP_DF.copy()
fin_df2 = GPT_SP_DF2.copy()

spring_layout()를 이용하여 노드의 위치를 저장합니다.

In [ ]:
#추가했음(방법2)
pos = nx.spring_layout(ssan_G)
pos_95 = nx.spring_layout(ssan_G_95)
pos_90 = nx.spring_layout(ssan_G_90)
pos_85 = nx.spring_layout(ssan_G_85)
pos_80 = nx.spring_layout(ssan_G_80)

### Data visualization

Graph_and_Visual 함수를 통해 thre(threshold) 별 노드 별 관계그래프를 시각화합니다.

### Country-Firm

country와 firm 사이 thre(threshold) 별 관계그래프를 시각화합니다.

In [ ]:
G = Graph_and_Visual(fin_df, 'source', 'target', ['Country','Firm'], 'Country and Firm',pos)
statistic(G)

In [ ]:
# 95
G = Graph_and_Visual(fin_df, 'source_rep_95', 'target_rep_95', ['Country','Firm'], 'Country and Firm',pos_95)
statistic(G)

In [ ]:
# 90
G = Graph_and_Visual(fin_df, 'source_rep_90', 'target_rep_90', ['Country','Firm'], 'Country and Firm',pos_90)
statistic(G)

In [ ]:
# 85
G = Graph_and_Visual(fin_df, 'source_rep_85', 'target_rep_85', ['Country','Firm'], 'Country and Firm',pos_85)
statistic(G)

In [ ]:
# 80
G = Graph_and_Visual(fin_df, 'source_rep_80', 'target_rep_80', ['Country','Firm'], 'Country and Firm',pos_80)
statistic(G)

### Country-Country

country 사이 thre(threshold) 별 관계그래프를 시각화합니다.

In [ ]:
G = Graph_and_Visual(fin_df, 'source', 'target', ['Country'], 'Country',pos)
statistic(G)

In [ ]:
# 95
G = Graph_and_Visual(fin_df, 'source_rep_95', 'target_rep_95', ['Country'], 'Country',pos_95)
statistic(G)

In [ ]:
# 90
G = Graph_and_Visual(fin_df, 'source_rep_90', 'target_rep_90', ['Country'], 'Country',pos_90)
statistic(G)

In [ ]:
# 85
G = Graph_and_Visual(fin_df, 'source_rep_85', 'target_rep_85', ['Country'], 'Country',pos_85)
statistic(G)

In [ ]:
# 80
G = Graph_and_Visual(fin_df, 'source_rep_80', 'target_rep_80', ['Country'], 'Country',pos_80)
statistic(G)

### Country-Resource

country와 resource 사이 thre(threshold) 별 관계그래프를 시각화합니다.

In [ ]:
G = Graph_and_Visual(fin_df, 'source', 'target', ['Country','Resource'], 'Country and Resource',pos)
statistic(G)

In [ ]:
# 95
G = Graph_and_Visual(fin_df, 'source_rep_95', 'target_rep_95', ['Country','Resource'], 'Country and Resource',pos_95)
statistic(G)

In [ ]:
# 90
G = Graph_and_Visual(fin_df, 'source_rep_90', 'target_rep_90', ['Country','Resource'], 'Country and Resource',pos_90)
statistic(G)

In [ ]:
# 85
G = Graph_and_Visual(fin_df, 'source_rep_85', 'target_rep_85', ['Country','Resource'], 'Country and Resource',pos_85)
statistic(G)

In [ ]:
# 80
G = Graph_and_Visual(fin_df, 'source_rep_80', 'target_rep_80', ['Country','Resource'], 'Country and Resource',pos_80)
statistic(G)

### Country-Technology

country와 technology 사이 thre(threshold) 별 관계그래프를 시각화합니다.

In [ ]:
G = Graph_and_Visual(fin_df, 'source', 'target', ['Country','Technology'], 'Country and Technology',pos)
statistic(G)

In [ ]:
# 95
G = Graph_and_Visual(fin_df, 'source_rep_95', 'target_rep_95', ['Country','Technology'], 'Country and Technology',pos_95)
statistic(G)

In [ ]:
# 90
G = Graph_and_Visual(fin_df, 'source_rep_90', 'target_rep_90', ['Country','Technology'], 'Country and Technology',pos_90)
statistic(G)

In [ ]:
# 85
G = Graph_and_Visual(fin_df, 'source_rep_85', 'target_rep_85', ['Country','Technology'], 'Country and Technology',pos_85)
statistic(G)

In [ ]:
# 80
G = Graph_and_Visual(fin_df, 'source_rep_80', 'target_rep_80', ['Country','Technology'], 'Country and Technology',pos_80)
statistic(G)

### Firm-Firm

firm 사이 thre(threshold) 별 관계그래프를 시각화합니다.

In [ ]:
G = Graph_and_Visual(fin_df, 'source', 'target', ['Firm'], 'Firm',pos)
statistic(G)

In [ ]:
# 95
G = Graph_and_Visual(fin_df, 'source_rep_95', 'target_rep_95', ['Firm'], 'Firm',pos_95)
statistic(G)

In [ ]:
# 90
G = Graph_and_Visual(fin_df, 'source_rep_90', 'target_rep_90', ['Firm'], 'Firm',pos_90)
statistic(G)

In [ ]:
# 85
G = Graph_and_Visual(fin_df, 'source_rep_85', 'target_rep_85', ['Firm'], 'Firm',pos_85)
statistic(G)

In [ ]:
# 80
G = Graph_and_Visual(fin_df, 'source_rep_80', 'target_rep_80', ['Firm'], 'Firm',pos_80)
statistic(G)

### Firm-Resource

firm과 resource 사이 thre(threshold) 별 관계그래프를 시각화합니다.

In [ ]:
G = Graph_and_Visual(fin_df, 'source', 'target', ['Firm','Resource'], 'Firm and Resource',pos)
statistic(G)

In [ ]:
# 95
G = Graph_and_Visual(fin_df, 'source_rep_95', 'target_rep_95', ['Resource','Firm'], 'Firm and Resource',pos_95)
statistic(G)

In [ ]:
# 90
G = Graph_and_Visual(fin_df, 'source_rep_90', 'target_rep_90', ['Resource','Firm'], 'Firm and Resource',pos_90)
statistic(G)

In [ ]:
# 85
G = Graph_and_Visual(fin_df, 'source_rep_85', 'target_rep_85', ['Resource','Firm'], 'Firm and Resource',pos_85)
statistic(G)

In [ ]:
# 80
G = Graph_and_Visual(fin_df, 'source_rep_80', 'target_rep_80', ['Resource','Firm'], 'Firm and Resource',pos_80)
statistic(G)

### Firm-Technology

firm과 technology 사이 thre(threshold) 별 관계그래프를 시각화합니다.

In [ ]:
G = Graph_and_Visual(fin_df, 'source', 'target', ['Firm','Technology'], 'Firm and Technology',pos)
statistic(G)

In [ ]:
# 95
G = Graph_and_Visual(fin_df, 'source_rep_95', 'target_rep_95', ['Technology','Firm'], 'Firm and Technology',pos_95)
statistic(G)

In [ ]:
# 90
G = Graph_and_Visual(fin_df, 'source_rep_90', 'target_rep_90', ['Technology','Firm'], 'Firm and Technology',pos_90)
statistic(G)

In [ ]:
# 85
G = Graph_and_Visual(fin_df, 'source_rep_85', 'target_rep_85', ['Technology','Firm'], 'Firm and Technology',pos_85)
statistic(G)

In [ ]:
# 80
G = Graph_and_Visual(fin_df, 'source_rep_80', 'target_rep_80', ['Technology','Firm'], 'Firm and Technology',pos_80)
statistic(G)

### Resource-Resource

In [ ]:
#추가했음(방법2)
pos = nx.spring_layout(gpt_G)
pos_95 = nx.spring_layout(gpt_G_95)
pos_90 = nx.spring_layout(gpt_G_90)
pos_85 = nx.spring_layout(gpt_G_85)
pos_80 = nx.spring_layout(gpt_G_80)

resouce 사이 thre(threshold) 별 관계그래프를 시각화합니다.

In [ ]:
G = Graph_and_Visual(fin_df2, 'source', 'target', ['Resource'], 'Resource',pos)
statistic(G)

In [ ]:
# 95
G = Graph_and_Visual(fin_df2, 'source_rep_95', 'target_rep_95', ['Resource'], 'Resource',pos_95)
statistic(G)

In [ ]:
# 90
G = Graph_and_Visual(fin_df2, 'source_rep_90', 'target_rep_90', ['Resource'], 'Resource',pos_90)
statistic(G)

In [ ]:
# 85
G = Graph_and_Visual(fin_df2, 'source_rep_85', 'target_rep_85', ['Resource'], 'Resource',pos_85)
statistic(G)

In [ ]:
# 80
G = Graph_and_Visual(fin_df2, 'source_rep_80', 'target_rep_80', ['Resource'], 'Resource',pos_80)
statistic(G)

### Resource-Technology

resouce와 technology 사이 thre(threshold) 별 관계그래프를 시각화합니다.

In [ ]:
G = Graph_and_Visual(fin_df2, 'source', 'target', ['Resource','Technology'], 'Resource and Technology',pos)
statistic(G)

In [ ]:
# 95
G = Graph_and_Visual(fin_df2, 'source_rep_95', 'target_rep_95', ['Resource','Technology'], 'Resource and Technology',pos_95)
statistic(G)

In [ ]:
# 90
G = Graph_and_Visual(fin_df2, 'source_rep_90', 'target_rep_90', ['Resource','Technology'], 'Resource and Technology',pos_90)
statistic(G)

In [ ]:
# 85
G = Graph_and_Visual(fin_df2, 'source_rep_85', 'target_rep_85', ['Resource','Technology'], 'Resource and Technology',pos_85)
statistic(G)

In [ ]:
# 80
G = Graph_and_Visual(fin_df2, 'source_rep_80', 'target_rep_80', ['Resource','Technology'], 'Resource and Technology',pos_80)
statistic(G)

### Technology-Technology

technology 사이 thre(threshold) 별 관계그래프를 시각화합니다.

In [ ]:
G = Graph_and_Visual(fin_df2, 'source', 'target', ['Technology'], 'Technology',pos) #수정
statistic(G)

In [ ]:
# 95
G = Graph_and_Visual(fin_df2, 'source_rep_95', 'target_rep_95', ['Technology'], 'Technology',pos_95)
statistic(G)

In [ ]:
# 90
G = Graph_and_Visual(fin_df2, 'source_rep_90', 'target_rep_90', ['Technology'], 'Technology',pos_90)
statistic(G)

In [ ]:
# 85
G = Graph_and_Visual(fin_df2, 'source_rep_85', 'target_rep_85', ['Technology'], 'Technology',pos_85) #추가
statistic(G)

In [ ]:
# 80
G = Graph_and_Visual(fin_df2, 'source_rep_80', 'target_rep_80', ['Technology'], 'Technology',pos_80) #추가
statistic(G)

Entire_Graph_and_Visual: 모든 요소 별 방향성이 있는 그래프를 생성하고 그리는 함수를 정의합니다.

In [ ]:
def Entire_Graph_and_Visual(fin_df, source, target, Title, pos):
    # 특정 열만 필터링 할 필요X -> 전체 열을 고려하는 그래프를 생성하기 때문문
    
    # 필터링된 데이터프레임을 기반으로 그래프 다시 생성
    filtered_G = nx.from_pandas_edgelist(fin_df, source=source, target=target, edge_attr='edge', create_using=nx.DiGraph())
    
    # 색상 매핑 (이전에 정의된 'Country'와 'Firm'에 대한 색상만 사용)
    color_map_filtered = {'Country': 'royalblue', 'Firm': 'lightgreen','Resource': 'orange','Technology': 'darkred', 'Unknown':'gray'}

    # 필터링된 그래프의 노드에 NER 태그 속성 추가
    for _, row in fin_df.iterrows():
        if row[source] in filtered_G.nodes:
            filtered_G.nodes[row[source]]['ner'] = row['source_ner']
        if row[target] in filtered_G.nodes:
            filtered_G.nodes[row[target]]['ner'] = row['target_ner']

    # 노드 색상 할당 (필터링된 그래프에 대해)
    filtered_node_colors = [color_map_filtered[filtered_G.nodes[node]['ner']] for node in filtered_G.nodes]

    # 그래프 시각화
    plt.figure(figsize=(60,60))
    nx.draw(filtered_G, with_labels=True, node_color=filtered_node_colors, edge_color='gray', pos=pos, font_size=3, node_size=100)
    plt.title("Supply Chain Graph ({})".format(Title), fontdict={'fontsize': 100})
    plt.legend(['node 1', 'node 2', 'edge'], loc='upper right')
    plt.show()


    plt.savefig('network_{}.png'.format(Title))
    return filtered_G

Entire_Graph_and_Visual을 이용하여 thre(threshold) 별로 전체 그래프를 그립니다.

In [ ]:
G_raw = nx.from_pandas_edgelist(fin_df, source='source', target='target', edge_attr='edge', create_using=nx.DiGraph())
pos = nx.spring_layout(G_raw)

In [ ]:
G = Entire_Graph_and_Visual(fin_df, 'source', 'target', 'Raw', pos)
statistic(G)

In [ ]:
G_95 = nx.from_pandas_edgelist(fin_df, source='source_rep_95', target='target_rep_95', edge_attr='edge', create_using=nx.DiGraph()) #수정
pos_95 = nx.spring_layout(G_95)

In [ ]:
G = Entire_Graph_and_Visual(fin_df, 'source_rep_95', 'target_rep_95', '95%', pos_95) #수정
statistic(G)

In [ ]:
G_90 = nx.from_pandas_edgelist(fin_df, source='source_rep_90', target='target_rep_90', edge_attr='edge', create_using=nx.DiGraph()) #수정
pos_90 = nx.spring_layout(G_90)

In [ ]:
G = Entire_Graph_and_Visual(fin_df, 'source_rep_90', 'target_rep_90', '90%', pos_90) #수정
statistic(G)

In [ ]:
G_85 = nx.from_pandas_edgelist(fin_df, source='source_rep_85', target='target_rep_85', edge_attr='edge', create_using=nx.DiGraph()) #수정
pos_85 = nx.spring_layout(G_85)

In [ ]:
G = Entire_Graph_and_Visual(fin_df, 'source_rep_85', 'target_rep_85', '85%', pos_85) #수정
statistic(G)

# 5. Finding the Shortest Path

pickle 파일을 통해 각각의 그래프를 로드합니다. 각 그래프의 타겟으로 하는 노드마다의 도달가능한 노드와 최단경로를 계산하고 시각화합니다. 이후 소스노드와 타겟노드간의 네트워크 분석과 네트워크 시뮬레이션을 수행합니다. 시뮬레이션은 노드간의 새로운 엣지를 추가하여 새로운 최단경로를 계산하는 과정입니다. 마지막으로 네트워크 내에서 중요한 역할을 하는 노드(영향력있는 노드) 식별합니다. 영향력 있는 노드는 도달성 분석을 통해 특정 노드에서 다른 노드들에게 얼마나 많은 영향을 미칠 수 있는지 확인할 수 있습니다.

SSAN_SP_DF와 GPT_SP_DF2의 교집합을 벤다이어그램을 통해 시각화합니다.

In [ ]:
setA = set(SSAN_SP_DF['source'].to_list()+SSAN_SP_DF['target'].to_list())
setB = set(GPT_SP_DF2['source'].to_list()+GPT_SP_DF2['target'].to_list())

# 벤 다이어그램 그리기
venn2([setA, setB], ('SSAN', 'RE ChatGPT'))

# 그래프 표시
plt.show()

 'SSAN_SP_DF'와 'GPT_SP_DF2'가 각각 'source' 및 'target' 열을 가진 데이터프레임입니다.<br>각 데이터프레임에서 'source'와 'target'의 쌍을 기반으로 벤 다이어그램을 생성합니다.

In [ ]:
setA = set(list(zip(SSAN_SP_DF['source'], SSAN_SP_DF['target'])))
setB = set(list(zip(GPT_SP_DF2['source'], GPT_SP_DF2['target'])))

# 벤 다이어그램 그리기
venn2([setA, setB], ('SSAN', 'RE ChatGPT'))

# 그래프 표시
plt.show()

 'SSAN_SP_DF' 및 'GPT_SP_DF2'의 'source', 'target', 'edge' 열을 조합하여 튜플을 만든 후 이를 기반으로 집합을 생성합니다. <br> 이후 이러한 집합 간의 교집합을 시각화하기 위해 벤 다이어그램을 생성합니다.

In [ ]:
setA = set(list(zip(SSAN_SP_DF['source'], SSAN_SP_DF['target'], SSAN_SP_DF['edge'])))
setB = set(list(zip(GPT_SP_DF2['source'], GPT_SP_DF2['target'], GPT_SP_DF2['edge'])))

# 벤 다이어그램 그리기
venn2([setA, setB], ('SSAN', 'RE ChatGPT'))

# 그래프 표시
plt.show()

SP_DF를 기반으로 thre(threshold) 기반으로 필터링된 데이터프레임(raw_df, norm_95_df, norm_90_df, norm_85_df, norm_80_df)을 생성합니다.

In [ ]:
#thre 별로 source 열과 target 열이 다르면 source와 target 열 rename, 'source_ner'과 'target_ner'이 'unknown'이 아닌 것 중에 중복인 행 제거
raw_df = SP_DF[SP_DF['source'] != SP_DF['target']][['source','target','edge','company','source_ner','target_ner']][(SP_DF['source_ner']!='Unknown')&(SP_DF['target_ner']!='Unknown')].drop_duplicates(subset=['source', 'target'], keep='first').reset_index(drop=True)
norm_95_df = SP_DF[SP_DF['source_normalized_95'] != SP_DF['target_normalized_95']][['source_normalized_95','target_normalized_95','edge','company','source_ner','target_ner']].rename(columns={'source_normalized_95': 'source','target_normalized_95': 'target'})[(SP_DF['source_ner']!='Unknown')&(SP_DF['target_ner']!='Unknown')].drop_duplicates(subset=['source', 'target'], keep='first').reset_index(drop=True)
norm_90_df = SP_DF[SP_DF['source_normalized_90'] != SP_DF['target_normalized_90']][['source_normalized_90','target_normalized_90','edge','company','source_ner','target_ner']].rename(columns={'source_normalized_90': 'source','target_normalized_90': 'target'})[(SP_DF['source_ner']!='Unknown')&(SP_DF['target_ner']!='Unknown')].drop_duplicates(subset=['source', 'target'], keep='first').reset_index(drop=True)
norm_85_df = SP_DF[SP_DF['source_normalized_85'] != SP_DF['target_normalized_85']][['source_normalized_85','target_normalized_85','edge','company','source_ner','target_ner']].rename(columns={'source_normalized_85': 'source','target_normalized_85': 'target'})[(SP_DF['source_ner']!='Unknown')&(SP_DF['target_ner']!='Unknown')].drop_duplicates(subset=['source', 'target'], keep='first').reset_index(drop=True)
norm_80_df = SP_DF[SP_DF['source_normalized_80'] != SP_DF['target_normalized_80']][['source_normalized_80','target_normalized_80','edge','company','source_ner','target_ner']].rename(columns={'source_normalized_80': 'source','target_normalized_80': 'target'})[(SP_DF['source_ner']!='Unknown')&(SP_DF['target_ner']!='Unknown')].drop_duplicates(subset=['source', 'target'], keep='first').reset_index(drop=True)

처리한 데이터를 thre(threshold) 별로 csv파일로 변환합니다.

In [ ]:
raw_df.to_csv('ALL_df.csv',index = False)
norm_95_df.to_csv('ALL_df_95.csv',index = False)
norm_90_df.to_csv('ALL_df_90.csv',index = False)
norm_85_df.to_csv('ALL_df_85.csv',index = False)
norm_80_df.to_csv('ALL_df_80.csv',index = False)

thre(threshold) 별로 필터링한 데이터를 dc_check 함수를 통해 그래프 생성하고 degree_centrality를 출력합니다.

In [ ]:
G, dc_df = dc_check(raw_df, 'edge')

G_95, dc_df_95= dc_check(norm_95_df, 'edge')

G_90, dc_df_90= dc_check(norm_90_df, 'edge')

G_85, dc_df_85= dc_check(norm_85_df, 'edge')

G_80, dc_df_80= dc_check(norm_80_df, 'edge')

dc_check return 값 중 하나인 dc_df를 csv 파일로 extract합니다.

In [ ]:
#dc_df.to_csv('raw_dc',index = False)

statistic1 함수를 이용하여 node_count: node수 측정, edge_count: edge 수 측정, density: 밀도 측정, connected_components: 그래프에서 약한 구성요소의 수를 측정하여 각각을 출력하고 반환합니다.

In [ ]:
statistic1(G)
statistic1(G_95)
statistic1(G_90)
statistic1(G_85)

dc_df_85에서 필요한 열만 추출합니다. (확인용)

In [ ]:
#'Category' 열이 'Resource'인 dc_df_85을 필터링
# 'Node' 및 'Centrality' 열을 선택하고 데이터프레임을 표시할 때 인덱스를 숨기는 스타일을 적용
try:
    dc_df_hide = dc_df_85[dc_df_85['Category']=='Resource'][['Node','Centrality']].style.hide_index()
except:
    dc_df_hide = dc_df_85[dc_df_85['Category']=='Resource'][['Node','Centrality']].style.hide(axis='index')

dc_df_hide

pickle 파일을 import합니다.

In [ ]:
# Pickle 파일로부터 그래프 로드
with open('graph.pickle', 'rb') as f:
    G = pickle.load(f)

with open('graph_95.pickle', 'rb') as f:
    G_95 = pickle.load(f)

with open('graph_90.pickle', 'rb') as f:
    G_90 = pickle.load(f)

with open('graph_85.pickle', 'rb') as f:
    G_85 = pickle.load(f)

with open('graph_80.pickle', 'rb') as f:
    G_80 = pickle.load(f)


norm_80_df에서 필요한 열만 select합니다.(확인용)

In [ ]:
norm_80_df[(norm_80_df['source']=='January 15 and July 15 of each year')|(norm_80_df['target']=='January 15 and July 15 of each year')]
norm_80_df[(norm_80_df['source']=='2,821')|(norm_80_df['target']=='2,821')]

그래프 G에서 특정 소스 노드로부터 도달 가능한 노드 및 최단 경로를 찾는 코드입니다.<br>
 (*전체 데이터를 출력하시려면 밑에 주석 처리된 코드를 실행하시면 됩니다.)

In [ ]:
source_node = 'Tesla, Inc.'
# 특정 노드에서 도달할 수 있는 모든 노드 찾기
reachable_nodes = nx.descendants(G, source_node)

# 특정 노드에서 각 노드까지의 최단 경로 찾기
shortest_paths = {node: nx.shortest_path(G, source_node, node) for node in reachable_nodes}

# 결과 출력
#print("Reachable Nodes:", reachable_nodes)
#print("Shortest Paths:", shortest_paths)   
print("Reachable Nodes:", list(reachable_nodes)[:10])  # 처음 10개 노드만 출력
print("Shortest Paths:", {node: shortest_paths[node] for node in list(reachable_nodes)[:10]})  # 처음 10개 노드의 최단 경로만 출력

In [ ]:
'''실제 코드 입니다.
source_node = 'Tesla, Inc.'
# 특정 노드에서 도달할 수 있는 모든 노드 찾기
reachable_nodes = nx.descendants(G, source_node)

# 특정 노드에서 각 노드까지의 최단 경로 찾기
shortest_paths = {node: nx.shortest_path(G, source_node, node) for node in reachable_nodes}

# 결과 출력
print("Reachable Nodes:", reachable_nodes)
print("Shortest Paths:", shortest_paths)
'''

find_path: 그래프 G와 가능한 소스노드의 목록을 받아 각 소스에 대해 실제 소스노드를 찾은 다음 해당 소스에서 child까지의 최단 경로를 계산하는 함수입니다.
<br>결과는 shortest_paths라는 딕셔너리에 저장되며 함수에 의해 반환됩니다.

In [ ]:
def find_path(G, possible_sources):
    shortest_paths = {}
    # 그래프 노드 중에서 실제 존재하는 source_node 찾기
    for source in possible_sources:
        actual_source = next((node for node in G.nodes() if node == source), None)
        if actual_source:
            reachable_nodes = nx.descendants(G, actual_source)
            tmp_shortest_paths = {node: nx.shortest_path(G, actual_source, node) for node in reachable_nodes}
            
            print("Exist", actual_source)
            shortest_paths = {**shortest_paths, **tmp_shortest_paths}
        # print(source)
        
    return shortest_paths
        

material_li: 다양한 소재의 이름을 포함하는 리스트를 정의합니다.
<br> materials는 material_li를 이용하여 각 소재 이름을 소문자로 변환하여 새로운 리스트 생성합니다.

In [ ]:
material_li = ['Lead', 'Zinc Oxide', 'Stoneware', 'Diabase', 'Bone China', 'Sulfuric Acid', 'Bauxite Alumina', 'Diesel', 
               'Diopside', 'Benzene', 'Selenium', 'Boric Acid', 'Pearl', 'Peat', 'Onyx', 'Perlite', 'Fiberglass Resin', 
               'Iceland Spar', 'Fluorine', 'Spinel', 'Thorianite', 'Cement', 'Turpentine', 'Apatite', 'Petalite', 'Steatite', 
               'Synthetic Rubber', 'Beryllium', 'Coal Tar', 'Slate', 'Marcasite', 'Gelatin', 'Expanded Polystyrene', 'Glycerol',
                'Beryllium Copper', 'Spinel Gemstone', 'Rubber', 'Vanadinite', 'Vivianite', 'Chloroform', 'Pyrex', 'Pyrolusite',
                'Baddeleyite', 'Fullerene', 'Iron Pyrites', 'Peridot', 'Bromargyrite', 'Potassium Hydroxide', 'Limonite', 'Zinc',
                'Niobium', 'Osmium', 'Pumice', 'Scolecite', 'Taconite', 'Siderite', 'Liquid Nitrogen', 'Galena Lead', 'Sodium Chloride',
                'Vanadium', 'Silica Gel', 'Isopropanol', 'Lacquer', 'Aluminium Oxide', 'Cuprite', 'Dolomitic Limestone', 'Bornite', 'Biofuels', 
                'Cassiterite', 'Biotite Mica', 'Cobaltite', 'Tremolite', 'Mendelevium', 'Thorium', 'Fermium', 'Phlogopite Mica', 'Halite', 
                'Jasper', 'Roentgenium', 'Flerovium', 'Propane', 'Pyrolusite Manganese', 'Iolite', 'Dysprosium', 'Nephrite', 'Quartz', 'Neodymium', 
                'Nanomaterials', 'Praseodymium', 'Granite', 'Chromite', 'Vermiculite', 'Hydrogen', 'Hematite', 'PVA (Polyvinyl Alcohol)', 'Aquamarine', 
                'Thallium', 'Darmstadtium', 'Flint', 'Ceramics', 'Cryolite Aluminum', 'Corundum', 'Rosin', 'Actinium', 'Leucite', 'Fluorapatite', 
                'Muscovite', 'Magnetite Iron', 'Meitnerium', 'Carbon Black', 'Hydrogen Peroxide', 'Iron', 'Tungstate', 'Cryogenite', 
                'Oganesson', 'Oxygen', 'Cryolite', 'Citrine', 'Polybutadiene', 'Sodium Hydroxide', 'Chlorine', 'Basalt', 'Amber', 'Mica Schist', 
                'Quartzite', 'Siderite Iron', 'Chromite Ore', 'Soap', 'Brick', 'Silimanite', 'Methyl Isobutyl Ketone', 'Magnesia', 'Talc Mineral', 
                'Sulfur', 'Polyester', 'Plaster', 'Sand', 'Lapis Lazuli', 'Dye', 'Aragonite', 'Methanol', 'Magnetite Sand', 'Hydrochloric Acid', 
                'Chrysocolla', 'Calcite', 'Dubnium', 'Hydraulic Fluid', 'Periclase', 'Cerium', 'Phlogopite', 'Paraffin Wax', 'Trona', 'Epidote', 
                'Azurite', 'Livermorium', 'Diatomaceous Earth', 'Jet', 'Nitric Acid', 'Xenotime', 'Gold', 'Titanite', 'Graphite', 'Monazite', 
                'Sphalerite Zinc', 'Celestite', 'Cerussite', 'Uranium', 'Californium', 'Vesuvianite', 'Isobutane', 'Boron Nitride', 'Ruby', 
                'Helium', 'Thorite', 'Fatty Acids', 'Iron Slag', 'Bakelite', 'Topaz Gemstone', 'Columbite', 'Lepidolite', 'Prehnite', 'Zircon', 
                'Emerald', 'Hassium', 'Rhodonite', 'Ink', 'Rubidium', 'Neon', 'Realgar', 'Gabbro', 'Lawrencium', 'Steel', 'Fiberglass', 'Butane', 
                'Polystyrene', 'Orthoclase', 'Marble', 'Carbon', 'Promethium', 'Earthenware', 'Tetrafluoroethylene', 'Quartz Crystal', 'Propylene', 
                'Plagioclase', 'Coking Coal', 'Cinnabar Mercury', 'Chalcopyrite', 'Rayon', 'Astatine', 'Spandex', 'Opal', 'Bismaleimide', 'Acrylic', 
                'Sapphire', 'Moonstone Gem', 'Asphalt', 'Carbon Fiber', 'Triethanolamine', 'Glauberite', 'Coal Slag', 'Plutonium', 'Gypsum Alabaster', 
                'Almandine', 'Gallium', 'Moonstone', 'Spessartine', 'Pyrophyllite', 'Diamond', 'Quartz Sand', 'Bohrium', 'Turquoise', 'Copernicium', 
                'Technetium', 'Bromine', 'Wax', 'Nylon', 'Formaldehyde', 'Silver', 'Magnesium Oxide', 'Platinum', 'Polyurethane Foam', 'Fuel Oil', 
                'Toluene', 'Ethylene Glycol', 'Copper Sulfate', 'Limestone', 'Labradorite', 'Salt', 'Glycerine', 'Methanol Fuel', 'Mica', 'Naphtha', 
                'Bitumen', 'Iodine', 'Sulfur Crystal', 'Magnesite', 'Curium', 'Concrete', 'Radon', 'Petroleum', 'Antifreeze', 'Nickel', 'Greenockite', 
                'Scandium', 'Magnesite Magnesium', 'Uraninite', 'Sillimanite', 'Transformer Oil', 'Gypsum', 'Tanzanite Gem', 'Ytterbium', 'Goethite', 'Talcum', 
                'Fluorspar', 'Liquid Oxygen', 'Solder', 'Clay', 'Wollastonite', 'Terbium', 'Halloysite', 'Zeolite', 'Rhyolite', 'Ammonia', 'Cadmium', 'Jade', 
                'Molybdenite', 'Carbon Dioxide', 'Methyl Ethyl Ketone', 'Zirconium Dioxide', 'Fluorite', 'Cuprite Copper', 'Polyisoprene', 'Caprolactam', 
                'Soybean Oil', 'Feldspathoids', 'Germanium', 'Xenon', 'Malachite', 'Amino Resins', 'Tellurium', 'Smithsonite', 'Protactinium', 'Sphalerite', 
                'Iridium', 'Chalcocite', 'Ethylene', 'Serpentine', 'Tantalite', 'Barite', 'Phosphoric Acid', 'Orthoclase Feldspar', 'Celestine', 'Scheelite', 
                'Zirconium', 'Gadolinium', 'Alabaster', 'Limonite Iron', 'Vinyl Acetate', 'Styrene', 'Alumina', 'Rhodochrosite', 'Paint', 'Feldspar', 'Europium', 
                'Zinc Blende', 'Bentonite', 'Aniline', 'Polyethylene', 'Compressed Air', 'Nobelium', 'Kyanite', 'Titanium', 'Rutile Titanium', 'Palladium', 'Galena', 
                'Obsidian', 'Pisolite', 'Boron', 'Psilomelane', 'Neoprene', 'Sodium Carbonate', 'Riebeckite', 'Yttrium Oxide', 'Olivine', 'Hornblende', 'Sylvite', 
                'Malachite Green', 'Glucose', 'Kaolinite', 'Phenol', 'Topaz', 'Talc', 'Obsidian Glass', 'Tin', 'Borax', 'Cyclohexane', 'Glass Wool', 
                'Amethyst', 'Yttrium', 'Ferrite', 'Cryogenic Liquids', 'Magnetite', 'Nitrogen', 'Kerosene', 'Einsteinium', 'Kevlar', 'Aluminum', 
                'Cellulose', 'Cinnabar', 'Seaborgium', 'Sodium Silicate', 'Silicon', 'Staurolite', 'Strontianite', 'Sodalite', 'Bismuth', 'Erbium', 
                'Hematine', 'Opal Gemstone', 'Enamel', 'Phenacite', 'Arsenic', 'Natural Gas', 'Plasticizers', 'Holmium', 'Samarium', 'Ethane', 'Lithium', 
                'PVC', 'Bauxite', 'Polypropylene', 'Tungsten', 'Hydroboracite', 'Acetone', 'Vinyl Chloride', 'Detergent', 'Zincite', 'Montmorillonite', 
                'Silicone', 'Garnierite', 'Hauyne', 'Ethanol', 'Tennessine', 'Lactic Acid', 'Pitchblende', 'Andalusite', 'Oleum', 'Glass Fiber', 'Glass', 
                'Phosphorus', 'Furfural', 'Varnish', 'Cobalt', 'Lanthanum', 'Unakite', 'Nepheline', 'Staurolite Gem', 'Stibnite', 'Garnet', 'Urea', 
                'Porcelain', 'Octane', 'Neptunium', 'Microcline', 'Sorbitol', 'Mineral Spirits', 'Lutetium', 'Biotite', 'Monazite Rare Earths', 
                'Nihonium', 'Muscovite Mica', 'Thulium', 'Nitrogen Gas', 'Moscovium', 'Blue John', 'Solvents', 'Calcium Oxide', 'Chalk', 'Silica', 
                'Olivine Peridot', 'Orpiment', 'PVC Resin', 'Chromium', 'Schorl', 'Travertine', 'Dolostone', 'Coal', 'Wulfenite', 'Xylene', 'Hexane', 
                'Americium', 'Idocrase', 'Berkelium', 'Argon', 'Hafnium', 'Krypton', 'Pyrite', 'Strontium', 'Clinochlore', 'Detergent Chemicals', 'Methane', 
                'Manganese', 'Azomite', 'Ruthenium', 'Melamine', 'Agate', 'Molybdenum', 'Indium', 'Soapstone', 'Lazurite', 'Shale', 'Jet Fuel', 'Ceramic Fiber', 
                'Diatomite', 'Aspartame', 'Copper', 'Ulexite', 'Rutherfordium', 'Calcium Carbonate', 'Chert', 'Antimony', 'Gravel', 'Tourmaline', 'Ilmenite', 
                'Asbestos', 'Radium', 'Rhodium', 'Bioplastics', 'Mercury', 'Barium', 'Sodalite Group', 'Resorcinol', 'Dolomite', 'Milky Quartz', 
                'Terephthalic Acid', 'White Spirit', 'White Gold', 'Synthetic Sapphire', 'Polycarbonate','Silicone Rubber']
materials = [x.lower() for x in material_li]

find_path 함수를 사용하여 그래프에서 특정 출처로부터의 가장 짧은 경로를 찾고, 그 경로들을 필터링하여 특정 길이의 경로에 대한 정보를 출력합니다.

In [ ]:
possible_sources = ['Tesla, Inc.', 'Tesla','TSLA']

#find_path 함수를 사용하여 각 출처에서의 가장 짧은 경로를 찾고, 결과를 shortest_paths 변수에 저장
shortest_paths =find_path(G, possible_sources)
# shortest_paths에서 각 재료가 1홉으로 이어진 경로만 필터링, filtered_paths: shortest_paths에서 특정 길이의 경로를 필터링
# 먼저, 길이가 2인 경로를 필터링하여 path[1]에 해당하는 노드가 materials에 속하는지 확인
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if len(path) == 2 and any(material == path[1].lower() for material in materials)
}

print(filtered_paths)

#길이가 3인 경로를 필터링하여 path[2]에 해당하는 노드가 materials에 속하는지 확인
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if len(path) == 3 and any(material == path[2].lower() for material in materials)
}

print(filtered_paths)

In [ ]:
possible_sources = ['Dow Inc.', 'Dow Jones Indices', 'The Dow Chemical Dow Inc.', 'Siam Polyethylene Dow Inc. Limited', 'Sadara Chemical Dow Inc.','The Kuwait Styrene Dow Inc.', 'Saudi Arabian Oil Dow Inc.', 'Dow Inc.' ]
shortest_paths =find_path(G, possible_sources)
# shortest_paths에서 각 재료가 1홉으로 이어진 경로만 필터링
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if len(path) == 2 and any(material == path[1].lower() for material in materials)
    
}

print(filtered_paths)

데이터프레임에서 특정 조건을 만족하는 'source'와 'target' 열의 고유한 값을 추출합니다.(확인용)

In [ ]:
# 'source' 열에서 'dow'를 포함하는 경우(대소문자 무시) 해당되는 행들을 선택하고, 이들 중에서 'source' 열의 고유한 값들을 추출
unique_values_source = raw_df[raw_df['source'].str.contains('dow', case=False)]['source'].unique()

# 'target' 열에서 'dow'를 포함하는 경우(대소문자 무시) 해당되는 행들을 선택하고, 이들 중에서 'target' 열의 고유한 값들을 추출
unique_values_target = raw_df[raw_df['target'].str.contains('dow', case=False)]['target'].unique()

# 결과 출력
print(set(list(unique_values_source) + list(unique_values_target)))

shortest_paths 딕셔너리에서 길이가 3이며 특정 조건을 만족하는 경로들을 필터링하고 그 결과를 출력합니다. (확인용)

In [ ]:
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if len(path) == 3 and any(material == path[2].lower() for material in materials)
}

print(filtered_paths)

그래프G에서 특정 출처로부터의 가장 짧은 경로를 찾고, 해당 경로를 필터링합니다.<br>
exclude_nodes 리스트에 포함된 노드는 제외하고 경로에 포함된 노드 중에 materials 리스트에 속하는 재료가 있는 경우만을 필터링합니다.<br>
이 과정을 source, thre(threshold) 별로 반복합니다.
<br>(*모든 출력값을 원하시면 주석처리된 코드를 실행하시면 됩니다.)

In [ ]:
def print_filtered_paths(filtered_paths):
    count = 0
    for key, path in filtered_paths.items():
        if count >= 10:  # 처음 10개 항목까지만 출력
            break
        print(key, ":", path)
        count += 1

possible_sources = ['Tesla, Inc.', 'Tesla','TSLA']
exclude_nodes = ['sec']

# G에 대한 filtered_paths 출력
shortest_paths = find_path(G, possible_sources)
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
filtered_paths = {
    key: path for key, path in filtered_paths.items()
    if not any(node.lower() in exclude_nodes for node in path)
}
print("Filtered Paths for G:")
print_filtered_paths(filtered_paths)

# G_95에 대한 filtered_paths 출력
shortest_paths = find_path(G_95, possible_sources)
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print("\nFiltered Paths for G_95:")
print_filtered_paths(filtered_paths)

# G_90에 대한 filtered_paths 출력
shortest_paths = find_path(G_90, possible_sources)
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print("\nFiltered Paths for G_90:")
print_filtered_paths(filtered_paths)

# G_85에 대한 filtered_paths 출력
shortest_paths = find_path(G_85, possible_sources)
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print("\nFiltered Paths for G_85:")
print_filtered_paths(filtered_paths)

# G_80에 대한 filtered_paths 출력
shortest_paths = find_path(G_80, possible_sources)
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print("\nFiltered Paths for G_80:")
print_filtered_paths(filtered_paths)


In [ ]:
'''
possible_sources = ['Tesla, Inc.', 'Tesla','TSLA']
# 제거하고자 하는 노드 목록
exclude_nodes = [ 'sec']


shortest_paths =find_path(G, possible_sources)
# 'lithium' 또는 'Lithium'이 포함된 경로만 추출
# shortest_paths에서 각 재료가 포함된 경로만 필터링
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
# filtered_paths에서 특정 노드를 포함하지 않는 경로만 필터링
filtered_paths = {
    key: path for key, path in filtered_paths.items()
    if not any(node.lower() in exclude_nodes for node in path)
}
print(filtered_paths)

shortest_paths = find_path(G_95, possible_sources)
# 'lithium' 또는 'Lithium'이 포함된 경로만 추출
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print(filtered_paths)

shortest_paths = find_path(G_90, possible_sources)
# 'lithium' 또는 'Lithium'이 포함된 경로만 추출
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print(filtered_paths)

shortest_paths =find_path(G_85, possible_sources)
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print(filtered_paths)

shortest_paths =find_path(G_80, possible_sources)
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print(filtered_paths)
'''

In [ ]:
def print_filtered_paths(filtered_paths):
    count = 0
    for key, path in filtered_paths.items():
        if count >= 10:  # 처음 10개 항목까지만 출력
            break
        print(key, ":", path)
        count += 1

possible_sources = ['Apple, Inc.','Apple', 'AAPL']

# G에 대한 filtered_paths 출력
shortest_paths = find_path(G, possible_sources)
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print("Filtered Paths for G:")
print_filtered_paths(filtered_paths)

# G_95에 대한 filtered_paths 출력
shortest_paths = find_path(G_95, possible_sources)
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print("\nFiltered Paths for G_95:")
print_filtered_paths(filtered_paths)

# G_90에 대한 filtered_paths 출력
shortest_paths = find_path(G_90, possible_sources)
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print("\nFiltered Paths for G_90:")
print_filtered_paths(filtered_paths)

# G_85에 대한 filtered_paths 출력
shortest_paths = find_path(G_85, possible_sources)
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print("\nFiltered Paths for G_85:")
print_filtered_paths(filtered_paths)

# G_80에 대한 filtered_paths 출력
shortest_paths = find_path(G_80, possible_sources)
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print("\nFiltered Paths for G_80:")
print_filtered_paths(filtered_paths)


In [ ]:
'''
possible_sources = ['Apple, Inc.','Apple', 'AAPL']


shortest_paths = find_path(G, possible_sources)
# 'lithium' 또는 'Lithium'이 포함된 경로만 추출
# shortest_paths에서 각 재료가 포함된 경로만 필터링
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print(filtered_paths)

shortest_paths = find_path(G_95, possible_sources)
# 'lithium' 또는 'Lithium'이 포함된 경로만 추출
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print(filtered_paths)

shortest_paths =find_path(G_90, possible_sources)
# 'lithium' 또는 'Lithium'이 포함된 경로만 추출
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print(filtered_paths)

shortest_paths =find_path(G_85, possible_sources)
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print(filtered_paths)

shortest_paths =find_path(G_80, possible_sources)
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print(filtered_paths)
'''

특정 출처(여기서는 'Apple, Inc.', 'Apple', 'AAPL')로부터의 가장 짧은 경로를 찾고, 해당 경로를 특정 조건에 따라 필터링하여 출력합니다.

In [ ]:
possible_sources = ['Apple, Inc.','Apple', 'AAPL']


shortest_paths =find_path(G, possible_sources)
# shortest_paths에서 각 재료가 1홉으로 이어진 경로만 필터링
#길이가 2인 경로를 필터링하고, 해당 경로의 두 번째 노드가 materials 리스트에 속하는지 확인하여 filtered_paths에 저장
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if len(path) == 2 and any(material == path[1].lower() for material in materials)
}

print(filtered_paths)

#길이가 3인 경로를 필터링하고, 해당 경로의 세 번째 노드가 materials 리스트에 속하는지 확인하여 filtered_paths에 저장
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if len(path) == 3 and any(material == path[2].lower() for material in materials)
}

print(filtered_paths)

#길이가 4인 경로를 필터링하고, 해당 경로의 네 번째 노드가 materials 리스트에 속하는지 확인하여 filtered_paths에 저장
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if len(path) == 4 and any(material == path[3].lower() for material in materials)
}

print(filtered_paths)

데이터프레임에서 'samsung'을 포함하는 'source' 또는 'target' 값을 고유하게 추출하는 작업을 수행합니다.

In [ ]:
#'source' 열에서 'samsung' 포함하는 고유값 추출
#'source' 열에서 'samsung'을 포함하는 행을 선택하고, 해당 행들 중에서 'source' 열의 고유한 값들을 추출
unique_values_source = raw_df[raw_df['source'].str.contains('samsung', case=False)]['source'].unique()

#'target' 열에서 'samsung' 포함하는 고유값 추출
#'target' 열에서 'samsung'을 포함하는 행을 선택하고, 해당 행들 중에서 'target' 열의 고유한 값들을 추출
unique_values_target = raw_df[raw_df['target'].str.contains('samsung', case=False)]['target'].unique()

# 결과 출력
#'source'와 'target'에서 추출한 고유한 값들을 리스트로 만들고, 이를 합쳐서 중복을 제거한 후 출력
print(list(unique_values_source) + list(unique_values_target))

'Samsung' 및 'Samsung Electronics Co., Ltd.'로부터의 가장 짧은 경로를 찾고, 해당 경로를 특정 길이에 따라 필터링하여 출력하는 작업을 수행합니다.

In [ ]:
possible_sources = ['Samsung', 'Samsung Electronics Co., Ltd.']


shortest_paths =find_path(G, possible_sources)
# shortest_paths에서 각 재료가 1홉으로 이어진 경로만 필터링
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if len(path) == 2 # and any(material == path[1].lower() for material in materials)
}

print(filtered_paths)

filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if len(path) == 3 # and any(material == path[2].lower() for material in materials)
}

print(filtered_paths)

filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if len(path) == 4 #and any(material == path[3].lower() for material in materials)
}

print(filtered_paths)

filtered_paths = {
    key: path for key, path in shortest_paths.items()
}

DataFrame에서 'source' 또는 'target' 열이 'Samsung'인 행을 필터링합니다.(확인용)

In [ ]:
raw_df[(raw_df['source']=='Samsung')|(raw_df['target']=='Samsung')]

thre(threshold) 별로 find_path 함수를 사용하여 'Samsung' 및 'Samsung Electronics Co., Ltd.' 출처로부터의 최단 경로를 찾고, 해당 경로를 재료에 따라 필터링하여 결과를 출력합니다.<br>
(*모든 출력값을 원하시면 밑에 주석 처리된 코드를 실행하시면 됩니다.)

In [ ]:
def print_filtered_paths(filtered_paths):
    count = 0
    for key, path in filtered_paths.items():
        if count >= 10:  # 처음 10개 항목까지만 출력
            break
        print(key, ":", path)
        count += 1

possible_sources = ['Samsung', 'Samsung Electronics Co., Ltd.']

# G에 대한 filtered_paths 출력
shortest_paths = find_path(G, possible_sources)
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print("Filtered Paths for G:")
print_filtered_paths(filtered_paths)

# G_95에 대한 filtered_paths 출력
shortest_paths = find_path(G_95, possible_sources)
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print("\nFiltered Paths for G_95:")
print_filtered_paths(filtered_paths)

# G_90에 대한 filtered_paths 출력
shortest_paths = find_path(G_90, possible_sources)
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print("\nFiltered Paths for G_90:")
print_filtered_paths(filtered_paths)

# G_85에 대한 filtered_paths 출력
shortest_paths = find_path(G_85, possible_sources)
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print("\nFiltered Paths for G_85:")
print_filtered_paths(filtered_paths)

# G_80에 대한 filtered_paths 출력
shortest_paths = find_path(G_80, possible_sources)
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print("\nFiltered Paths for G_80:")
print_filtered_paths(filtered_paths)


In [ ]:
'''
possible_sources = ['Samsung', 'Samsung Electronics Co., Ltd.']

shortest_paths=find_path(G, possible_sources) #최단경로 확보
# 'lithium' 또는 'Lithium'이 포함된 경로만 추출
# shortest_paths에서 각 재료가 포함된 경로만 필터링
# 각 경로에 대해 해당 경로의 노드 중에서 materials 리스트에 속하는 재료가 포함되어 있는지 확인
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print(filtered_paths)

shortest_paths=find_path(G_95, possible_sources)
# 'lithium' 또는 'Lithium'이 포함된 경로만 추출
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print(filtered_paths)

shortest_paths=find_path(G_90, possible_sources)
# 'lithium' 또는 'Lithium'이 포함된 경로만 추출
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print(filtered_paths)

shortest_paths=find_path(G_85, possible_sources)
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print(filtered_paths)

shortest_paths=find_path(G_80, possible_sources)
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print(filtered_paths)
'''

최단 경로들 중에서 경로의 길이가 4인 경우(노드 수가 3인 경우)에 해당하는 노드 및 경로들을 선택하여 딕셔너리로 저장합니다.

In [ ]:
paths_hop_3 = {node: path for node, path in shortest_paths.items() if len(path) - 1 == 3}
#len(path)는 경로의 길이를 나타내며, 경로의 길이에서 1을 뺀 값이 3이 되는 경우

최단 경로들 중에서 경로의 길이가 4이며 (노드 수가 3인 경우), 해당 경로의 마지막 노드가 materials 리스트에 속하는 재료인 경우에 해당하는 노드 및 경로들을 선택하여 딕셔너리로 저장합니다. 이후 각 노드와 해당 노드에 이르는 최단 경로를 filtered_paths 딕셔너리에 저장합니다. (확인용)

In [ ]:
#연습코드
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if len(path) == 4 #and any(material == path[3].lower() for material in materials)
}

방향성 그래프를 생성하고 시각화합니다.

In [ ]:
# 각 노드 유형에 대한 색상을 지정합니다., 각 노드와 색상을 매핑하는 사전 정의
node_color_map = {
    'Company': 'green',
    'Country': 'blue',
    'Resource': 'orange',
    'Technology': 'red'
}

# Extracting unique nodes and their types
#'norm_85_df'에서 'source' 및 'target' 열을 사용하여 두 개의 데이터프레임(sources 및 targets)을 생성 열 이름은 각각 'node' 및 'type'으로 변경
sources = norm_85_df[['source', 'source_ner']].rename(columns={'source': 'node', 'source_ner': 'type'}) 
targets = norm_85_df[['target', 'target_ner']].rename(columns={'target': 'node', 'target_ner': 'type'}) 
#'sources' 및 'targets' 데이터프레임을 연결하고 중복을 제거한 후 'node' 열을 인덱스로 설정
all_nodes = pd.concat([sources, targets]).drop_duplicates().set_index('node') 
# Mapping of nodes to types
#'all_nodes' DataFrame의 'type' 열을 사용하여 딕셔너리(node_types)를 생성
node_types = all_nodes['type'].to_dict() 

# Initialize a directed graph
H = nx.DiGraph() #방향성 그래프 생성, 초기화

# Add edges to the graph
#딕셔너리를 반복하며, 각 경로에 대해 그래프 H에 간선을 추가
for source, targets in filtered_paths.items():
    for i in range(len(targets) - 1):
        H.add_edge(targets[i], targets[i + 1])

# Set the layout for the graph
# pos = nx.spring_layout(H, k=0.5, iterations=20)  # Adjust 'k' for distance between nodes


# Set graph layout
pos = nx.kamada_kawai_layout(H)

# Draw nodes with colors based on their types
#각 노드의 유형에 기반하여 노드 색상 목록(node_colors)을 작성
node_colors = [node_color_map.get(node_types.get(node, 'gray'), 'gray') for node in H.nodes()]
nx.draw_networkx_nodes(H, pos, node_color=node_colors, node_size=50)

# Draw edges
nx.draw_networkx_edges(H, pos)

# Draw labels below nodes
#labels_pos = {node: (coords[0], coords[1] - 0.1) for node, coords in pos.items()}
nx.draw_networkx_labels(H, pos, font_size=8)

plt.axis('off')
plt.show()


데이터프레임에서 'source' 또는 'target' 열에 'Shell'이라는 문자열을 포함하는 고유한 값을 찾고, 해당 값을 출력하는 과정입니다.

In [ ]:
#'source' 열에 'Shell'을 포함하는 행을 필터링하고, 그 중에서 고유한 값 찾아냄
unique_values_source = raw_df[raw_df['source'].str.contains('Shell', case=False)]['source'].unique()

#'target' 열에서 'Shell' 포함하는 고유값 추출
unique_values_target = raw_df[raw_df['target'].str.contains('Shell', case=False)]['target'].unique()

# 결과 출력, 집합으로 변환하여 중복제거 후
print(set(list(unique_values_source) + list(unique_values_target)))

그래프 (G_85)에 가능한 소스 노드 목록에서 다른 노드로의 가장 짧은 경로를 찾은 다음 경로는 특정 조건과 일치하는 경우에만 필터링합니다.<br> (*실제 데이터를 원하시면 밑에 코드를 실행하시면 됩니다.)

In [ ]:
def print_filtered_paths(filtered_paths):
    count = 0
    for key, path in filtered_paths.items():
        if count >= 10:  # 처음 10개 항목까지만 출력
            break
        print(key, ":", path)
        count += 1

possible_sources = ['Royal Dutch Shell', 'Shell Downstream Inc.', 'Shell New Energies US', 'Shell']

# G_85에 대한 filtered_paths 출력
shortest_paths = find_path(G_85, possible_sources)
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)
}
print("Filtered Paths for G_85:")
print_filtered_paths(filtered_paths)

In [ ]:
'''
#possible_sources = ['Samsung', 'Samsung Electronics Co., Ltd.']

possible_sources = ['Royal Dutch Shell', 'Shell Downstream Inc.', 'Shell New Energies US', 'Shell']

#각 가능한 소스에서 그래프의 다른 노드로의 최단 경로를 찾기 위해 함수 (find_path)를 호출
shortest_paths = find_path(G_85, possible_sources)
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any(any(material == node.lower() for material in materials) for node in path)#node == 'Cisco' for node in path) #any(material == node.lower() for material in materials) for node in path)
}
print(filtered_paths)
'''

### Check whether a key path exists for the final network

plot_individual_shortest_path(): 최단 경로들을 시각화하는 함수를 정의합니다.

In [ ]:
# 예시 데이터: 실제 데이터는 paths_hop_3 변수에 저장되어 있다고 가정합니다.

# 각 경로를 별도의 figure로 시각화하기 위한 함수 정의
def plot_individual_shortest_paths(paths):
    fig_width = 30  # 가로 길이 설정
    
    for key, path in paths.items():
        # if key == 'the U.S.':
            # fig_height = 0.5  # 각 figure의 세로 길이는 고정
            fig_width = 20  # 가로 길이 설정
            fig_height = 0.7
            fig, ax = plt.subplots(figsize=(fig_width, fig_height))
            
            # 경로 길이에 따라 x 위치를 등간격으로 설정, 노드의 x위치 및 y위치 설정
            x_positions = np.linspace(0, fig_width, len(path))
            y_positions = np.zeros(len(path))  # y 위치는 0으로 고정
            
            # 노드 색상 결정
            node_colors = ['orange' if i in [0, len(path) - 1] else 'gray' for i in range(len(path))]
            
            # 노드와 경로 그리기
            ax.plot(x_positions, y_positions, '-o', color='black', markersize=10, markerfacecolor='gray')
            ax.scatter(x_positions[[0, -1]], y_positions[[0, -1]], color='orange', zorder=5)  # 첫 번째와 마지막 노드 강조
            
            # 노드 이름 텍스트로 표기
            for x, label in zip(x_positions, path):
                ax.text(x, -0.1, label, ha='center', va='top', fontsize=15)
            
            # 축 및 테두리 제거
            ax.axis('off')
            
            # 그래프 표시
            plt.show()

# 최단 경로 시각화 함수 호출
plot_individual_shortest_paths(filtered_paths)


plot_individual_shortest_paths(): 주어진 최단 경로들을 시각화하는 함수를 정의합니다. <br>위 함수 내용을 동일하지만 그래프 크기에 있어서 차이가 존재합니다.

In [ ]:
# 예시 데이터: 실제 데이터는 paths_hop_3 변수에 저장되어 있다고 가정합니다.

# 각 경로를 별도의 figure로 시각화하기 위한 함수 정의
def plot_individual_shortest_paths(paths):
    fig_width = 30  # 가로 길이 설정
    for key, path in paths.items():
        fig_height = 0.5  # 각 figure의 세로 길이는 고정
        fig, ax = plt.subplots(figsize=(fig_width, fig_height))
        
        # 경로 길이에 따라 x 위치를 등간격으로 설정
        x_positions = np.linspace(0, fig_width, len(path))
        y_positions = np.zeros(len(path))  # y 위치는 0으로 고정
        
        # 노드 색상 결정
        node_colors = ['orange' if i in [0, len(path) - 1] else 'gray' for i in range(len(path))]
        
        # 노드와 경로 그리기
        ax.plot(x_positions, y_positions, '-o', color='black', markersize=15, markerfacecolor='gray')
        ax.scatter(x_positions[[0, -1]], y_positions[[0, -1]], color='orange', zorder=5)  # 첫 번째와 마지막 노드 강조
        
        # 노드 이름 텍스트로 표기
        for x, label in zip(x_positions, path):
            ax.text(x, -0.1, label, ha='center', va='top', fontsize=8)
        
        # 축 및 테두리 제거
        ax.axis('off')
        
        # 그래프 표시
        plt.show()

# 최단 경로 시각화 함수 호출
plot_individual_shortest_paths(paths_hop_3)

plot_shortest_paths(): 주어진 여러 최단 경로들을 시각화하는 함수를 정의합니다.

In [ ]:
#수정
# 경로를 시각화하기 위한 함수 정의
def plot_shortest_paths(paths):
    # 시각화 설정
    fig_width = 30  # 가로 길이 설정
    fig_height = len(paths) * 0.5  # 세로 길이는 경로 수에 비례
    fig, ax = plt.subplots(figsize=(fig_width, fig_height))
    
    # 각 경로에 대해 반복하여 시각화
    for i, (key, path) in enumerate(paths.items()):
        # 경로 길이에 따라 x 위치를 균등하게 분할
        x_positions = np.arange(0, fig_width, fig_width / (len(path) - 1))
        y_positions = [i] * len(path)
        
        # 노드와 경로 그리기
        ax.plot(x_positions, y_positions[:-1], '-o', color='black', markersize=10)
        
        # 노드 이름 텍스트로 표기
        for x, label in zip(x_positions, path):
            ax.text(x, i, label, ha='center', va='center', fontsize=8)

    # 축 및 테두리 제거
    ax.axis('off')
    
    # 그래프 표시
    plt.tight_layout()
    plt.show()

# 최단 경로 시각화 함수 호출
plot_shortest_paths(paths_hop_3)

plot_timeline(): 여러 최단 경로들에 대한 타임라인을 그리는 함수를 정의합니다.

In [ ]:
# 각 경로를 시각화하기 위한 함수 정의
def plot_timeline(paths, ax, y_height, node_color, start_end_color):
    # 각 경로에 대한 타임라인 그리기
    for i, (node, path) in enumerate(paths.items()):
        # 노드의 x 위치 설정
        x_positions = range(len(path))
        y_positions = [y_height] * len(path)
        
        # 타임라인과 노드 그리기
        ax.plot(x_positions, y_positions, color='grey', linestyle='-', marker='o', markersize=10)
        ax.scatter([x_positions[0], x_positions[-1]], [y_height, y_height], color=start_end_color, zorder=5)
        
        # 노드 레이블 추가
        for x, label in zip(x_positions, path):
            color = node_color if x not in (0, len(path) - 1) else start_end_color
            ax.text(x, y_height, label, ha='center', va='center', bbox=dict(facecolor=color, edgecolor='none', boxstyle='round,pad=0.5'))
        
        # 다음 경로를 위해 y 위치 조정
        y_height -= 1

# 전체 플롯 설정
fig, ax = plt.subplots(figsize=(10, 3))
ax.set_ylim(-len(paths_hop_3 ), 1)  # y축의 범위 설정
ax.set_xlim(-1, max(len(path) for path in shortest_paths.values()))  # x축의 범위 설정

# 경로 타임라인 그리기
plot_timeline(paths_hop_3 , ax, 0, 'grey', 'orange')

# 그래프 스타일 설정
ax.axis('off')  # 축 제거
plt.show()

In [ ]:
# 수정된 경로 시각화 함수 정의
def plot_timeline2(paths, y_height, node_color, start_end_color):
    # 시각화의 가로 길이 설정
    fig_width = 20
    # 전체 플롯 설정
    fig, ax = plt.subplots(figsize=(fig_width, len(paths)))

    # 각 경로에 대한 타임라인 그리기
    for i, (node, path) in enumerate(paths.items()):
        # 노드의 x 위치 설정
        x_positions = np.linspace(0, fig_width, len(path))
        y_positions = [y_height] * len(path)
        
        # 타임라인과 노드 그리기
        ax.plot(x_positions, y_positions, linestyle='-', marker='o', markersize=10, color=node_color)
        ax.scatter([x_positions[0], x_positions[-1]], [y_height, y_height], color=start_end_color, zorder=5)
        
        # 노드 레이블 추가
        for x, label in zip(x_positions, path):
            ax.text(x, y_height - 0.1, label, ha='center', va='top', fontsize=9)

        # 다음 경로를 위해 y 위치 조정
        y_height -= 1

    # 그래프 스타일 설정
    ax.axis('off')  # 축 제거
    ax.set_ylim(-len(paths) - 1, 1)  # y축의 범위 설정
    ax.set_xlim(-1, fig_width + 1)  # x축의 범위 설정

    # 그래프 표시
    plt.tight_layout()  # 레이아웃 조정
    plt.show()



# 경로 타임라인 그리기 함수 호출
plot_timeline2(paths_hop_3, 0, 'grey', 'orange')


주어진 그래프에서 특정 출발 노드로부터 도달 가능한 노드들과 해당 노드들 간의 최단 경로를 시각화합니다.

In [ ]:
# source_node에서 도달 가능한 모든 노드 찾기
reachable_nodes = nx.descendants(G_80, source_node) | {source_node}

# source_node에서 각 노드까지의 최단 경로 찾기
shortest_paths = {node: nx.shortest_path(G_80, source_node, node) for node in reachable_nodes}

# 그래프 시각화
plt.figure(figsize=(10, 5))  # 그래프 크기 설정

# 모든 노드의 위치를 동일한 간격으로 설정
node_positions = {node: (i, 0) for i, node in enumerate(G_80.nodes())}

# 모든 노드와 엣지를 회색으로 그리기
nx.draw_networkx_nodes(G_80, node_positions, node_color='grey', alpha=0.5)
nx.draw_networkx_edges(G_80, node_positions, edge_color='grey', alpha=0.5)

# 각 최단 경로를 그리기
for path in shortest_paths.values():
    # 경로상의 노드 위치 지정
    path_positions = {node: node_positions[node] for node in path}
    # 시작, 종료 노드만 주황색으로 표시
    colors = ['orange' if node == source_node or node == path[-1] else 'grey' for node in path]
    nx.draw_networkx_nodes(G_80, path_positions, nodelist=path, node_color=colors)
    # 경로상의 엣지를 그리기
    nx.draw_networkx_edges(G_80, node_positions, edgelist=list(zip(path[:-1], path[1:])), edge_color='orange', width=2)

# 노드 레이블 추가
nx.draw_networkx_labels(G_80, node_positions)

# 축 제거
plt.axis('off')
plt.show()

## Heavy computation

주어진 그래프에서 특정 출발 노드로부터 도달 가능한 노드들과 해당 노드들 간의 최단 경로를 시각화합니다.

In [ ]:
# 가정: G는 이미 생성된 네트워크X의 DiGraph() 객체입니다.
# source_node는 시작 노드입니다.


# 특정 노드에서 도달할 수 있는 모든 노드 찾기
reachable_nodes = nx.descendants(G_80, source_node)

# 최단 경로 찾기
shortest_paths = {node: nx.shortest_path(G_80, source_node, node) for node in reachable_nodes}

# 새로운 그래프 생성 및 최단 경로의 노드와 엣지만 추가
H = nx.DiGraph()
for path in shortest_paths.values():
    nx.add_path(H, path)

# 최단 경로를 그래프에 표시
pos = nx.kamada_kawai_layout(G)  # 새로운 그래프의 노드 위치 결정
plt.figure(figsize=(60, 60))

# 노드와 엣지를 그립니다.
nx.draw_networkx_nodes(H, pos, node_color='lightblue')
nx.draw_networkx_edges(H, pos, edge_color='lightblue', width=2)
nx.draw_networkx_labels(H, pos)

# 시작 노드를 강조합니다.
nx.draw_networkx_nodes(H, pos, nodelist=[source_node], node_color='green', node_size=300)

plt.title('Shortest Paths from Source Node')
plt.axis('off')  # 축 제거
plt.show()


주어진 그래프에서 특정 출발 노드로부터 도달 가능한 노드들과 해당 노드들 간의 최단 경로를 시각화합니다.

In [ ]:
# 최단 경로를 그래프에 표시
pos = nx.spring_layout(G_80) #노드 위치결정
plt.figure(figsize=(60, 60))

# 모든 노드와 엣지를 그립니다.
nx.draw_networkx_nodes(G_80, pos, node_color='lightgray')
nx.draw_networkx_edges(G_80, pos, alpha=0.3)

# 각 최단 경로를 진하게 그립니다.
for path in shortest_paths.values():
    nx.draw_networkx_nodes(G_80, pos, nodelist=path, node_color='blue')
    nx.draw_networkx_edges(G_80, pos, edgelist=list(zip(path, path[1:])), edge_color='blue', width=2)
    nx.draw_networkx_labels(G_80, pos)

# 시작 노드를 강조합니다.
nx.draw_networkx_nodes(G_80, pos, nodelist=[source_node], node_color='green', node_size=300)

plt.title('Shortest Paths from Source Node')
plt.axis('off')  # 축 제거
plt.show()


주어진 그래프에서 특정 출발 노드로부터 도달 가능한 노드 및 엣지를 강조하여 시각화합니다.

In [ ]:
# 가정: G는 이미 생성된 네트워크X의 DiGraph() 객체입니다.
# source_node는 시작 노드입니다.


# source_node에서 도달할 수 있는 모든 노드와 엣지를 구합니다.
reachable_nodes = nx.descendants(G, source_node) | {source_node}
reachable_edges = [(u, v) for u, v in G.edges() if u == source_node or v == source_node or u in reachable_nodes or v in reachable_nodes]

# 모든 노드와 엣지에 대한 색상과 투명도 설정
node_color = []
node_alpha = []
for node in G.nodes():
    if node in reachable_nodes:
        node_color.append('lightblue')  # 도달 가능한 노드는 진한 색
        node_alpha.append(1.0)
    else:
        node_color.append('gray')  # 나머지 노드는 다른 색과 투명도 0.7
        node_alpha.append(0.7)

edge_color = []
edge_alpha = []
for edge in G.edges():
    if edge in reachable_edges:
        edge_color.append('lightblue')  # 도달 가능한 엣지는 진한 색
        edge_alpha.append(1.0)
    else:
        edge_color.append('gray')  # 나머지 엣지는 다른 색과 투명도 0.7
        edge_alpha.append(0.7)

# 그래프 시각화
pos = nx.spring_layout(G)  # 노드 위치 결정

# 노드와 엣지 그리기
plt.figure(figsize=(60,60))

nx.draw_networkx_nodes(G, pos, node_color=node_color, alpha=node_alpha)
nx.draw_networkx_edges(G, pos, edge_color=edge_color, alpha=edge_alpha)
nx.draw_networkx_labels(G, pos)  # 노드 레이블 그리기

plt.title('Graph Visualization with Reachable Nodes and Edges Highlighted')
plt.axis('off')  # 축 제거
plt.show()


주어진 그래프에서 노드와 엣지를 강조하여 시각화합니다.

In [ ]:
# 노드와 엣지 그리기
plt.figure(figsize=(60,60)) #그림의 크기 설정

nx.draw_networkx_nodes(G, pos, node_color=node_color, alpha=node_alpha) #노드의 색상과 투명도 결정
nx.draw_networkx_edges(G, pos, edge_color=edge_color, alpha=edge_alpha) #엣지의 색상과 투명도 결정
nx.draw_networkx_labels(G, pos)  # 노드 레이블 그리기

plt.title('Graph Visualization with Reachable Nodes and Edges Highlighted')
plt.axis('off')  # 축 제거
plt.show()

### 네트워크 분석

네트워크 분석 및 시뮬레이션을 수행합니다.
<br>1. 최단경로분석
<br>2. 탄력성 및 견고성 분석
<br>3. 네트워크 시뮬레이션

(*전체 출력을 원하시면 아래에 주석 처리된 코드를 실행하시면 됩니다.)

In [ ]:
# Path Analysis: Find the shortest path between two nodes.
# For this example, let's find the shortest path from 'FDA' to 'Belgium'
#최단경로 분석: 그래프 G에서 'FDA'에서 'Belgium'까지의 최단 경로
shortest_path = nx.shortest_path(G, source='FDA', target='Belgium')

# Robustness & Resilience Analysis: Remove a node and check the effect on the network.
# For this example, let's remove 'U.S.' and see how it affects the connectivity.
# Robustness & Resilience Analysis: U.S.' 노드를 제거한 후 그래프에서 연결된 구성 요소를 식별
G.remove_node('U.S.')
connected_components_after_removal = list(nx.connected_components(G.to_undirected()))

# Network Simulation: Simulate scenarios by adding or removing nodes or edges.
# For example, let's see how adding a new connection affects the shortest path.
# Network Simulation: 'Health Care'와 'Belgium' 노드 간에 새로운 엣지를 추가하여 새로운 연결을 시뮬레이션 -> 새로운 최단경로
G.add_edge('Health Care', 'Belgium')
new_shortest_path = nx.shortest_path(G, source='Health Care', target='Belgium')

# Display the results
#shortest_path, connected_components_after_removal, new_shortest_path
print(shortest_path)

# Print the first 10 connected components
for i, component in enumerate(connected_components_after_removal[1:11], start=1):
    print(f"Connected Component {i}: {component}")
    
print(new_shortest_path)

In [ ]:
'''
# Path Analysis: Find the shortest path between two nodes.
# For this example, let's find the shortest path from 'FDA' to 'Belgium'
#최단경로 분석: 그래프 G에서 'FDA'에서 'Belgium'까지의 최단 경로
shortest_path = nx.shortest_path(G, source='FDA', target='Belgium')

# Robustness & Resilience Analysis: Remove a node and check the effect on the network.
# For this example, let's remove 'U.S.' and see how it affects the connectivity.
# Robustness & Resilience Analysis: U.S.' 노드를 제거한 후 그래프에서 연결된 구성 요소를 식별
G.remove_node('U.S.')
connected_components_after_removal = list(nx.connected_components(G.to_undirected()))

# Network Simulation: Simulate scenarios by adding or removing nodes or edges.
# For example, let's see how adding a new connection affects the shortest path.
# Network Simulation: 'Health Care'와 'Belgium' 노드 간에 새로운 엣지를 추가하여 새로운 연결을 시뮬레이션 -> 새로운 최단경로
G.add_edge('Health Care', 'Belgium')
new_shortest_path = nx.shortest_path(G, source='Health Care', target='Belgium')

# Display the results
shortest_path, connected_components_after_removal, new_shortest_path
'''

주어진 데이터프레임을 기반으로 필터링된 그래프를 생성하고, 특정 노드를 선택한 후 해당 노드로부터 도달 가능한 모든 노드를 찾는 과정을 수행합니다.

In [ ]:
df = SSAN_SP_DF

In [ ]:
# 'source_ner' 및 'target_ner' 열이 'Country'가 아닌 경우에 해당하는 행만을 포함하는 데이터프레임을 생성
filtered_df = df[~df['source_ner'].isin(['Country']) & 
                 ~df['target_ner'].isin(['Country'])]

# 필터링된 데이터프레임을 기반으로 그래프 다시 생성
filtered_G = nx.from_pandas_edgelist(filtered_df, source='source_rep_85', target='target_rep_85', edge_attr='edge', create_using=nx.DiGraph())

# Now let's choose a node. For this example, we'll use 'the U.S. Food and Drug Administration' as the source node.
# Now let's choose a node. For this example, we'll use 'Tesla, Inc.' as the source node.
#출발노드 선택
source_node = 'Tesla, Inc.'

# Get all nodes reachable from the source node
#도달가능한 노드 및 거리 계산
reachable_nodes = nx.single_source_shortest_path_length(filtered_G, source_node)

#도달가능한 모든 노드 목록 출력
reachable_nodes.keys()  # This will list all the nodes that can be reached from the source node.


'Country'를 제외한 노드에 대한 그래프를 생성하고, 특정 노드로부터 도달 가능한 모든 노드 중에서 각(Tesla, Apple, ...) 회사에 속하지 않는 노드를 찾는 과정을 수행합니다.

In [ ]:
# Country를 제외한 노드에 대한 그래프 생성
filtered_df = df[~df['source_ner'].isin(['Country']) & ~df['target_ner'].isin(['Country'])]

# 필터링된 데이터프레임을 기반으로 그래프 다시 생성
filtered_G = nx.from_pandas_edgelist(filtered_df, source='source_rep_85', target='target_rep_85', edge_attr='edge', create_using=nx.DiGraph())

# Now let's choose a node. For this example, we'll use 'Tesla, Inc.' as the source node.
#출발 노드 선택
source_node = 'Tesla, Inc.'


# 도달 가능한 모든 노드 구하기
reachable_nodes = nx.single_source_shortest_path_length(filtered_G, source_node)

# 'TSLA' 회사에 속하지 않는 노드 구하기
non_tsla_nodes = [] #'TSLA' 회사에 속하지 않는 노드를 저장할 빈 리스트를 생성
for node in reachable_nodes: #도달가능한 각 노드에 대해

    # 현재 노드에 해당하는 데이터프레임을 추출합니다.
    node_df = df[(df['source'] == node) | (df['target'] == node)]

    # 데이터프레임이 비어있지 않고, 'TSLA'가 아닌 회사인지 확인합니다.
    if not node_df.empty and all(node_df['company'] != 'TSLA'):
        non_tsla_nodes.append(node)
    else:
        print(node)

print(non_tsla_nodes)
#결론:  'Tesla, Inc.' 노드로부터 도달 가능하면서 'TSLA' 회사에 속하지 않는 모든 노드 찾기

In [ ]:
# Country를 제외한 노드에 대한 그래프 생성
filtered_df = df[~df['source_ner'].isin(['Country']) & ~df['target_ner'].isin(['Country'])]

# 필터링된 데이터프레임을 기반으로 그래프 다시 생성
filtered_G = nx.from_pandas_edgelist(filtered_df, source='source_rep_85', target='target_rep_85', edge_attr='edge', create_using=nx.DiGraph())

# Now let's choose a node. For this example, we'll use 'Apple Inc.' as the source node.
#출발 노드 선택: Apple Inc.
source_node = 'Apple Inc.'


# 도달 가능한 모든 노드 구하기
reachable_nodes = nx.single_source_shortest_path_length(filtered_G, source_node)

# 'AAPL' 회사에 속하지 않는 노드 구하기
non_AAPL_nodes = []
for node in reachable_nodes:

    # 현재 노드에 해당하는 데이터프레임을 추출합니다.
    node_df = df[(df['source'] == node) | (df['target'] == node)]

    # 데이터프레임이 비어있지 않고, 'AAPL'가 아닌 회사인지 확인합니다.
    if not node_df.empty and all(node_df['company'] != 'AAPL'):
        non_AAPL_nodes.append(node)
    else:
        print(node)

print(non_AAPL_nodes)

각 열에서 해당 조건을 만족하는 열의 빈도 출력합니다. (확인용)

In [ ]:
#'source_rep_85' 열의 소문자 형태가 'the european union'인 행을 선택하고, 해당 행들 중 'source_ner' 열의 값들을 계산하여 카운트
#source_ner 값의 빈도 출력
df[(df['source_rep_85'].str.lower().isin(['the european union']))]['source_ner'].value_counts()

In [ ]:
#'target_rep_85' 열의 소문자 형태가 'the european union'인 행을 선택하고, 해당 행들 중 'target_ner' 열의 값들을 계산하여 카운트
#target_ner의 빈도 출력
df[(df['target_rep_85'].str.lower().isin(['the european union']))]['target_ner'].value_counts()

각 열의 조건을 만족하는 행을 출력합니다. (확인용)

In [ ]:
#'source_rep_85' 열의 소문자 형태가 'european'인 행이거나 'target_rep_85' 열의 소문자 형태가 'the european union'인 행들을 선택
df[(df['source_rep_85'].str.lower().isin(['european'])) | (df['target_rep_85'].str.lower().isin(['the european union']))]

In [ ]:
# 'source_rep_85' 열의 소문자 형태가 'ge'인 행이거나 'target_rep_85' 열의 소문자 형태가 'ge'인 행들을 선택
df[(df['source_rep_85'].str.lower().isin(['ge'])) | (df['target_rep_85'].str.lower().isin(['ge']))]

주어진 데이터프레임을 기반으로 네트워크 그래프를 만듭니다. <br> 노드 중심성을 계산한 다음, 노드를 NER 태그에 따라 카테고리화합니다. <br> 각 카테고리에서 중심성이 높은 상위 10개의 노드를 선택하여 이 정보들을 DataFrame에 저장합니다.

In [ ]:

# Creating a directed graph
G = nx.DiGraph()

# Adding edges from the DataFrame
for _, row in df.iterrows():
    G.add_edge(row['source_rep_85'], row['target_rep_85'], edge=row['edge'])

# Calculate centrality measures (e.g., degree centrality)
centrality = nx.degree_centrality(G)

# Preparing to categorize nodes by their NER tags
node_categories = defaultdict(list)

# Categorizing nodes
for node in G.nodes:
    # Finding the node in the DataFrame and categorizing it
    if node in df['source_rep_85'].values:
        category = df[df['source_rep_85'] == node]['source_ner'].values[0]
    elif node in df['target_rep_85'].values:
        category = df[df['target_rep_85'] == node]['target_ner'].values[0]
    else:
        category = 'Unknown'
    
    node_categories[category].append(node)

# # Sorting nodes in each category by centrality
# 중심성이 높은 상위 10개 노드 top_centrality_by_category 딕션너리에 저장
top_centrality_by_category = {category: sorted(nodes, key=lambda x: centrality[x], reverse=True)[:10]
                              for category, nodes in node_categories.items()}

# top_centrality_by_category
# Converting the top_centrality_by_category dictionary to a DataFrame
# top_centrality_by_category기반으로 데이터프레임 생성
df_top_centrality = pd.DataFrame([{"Category": category, "Node": node, "Centrality": centrality[node]}
                                  for category, nodes in top_centrality_by_category.items()
                                  for node in nodes])

df_top_centrality

'Country'를 제외한 노드를 기반으로 방향성 있는 그래프를 생성하고, 지정된 출발 노드에서 도달 가능한 모든 노드를 찾은 후에는 'AAPL' 회사에 속하지 않는 노드들을 확인하고 해당 노드들의 목록을 출력하는 과정을 수행합니다.

In [ ]:
# Country를 제외한 노드에 대한 그래프 생성
filtered_df = df[~df['source_ner'].isin(['Country']) & ~df['target_ner'].isin(['Country'])]

# 필터링된 데이터프레임을 기반으로 그래프 다시 생성
filtered_G = nx.from_pandas_edgelist(filtered_df, source='source_rep_85', target='target_rep_85', edge_attr='edge', create_using=nx.DiGraph())

# Now let's choose a node. For this example, we'll use 'the U.S. Food and Drug Administration' as the source node.
# 출발노드 지정
#source_node = 'the U.S. Food and Drug Administration'
#print(filtered_G.nodes)
source_node = 'iPad'



# 도달 가능한 모든 노드 구하기
# 출발노드에서 도달가능한 모든 노드 찾아 그 거리 계산
reachable_nodes = nx.single_source_shortest_path_length(filtered_G, source_node)

# 'AAPL' 회사에 속하지 않는 노드 구하기
non_AAPL_nodes = []
for node in reachable_nodes:

    # 현재 노드에 해당하는 데이터프레임을 추출합니다.
    node_df = df[(df['source'] == node) | (df['target'] == node)]

    # 데이터프레임이 비어있지 않고, 'AAPL'이 아닌 회사인지 확인합니다.
    if not node_df.empty and all(node_df['company'] != 'AAPL'):
        non_AAPL_nodes.append(node)
    else:
        print(node)

print(non_AAPL_nodes)

find_single_points_of_failure(): 방향성 있는 그래프에서 단일 실패점을 찾는 함수를 정의합니다.
<br>(*상위 10개 데이터에 대해서만 출력하였습니다.)

In [ ]:
# 단일 실패점을 찾는 함수 정의
def find_single_points_of_failure(graph):
    spofs = []  # 단일 실패점 목록을 저장할 리스트
    for node in graph.nodes():
        H = graph.copy()  # 네트워크의 복사본을 생성
        H.remove_node(node)  # 현재 노드를 복사본에서 제거
        
        # 네트워크가 여전히 강하게 연결되어 있는지 확인
        # (방향성이 있는 그래프에 대한 강한 연결성 체크)
        if not nx.is_strongly_connected(H):
            spofs.append(node)  # 그래프가 여전히 강하게 연결되어 있지 않다면, 현재 노드는 단일 실패점
            
    return spofs

# 단일 실패점 찾기
# 단일 실패점을 찾아 single_points_of_failure에 저장
single_points_of_failure = find_single_points_of_failure(G)

# 결과 출력
print("First 10 elements in Single Points of Failure:", single_points_of_failure[:10])

thre(threshold) 별로 주어진 그래프에서 노드의 도달성 분석을 통해 영향력을 계산하고, 그 영향력에 따라 상위 30개 노드를 출력합니다.

네트워크 내에서 중요한 역할을 하는 노드를 식별합니다.

영향력 있는 노드 찾기: 도달성 분석을 통해 특정 노드에서 다른 노드들에게 얼마나 많은 영향을 미칠 수 있는지 확인할 수 있습니다. <br> 예를 들어, 노드 A에서 다른 모든 노드까지 도달할 수 있다면, 노드 A는 네트워크에서 중요한 역할을 하는 것으로 간주될 수 있습니다.

In [ ]:
# 주어진 데이터프레임 df에서 방향성 그래프 생성
G=nx.from_pandas_edgelist(df, source='source_rep_85', target='target_rep_85', edge_attr='edge', create_using=nx.DiGraph())
# 여기에 G에 노드와 엣지를 추가하는 코드를 넣어주세요.
# 추가했음
for _, row in df.iterrows():
    G.add_node(row['source_rep_85'])
    G.add_node(row['target_rep_85'])
    G.add_edge(row['source_rep_85'], row['target_rep_85'], edge=row['edge'])

# 노드별 영향력 점수 계산
influence_scores = {} #노드별 영향력 점수를 저장할 빈 딕셔너리를 생성
for node in G.nodes:
    reachable_nodes = nx.descendants(G, node) #현재 노드에서 도달가능한 모든 하위 노드 찾기
    influence_score = len(reachable_nodes) / (len(G.nodes) - 1) #도달 가능한 노드의 수를 전체 노드 수로 나누어 영향력 점수를 계산
    influence_scores[node] = influence_score #딕션너리에 점수 저장

# 영향력 점수를 기준으로 내림차순으로 상위 30개의 노드 추출
top_30_nodes = sorted(influence_scores, key=influence_scores.get, reverse=True)[:30]

# 결과 출력
for i, node in enumerate(top_30_nodes, start=1):
    print(node)

In [ ]:
# 네트워크 내에서 중요한 역할을 하는 노드 식별
# 데이터프레임을 기반으로 방향성 있는 그래프 G를 생성, source: 출발노드, target: 도착노드
G=nx.from_pandas_edgelist(df, source='source', target='target', edge_attr='edge', create_using=nx.DiGraph())
# 여기에 G에 노드와 엣지를 추가하는 코드를 넣어주세요.
# 추가했음
for _, row in df.iterrows():
    G.add_node(row['source'])
    G.add_node(row['target'])
    G.add_edge(row['source'], row['target'], edge=row['edge'])

# 노드별 영향력 점수 계산
influence_scores = {}
for node in G.nodes:
    reachable_nodes = nx.descendants(G, node) 
    influence_score = len(reachable_nodes) / (len(G.nodes) - 1)
    influence_scores[node] = influence_score

# 영향력 점수를 기준으로 상위 30개의 노드 추출
top_30_nodes = sorted(influence_scores, key=influence_scores.get, reverse=True)[:30]

# 결과 출력
for i, node in enumerate(top_30_nodes, start=1):
    print(node)

상위 30개 노드와 각 노드의 영향력 점수를 함께 출력합니다. (확인용)

In [ ]:
# 결과 출력
for i, node in enumerate(top_30_nodes, start=1):
    print(node, influence_scores[node])

'eOne'이라는 값이 'source_rep_85' 또는 'target_rep_85' 열에 포함된 행들을 필터링합니다. (확인용)

In [ ]:
df[(df['source_rep_85'].isin(['eOne'])) | (df['target_rep_85'].isin(['eOne']))]

그래프 (filtered_G)의 노드에 대한 영향력 점수를 계산하는 과정입니다. <br> 특정 조건에 따라 도달 가능한 노드를 출력하고, 영향력 점수를 기준으로 상위 30개 노드를 식별한 후 이를 출력합니다.

네트워크 내에서 중요한 역할을 하는 노드(영향력있는 노드) 식별합니다. <br>
영향력 있는 노드 찾기: 도달성 분석을 통해 특정 노드에서 다른 노드들에게 얼마나 많은 영향을 미칠 수 있는지 확인할 수 있습니다. <br>
예를 들어, 노드 A에서 다른 모든 노드까지 도달할 수 있다면, 노드 A는 네트워크에서 중요한 역할을 하는 것으로 간주될 수 있습니다.

In [ ]:
# 여기에 G에 노드와 엣지를 추가하는 코드를 넣어주세요.

# 노드별 영향력 점수 계산
influence_scores = {}
for node in filtered_G.nodes:
    reachable_nodes = nx.descendants(filtered_G, node)
    influence_score = len(reachable_nodes) / (len(filtered_G.nodes) - 1)
    # if influence_score>0.002:
    #     print(node, reachable_nodes)
    if node == 'U.S' or node == 'U.S':
        print(node, reachable_nodes)
    influence_scores[node] = influence_score

# 영향력 점수를 기준으로 상위 30개의 노드 추출
top_30_nodes = sorted(influence_scores, key=influence_scores.get, reverse=True)[:30]

# 결과 출력
for i, node in enumerate(top_30_nodes, start=1):
    print(f"Rank {i}: Node {node}, Influence Score: {influence_scores[node]}")

그래프 G에서 노드를 하나씩 제거하고, 제거된 노드로 인해 그래프의 강한 연결성이 깨지는지를 확인하는 과정입니다.
<br> (*출력값은 주석처리 하였습니다. 데이터를 원하시면 주석처리한 코드를 실행하시면 됩니다.)

In [ ]:
# 잠재적인 위험 요소 또는 취약점 찾기:
# 단일 실패점 찾기: 네트워크 내에서 특정 노드가 실패하거나 제거되었을 때, 다른 노드들 간의 연결성이 손실되는 경우가 있을 수 있습니다. 
# 이를 "단일 실패점"이라고 합니다. 도달성 분석을 통해 이러한 단일 실패점을 찾을 수 있습니다.

for node in G.nodes:
    H = G.copy()
    H.remove_node(node)
    #만약 H가 더 이상 강하게 연결되어 있지 않다면, '현재 노드를 제거하면 그래프의 연결성이 깨진다'는 메시지를 출력
    
    # 전체 데이터를 원하시면 아래의 코멘트 처리한 코드를 돌리시면 됩니다.
    #if not nx.is_strongly_connected(H):  # 노드 제거 후 네트워크가 더 이상 강하게 연결되어 있지 않음
        #print(f"Removing {node} breaks the network connectivity")
        

Louvain 알고리즘을 사용하여 네트워크를 여러 번 실행하고 최적의 커뮤니티 분할을 찾는 과정을 수행합니다. <br>
각 실행에서 모듈러리티를 계산하고, 최종적으로 최고의 모듈러리티를 갖는 파티션을 선택합니다.
<br>(*전체 데이터 출력을 원하시면 밑에 주석처리된 코드를 실행하시면 됩니다.)

In [ ]:
def print_first_10_elements(result):
    for item in list(result)[:10]:
        print(item)

partitions=[]
modularities=[]

G_undirected = G.to_undirected()

best_modularity = -1
best_partition = None

# 여러 번 실행하여 최적의 커뮤니티 분할 찾기
for seed in range(10):
    partition = community_louvain.best_partition(G_undirected, random_state=seed)
    modularity = community_louvain.modularity(partition, G_undirected)
    
    partitions.append(partition)
    modularities.append(modularity)
    
    print(f"Run {seed}: Modularity = {modularity:.4f}")
    
    if seed >= 9:
        break

# 가장 높은 모듈라리티를 가진 파티션 찾기
best_partition_index = np.argmax(modularities)
best_partition = partitions[best_partition_index]
best_modularity = modularities[best_partition_index]

print("\nBest Partition:")
print_first_10_elements(best_partition.items())
print(f"Best Modularity: {best_modularity:.4f}")

# 노드의 실제 이름을 사용하여 결과 출력
best_partition_named = {node: community for node, community in best_partition.items()}
print("\nBest Partition:")
print_first_10_elements(best_partition_named.items())
print("Best Modularity:", best_modularity)

# 커뮤니티 크기 계산
community_sizes = {}
for node, comm in best_partition_named.items():
    if comm not in community_sizes:
        community_sizes[comm] = 0
    community_sizes[comm] += 1

# 결과 출력
print("\nCommunity Sizes:")
print_first_10_elements(community_sizes.items())


In [ ]:
'''
# 커뮤니티 검출 (Community Detection):
# Louvain 메소드를 사용합니다.

#modularities와 partitions 리스트 선언
partitions=[]
modularities=[]

# community_louvain only takes undirected graphs
G_undirected = G.to_undirected()

# 여러 번 실행하여 최적의 커뮤니티 분할 찾기
best_modularity = -1 #최적의 모듈러리티를 저장하는 변수 초기화
best_partition = None #최적의 분할을 저장하는 변수 초기화
# 여러 번 실행
for seed in range(10):
    # Louvain 커뮤니티 검출 실행
    # Louvain 알고리즘을 실행하고 현재의 시드값으로 커뮤니티 파티션을 얻음
    partition = community_louvain.best_partition(G_undirected, random_state=seed)
    #얻은 파티션에 대한 모듈러리티 계산
    modularity = community_louvain.modularity(partition, G_undirected)
    
    # 결과 저장
    partitions.append(partition)
    modularities.append(modularity)
    
    print(f"Run {seed}: Modularity = {modularity:.4f}")

# 가장 높은 모듈라리티를 가진 파티션 찾기
best_partition_index = np.argmax(modularities)
best_partition = partitions[best_partition_index]
best_modularity = modularities[best_partition_index]

print("\nBest Partition:")
print(best_partition)
print(f"Best Modularity: {best_modularity:.4f}")

#G_undirected에 대해서도 반복
for i in range(10):
    partition = community_louvain.best_partition(G_undirected, random_state=i)
    modularity = community_louvain.modularity(partition, G_undirected)
    print(f"Run {i}: Modularity = {modularity:.4f}")
    
    if modularity > best_modularity:
        best_modularity = modularity
        best_partition = partition

# 노드의 실제 이름을 사용하여 결과 출력
best_partition_named = {node: community for node, community in best_partition.items()}
print("\nBest Partition:")
print(best_partition_named)
print("Best Modularity:", best_modularity)

# 커뮤니티 크기 계산
community_sizes = {}
for node, comm in best_partition_named.items():
    if comm not in community_sizes:
        community_sizes[comm] = 0
    community_sizes[comm] += 1

# 결과 출력
for comm, size in community_sizes.items():
    print(f"Community {comm}: {size} nodes")
    
'''    

# 6. SSAN+GPT Graph Visualization

SSAN 모델을 통해 분석한 데이터와 GPT를 통해 획득한 데이터를 전처리하여 merge한 파일을 import합니다. 각 소스노드에서 타겟노드에 도달하는 최단거리를 파악합니다. 마지막으로 4. analysis와 마찬가지로 데이터프레임에서 threshold별, node별로 출발지와 목적지 정보를 기반으로 그래프를 생성하고 시각화하는 기능을 수행합니다. 이를 통해 supply chain network 데이터를 분석하고 시각화하는 기능을 수행할 수 있습니다.

파일을 import합니다.

In [ ]:
df = pd.read_csv('ALL_df.csv')
df_95 = pd.read_csv('ALL_df_95.csv')
df_90 = pd.read_csv('ALL_df_90.csv')
df_85 = pd.read_csv('ALL_df_85.csv')
df_80 = pd.read_csv('ALL_df_80.csv')

pickle 파일 import합니다.

In [ ]:
# Pickle 파일로부터 그래프 로드
with open('graph.pickle', 'rb') as f:
    G = pickle.load(f)

with open('graph_95.pickle', 'rb') as f:
    G_95 = pickle.load(f)

with open('graph_90.pickle', 'rb') as f:
    G_90 = pickle.load(f)

with open('graph_85.pickle', 'rb') as f:
    G_85 = pickle.load(f)

with open('graph_80.pickle', 'rb') as f:
    G_80 = pickle.load(f)

find_paths_to_targets: 방향성이 있는 그래프 G에서 주어진 대상 노드 목록으로부터 들어오는 모든 경로를 찾아주는 함수를 정의합니다.

In [ ]:
def find_paths_to_targets(G, target_nodes):
    all_paths = {} #모든 경로를 저장할 빈 딕셔너리를 초기화
    for target in target_nodes: #대상 노드 목록에 대해 반복
        # 해당 대상 노드로 들어오는 모든 경로 찾기
        paths_to_target = [] #현재 대상 노드로 들어오는 경로를 저장할 빈 리스트를 초기화
        for source in G.nodes(): #모든 노드에 대해 반복
            if source != target: #출발 노드와 대상 노드가 같지 않은 경우에 실행
                paths = list(nx.all_simple_paths(G, source, target)) #현재 출발 노드에서 대상 노드까지의 모든 단순 경로 탐색
                if paths:
                    paths_to_target.extend(paths) #찾은 경로가 있다면 찾은 경로를 paths_to_target 리스트에 추가
        
        # 찾은 경로를 결과 딕셔너리에 추가
        if paths_to_target:
            all_paths[target] = paths_to_target #현재 대상 노드를 key로, 대상 노드로 들어오는 모든 경로를 value로 저장
    
    return all_paths

기존의 피클파일이 너무 커서 결과값 출력 간소화를 위해 원본 데이터에서 샘플링을 수행하였습니다.<br> 
전체 데이터로 처리한 결과값을 보려면 아래의 주석처리된 셀을 실행하시면 됩니다. 

In [ ]:
# sampling due to the file size
def G_sampling(pickle, sample_size):
    random.seed(123)
    G = pickle
    sample_size = sample_size
    num_nodes_to_sample = int(sample_size * G.number_of_nodes())
    sampled_nodes = random.sample(list(G.nodes()), num_nodes_to_sample)
    G_sample = G.subgraph(sampled_nodes)
    return G_sample

In [ ]:
# sampled pickle files

G_sample = G_sampling(G, 0.1)
G_95_sample = G_sampling(G_95, 0.1)
G_90_sample = G_sampling(G_90, 0.1)
G_85_sample = G_sampling(G_85, 0.1)
G_80_sample = G_sampling(G_80, 0.1)


In [ ]:
# simplified version with the sampled data
# 예시 대상 노드 목록 (실제 대상 노드로 변경 필요)
target_nodes = ['U.S.', 'FDA']
#target_nodes = ['TargetNode1', 'TargetNode2']


found_paths = find_paths_to_targets(G_sample, target_nodes)

# 결과 출력
print(found_paths)


In [ ]:
# heavy computation
"""
# 예시 대상 노드 목록 (실제 대상 노드로 변경 필요)
target_nodes = ['U.S.', 'FDA']
#target_nodes = ['TargetNode1', 'TargetNode2']


found_paths = find_paths_to_targets(G, target_nodes)

# 결과 출력
print(found_paths)
"""

find_path는 그래프 G와 가능한 출발 노드 목록 possible_sources로부터 각 출발 노드에서 다른 노드로의 최단 경로를 찾아주는 역할을 하는 함수입니다.(중복)
find_node는 df에서 source 또는 target 열에 주어진 name을 포함하는 특정 노드를 찾아주는 역할을 합니다.

In [ ]:
def find_path(G, possible_sources):
    shortest_paths = {}
    # 그래프 노드 중에서 실제 존재하는 source_node 찾기
    for source in possible_sources: #가능한 출발노드 목록에 대해 반복
        #현재 주어진 source가 실제 그래프 노드에 존재하는지 확인하고, 존재한다면 actual_source에 해당 노드를 할당
        actual_source = next((node for node in G.nodes() if node == source), None)
        if actual_source:
            reachable_nodes = nx.descendants(G, actual_source)
            tmp_shortest_paths = {node: nx.shortest_path(G, actual_source, node) for node in reachable_nodes}
            
            print("Exist", actual_source)
            shortest_paths = {**shortest_paths, **tmp_shortest_paths}
        # print(source)
        
    return shortest_paths

def find_node(name):
    # 데이터프레임에서 source 열에 주어진 name을 포함하는 행을 필터링하고, 해당하는 source 값을 고유하게 추출
    unique_values_source = df[df['source'].str.contains(name, case=False)]['source'].unique()
    # 데이터프레임에서 target 열에 주어진 name을 포함하는 행을 필터링하고, 해당하는 target 값을 고유하게 추출
    unique_values_target = df[df['target'].str.contains(name, case=False)]['target'].unique()
    # 결과 출력
    
    return (set(list(unique_values_source) + list(unique_values_target))) 

# shortest_path = nx.shortest_path(G, source='Newmont Corporation', target=5)

print(find_node('DuPont'))

 possible_sources 리스트의 노드에서 시작하여, 해당 노드로 가는 모든 최단 경로를 찾고, 특정 조건에 따라 필터링합니다.<br>
 (전체데이터를 출력하시려면 밑에 코드를 실행하시면 됩니다.)

In [ ]:
def print_filtered_paths(filtered_paths):
    count = 0
    for key, path in filtered_paths.items():
        if count >= 10:  # 처음 10개 항목까지만 출력
            break
        print(key, ":", path)
        count += 1

possible_sources = ['DuPont Asian Group', 'DuPont Teijin Films', 'The Dow Chemical DuPont','DuPont']
exclude_words = ['sec', 'covid','board of directors','results of operations','esg','consolidated financial statements',
                 'pcaob']

# find_path 함수를 사용하여 주어진 possible_sources에서 시작하는 모든 최단 경로 계산
shortest_paths = find_path(G, possible_sources)

# 최단 경로 중에서 exclude_words에 해당하는 단어를 포함하는 노드를 제외하고, 나머지 경로들을 filtered_paths에 저장
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if all(not any(exclude_word in node.lower() for exclude_word in exclude_words) for node in path)
}

print("Filtered Paths:")
print_filtered_paths(filtered_paths)


In [ ]:
'''실제코드 입니다.
#최단 경로를 찾을 시작 노드들로 구성된 리스트
possible_sources = ['DuPont Asian Group', 'DuPont Teijin Films', 'The Dow Chemical DuPont','DuPont']
#경로에서 제외할 단어들로 구성된 리스트
exclude_words = ['sec', 'covid','board of directors','results of operations','esg','consolidated financial statements',
                 'pcaob']

#find_path 함수를 사용하여 주어진 possible_sources에서 시작하는 모든 최단 경로 계산
shortest_paths =find_path(G, possible_sources)

#최단 경로 중에서 exclude_words에 해당하는 단어를 포함하는 노드를 제외하고, 나머지 경로들을 filtered_paths에 저장
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if all(not any(exclude_word in node.lower() for exclude_word in exclude_words) for node in path)
    # if len(path) == 3 and any(material == path[2].lower() for material in materials)
}

print(filtered_paths)
'''

possible_sources 리스트에 있는 노드에서 시작하여, 해당 노드로 가는 모든 최단 경로를 찾은 다음 특정 조건을 만족하는 경로들을 필터링합니다.

In [ ]:
#최단 경로를 찾을 시작 노드들로 구성된 리스트
possible_sources = ['BHP Group', 'BHP Copper, Inc']

#find_path 함수를 사용하여 주어진 possible_sources에서 시작하는 모든 최단 경로 탐색
shortest_paths =find_path(G, possible_sources)

#최단 경로 중에서 해당 경로에 포함된 노드 중 적어도 하나가 'Tesla'를 포함하면 해당 경로를 filtered_paths에 저장
filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any('Tesla' in node for node in path) #.lower()
    # if len(path) == 3 and any(material == path[2].lower() for material in materials)
}

print(filtered_paths)

In [ ]:
possible_sources = ['Qualcomm', 'Qualcomm Incorporated', 'QUALCOMM Incorporated', 'Qualcomm Technologies, Inc.']

shortest_paths =find_path(G, possible_sources)

filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any('Samsung' in node for node in path) #.lower()
    # if len(path) == 3 and any(material == path[2].lower() for material in materials)
}

print(filtered_paths)

In [ ]:
possible_sources = ['Foxconn Technology Group']

shortest_paths =find_path(G, possible_sources)

filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any('Best Buy' in node for node in path) #.lower()
    # if len(path) == 3 and any(material == path[2].lower() for material in materials)
}

print(filtered_paths)

In [ ]:
possible_sources = ['Rio Tinto plc']

shortest_paths =find_path(G, possible_sources)

filtered_paths = {
    key: path for key, path in shortest_paths.items()
    if any('Apple' in node for node in path) #.lower()
    # if len(path) == 3 and any(material == path[2].lower() for material in materials)
}

print(filtered_paths)

Graph_and_Visual(): 각 노드간의 관계그래프를 시각화하는 함수를 정의합니다.<br>(앞서 정의한 Graph_and_Visual과 파일을 저장하는 과정에 있어 약간의 차이가 존재합니다.)

In [ ]:
def Graph_and_Visual(fin_df, source, target, isin_ner_li, Title, thre, pos):
    # 'Country' 또는 'Firm'으로 태그된 노드만 포함하는 행을 필터링
    filtered_df = fin_df[(fin_df['source_ner'].isin(isin_ner_li)) & (fin_df['target_ner'].isin(isin_ner_li))]

    # 필터링된 데이터프레임을 기반으로 그래프 다시 생성
    filtered_G = nx.from_pandas_edgelist(filtered_df, source=source, target=target, edge_attr='edge', create_using=nx.DiGraph())
    
    # 색상 매핑 (이전에 정의된 'Country'와 'Firm'에 대한 색상만 사용)
    color_map_filtered = {'Country': 'royalblue', 'Firm': 'lightgreen','Resource': 'orange','Technology': 'darkred'}

    # 필터링된 그래프의 노드에 NER 태그 속성 추가
    for _, row in filtered_df.iterrows():
        if row[source] in filtered_G.nodes:
            filtered_G.nodes[row[source]]['ner'] = row['source_ner']
        if row[target] in filtered_G.nodes:
            filtered_G.nodes[row[target]]['ner'] = row['target_ner']

    # 노드 색상 할당 (필터링된 그래프에 대해)
    filtered_node_colors = [color_map_filtered[filtered_G.nodes[node]['ner']] for node in filtered_G.nodes]

    # 그래프 시각화
    plt.figure(figsize=(60,60))
    nx.draw(filtered_G, with_labels=True, node_color=filtered_node_colors, edge_color='gray', pos=pos, font_size=3, node_size=100)
    plt.title("Supply Chain Graph ({})".format(Title), fontdict={'fontsize': 100})
    plt.savefig("./ALL_graph_{}_{}.png".format(Title,thre))
    # plt.show()
    return filtered_G

df_95, df_90, df_85, df_80 데이터프레임을 기존의 df 데이터프레임에 추가한 후 중복된 행을 제거하는 과정을 수행합니다.(확인용)

In [ ]:
#수정했음
tmp_df = pd.concat([df, df_95], ignore_index=True)
tmp_df = tmp_df.drop_duplicates().reset_index(drop=True)

tmp_df = pd.concat([tmp_df, df_90], ignore_index=True)
tmp_df = tmp_df.drop_duplicates().reset_index(drop=True)

tmp_df = pd.concat([tmp_df, df_85], ignore_index=True)
tmp_df = tmp_df.drop_duplicates().reset_index(drop=True)

tmp_df = pd.concat([tmp_df, df_80], ignore_index=True)
tmp_df = tmp_df.drop_duplicates().reset_index(drop=True)

In [ ]:
#수정했음(빠른 컴파일, 원래 코드)
tmp_df = pd.concat([df, df_95], ignore_index=True)
tmp_df = tmp_df.drop_duplicates().reset_index(drop=True)

tmp_df = pd.concat([df, df_90], ignore_index=True)
tmp_df = tmp_df.drop_duplicates().reset_index(drop=True)

tmp_df = pd.concat([df, df_85], ignore_index=True)
tmp_df = tmp_df.drop_duplicates().reset_index(drop=True)

tmp_df = pd.concat([df, df_80], ignore_index=True)
tmp_df = tmp_df.drop_duplicates().reset_index(drop=True)

### Data visualization

thre(threshold) 별로 Graph_and_Visual 함수를 호출하여 데이터프레임 df를 기반으로 그래프를 생성하고 시각화합니다. <br>이후 statistic 함수를 호출하여 생성된 그래프의 통계적 정보를 출력합니다.

In [ ]:
#데이터프레임을 이용하여 그래프를 생성, source와 target: 엣지의 출발 노드와 도착 노드, edge_attr: 엣지의 속성, create_using: 사용할 그래프 유형을 지정
G_raw = nx.from_pandas_edgelist(df, source='source', target='target', edge_attr='edge', create_using=nx.DiGraph())
#그래프를 시각화하기 위한 레이아웃을 생성하는 함수 -> 힘 기반의 레이아웃 알고리즘을 사용하여 그래프의 노드들을 배치
#pos 변수에 spring_layout로 얻은 레이아웃이 저장 -> 레이아웃은 그래프를 시각화할 때 각 노드의 위치를 지정
pos = nx.spring_layout(G_raw)

In [ ]:
# F-R-T
#데이터프레임 df를 이용하여 그래프를 생성하고 시각화
G = Graph_and_Visual(df, 'source', 'target', ['Firm','Resource','Technology'], 'Firm and Resource and Technology','raw',pos)
#그래프 G의 통계적 정보를 출력
statistic(G)

In [ ]:
# C-C
G = Graph_and_Visual(df, 'source', 'target', ['Country'], 'Country','raw',pos)
statistic(G)

In [ ]:
# C-F
G = Graph_and_Visual(df, 'source', 'target', ['Country','Firm'], 'Country and Firm','raw',pos)
statistic(G)
# C-R
G = Graph_and_Visual(df, 'source', 'target', ['Country','Resource'], 'Country and Resource','raw',pos)
statistic(G)
# C-T
G = Graph_and_Visual(df, 'source', 'target', ['Country','Technology'], 'Country and Technology','raw',pos)
statistic(G)
# F-F
G = Graph_and_Visual(df, 'source', 'target', ['Firm'], 'Firm','raw',pos)
statistic(G)
# F-R
G = Graph_and_Visual(df, 'source', 'target', ['Firm','Resource'], 'Firm and Resource','raw',pos)
statistic(G)
# F-T
G = Graph_and_Visual(df, 'source', 'target', ['Firm','Technology'], 'Firm and Technology','raw',pos)
statistic(G)
# R-R
G = Graph_and_Visual(df, 'source', 'target', ['Resource'], 'Resource','raw',pos)
statistic(G)
# R-T
G = Graph_and_Visual(df, 'source', 'target', ['Resource','Technology'], 'Resource and Technology','raw',pos)
statistic(G)
# T-T
G = Graph_and_Visual(df, 'source', 'target', ['Technology'], 'Technology','raw',pos)
statistic(G)

In [ ]:
G_95 = nx.from_pandas_edgelist(df_95, source='source', target='target', edge_attr='edge', create_using=nx.DiGraph())
pos_95 = nx.spring_layout(G_95)

In [ ]:
# C-C
G = Graph_and_Visual(df_95, 'source', 'target', ['Country'], 'Country','95',pos_95)
statistic(G)

In [ ]:
# C-F
G = Graph_and_Visual(df_95, 'source', 'target', ['Country','Firm'], 'Country and Firm','95',pos_95)
statistic(G)
# C-R
G = Graph_and_Visual(df_95, 'source', 'target', ['Country','Resource'], 'Country and Resource','95',pos_95)
statistic(G)
# C-T
G = Graph_and_Visual(df_95, 'source', 'target', ['Country','Technology'], 'Country and Technology','95',pos_95)
statistic(G)
# F-F
G = Graph_and_Visual(df_95, 'source', 'target', ['Firm'], 'Firm','95',pos_95)
statistic(G)
# F-R
G = Graph_and_Visual(df_95, 'source', 'target', ['Firm','Resource'], 'Firm and Resource','95',pos_95)
statistic(G)
# F-T
G = Graph_and_Visual(df_95, 'source', 'target', ['Firm','Technology'], 'Firm and Technology','95',pos_95)
statistic(G)
# R-R
G = Graph_and_Visual(df_95, 'source', 'target', ['Resource'], 'Resource','95',pos_95)
statistic(G)
# R-T
G = Graph_and_Visual(df_95, 'source', 'target', ['Resource','Technology'], 'Resource and Technology','95',pos_95)
statistic(G)
# T-T
G = Graph_and_Visual(df_95, 'source', 'target', ['Technology'], 'Technology','95',pos_95)
statistic(G)

In [ ]:
G_90 = nx.from_pandas_edgelist(df_90, source='source', target='target', edge_attr='edge', create_using=nx.DiGraph())
pos_90 = nx.spring_layout(G_90)

In [ ]:
# C-C
G = Graph_and_Visual(df_90, 'source', 'target', ['Country'], 'Country','90',pos_90)
statistic(G)

In [ ]:
# C-F
G = Graph_and_Visual(df_90, 'source', 'target', ['Country','Firm'], 'Country and Firm','90',pos_90)
statistic(G)
# C-R
G = Graph_and_Visual(df_90, 'source', 'target', ['Country','Resource'], 'Country and Resource','90',pos_90)
statistic(G)
# C-T
G = Graph_and_Visual(df_90, 'source', 'target', ['Country','Technology'], 'Country and Technology','90',pos_90)
statistic(G)
# F-F
G = Graph_and_Visual(df_90, 'source', 'target', ['Firm'], 'Firm','90',pos_90)
statistic(G)
# F-R
G = Graph_and_Visual(df_90, 'source', 'target', ['Firm','Resource'], 'Firm and Resource','90',pos_90)
statistic(G)
# F-T
G = Graph_and_Visual(df_90, 'source', 'target', ['Firm','Technology'], 'Firm and Technology','90',pos_90)
statistic(G)
# R-R
G = Graph_and_Visual(df_90, 'source', 'target', ['Resource'], 'Resource','90',pos_90)
statistic(G)
# R-T
G = Graph_and_Visual(df_90, 'source', 'target', ['Resource','Technology'], 'Resource and Technology','90',pos_90)
statistic(G)
# T-T
G = Graph_and_Visual(df_90, 'source', 'target', ['Technology'], 'Technology','90',pos_90)
statistic(G)

In [ ]:
G_85 = nx.from_pandas_edgelist(df_85, source='source', target='target', edge_attr='edge', create_using=nx.DiGraph())
pos_85 = nx.spring_layout(G_85)

In [ ]:
# C-C
G = Graph_and_Visual(df_85, 'source', 'target', ['Country'], 'Country','85',pos_85)
statistic(G)

In [ ]:
# C-F
G = Graph_and_Visual(df_85, 'source', 'target', ['Country','Firm'], 'Country and Firm','85',pos_85)
statistic(G)
# C-R
G = Graph_and_Visual(df_85, 'source', 'target', ['Country','Resource'], 'Country and Resource','85',pos_85)
statistic(G)
# C-T
G = Graph_and_Visual(df_85, 'source', 'target', ['Country','Technology'], 'Country and Technology','85',pos_85)
statistic(G)
# F-F
G = Graph_and_Visual(df_85, 'source', 'target', ['Firm'], 'Firm','85',pos_85)
statistic(G)
# F-R
G = Graph_and_Visual(df_85, 'source', 'target', ['Firm','Resource'], 'Firm and Resource','85',pos_85)
statistic(G)
# F-T
G = Graph_and_Visual(df_85, 'source', 'target', ['Firm','Technology'], 'Firm and Technology','85',pos_85)
statistic(G)
# R-R
G = Graph_and_Visual(df_85, 'source', 'target', ['Resource'], 'Resource','85',pos_85)
statistic(G)
# R-T
G = Graph_and_Visual(df_85, 'source', 'target', ['Resource','Technology'], 'Resource and Technology','85',pos_85)
statistic(G)
# T-T
G = Graph_and_Visual(df_85, 'source', 'target', ['Technology'], 'Technology','85',pos_85)
statistic(G)

## 결과해석부

결과적으로 그래프의 형태를 보면 임계값이 낮아짐에 따라 시각적으로 노드 간의 연결이 더 많아지는 것을 볼 수 있습니다. 또한, 그래프의 형태를 살펴보면 허브와 스포크 구조의 특징이 나타납니다. 이는 여러 중심 노드가 많은 다른 노드와 연결되어 있으며, 이러한 중심 노드들이 전체 네트워크를 구성하는 데 상대적으로 중요한 역할을 한다는 것을 알 수 있습니다.

In [ ]:
!jupyter nbconvert --to html final_notebook.ipynb